### Naive pattern matching algorithm 

In [14]:
def naive(text, pattern, _ = None):
    matches = []
    for i in range(len(text)-len(pattern)+1):
        if text[i:i+len(pattern)] == pattern:
            matches.append(i)
    return matches

In [15]:
naive("abccd","cc")

[2]

### Finite automata

In [4]:
import re

def transition_table(pattern):
    t_table = []
    alphabet = set(pattern)
    
    for q in range(0, len(pattern) + 1):
        
        t_table.append({})
        for symbol in alphabet:
            
            k = min(len(pattern) + 1, q + 1)
            suffix = pattern[:q] + symbol
            
            while k >= 0 and pattern[:k] != suffix[q - k + 1:]:
                k = k - 1
                #print(k)
            t_table[q][symbol] = k
    return t_table

def finite_automata(text, pattern, preprocessing = None):
    matches = []
    q = 0
    
    if preprocessing is None:
        t_table = transition_table(pattern)
    else:
        t_table = preprocessing
    
    for s in range(len(text)):
        if text[s] in t_table[q]:
            q = t_table[q][text[s]]
            if q == len(t_table) - 1:
                matches.append(s+1-q)
                # s + 1 - ponieważ przeczytaliśmy znak o indeksie s, więc przesunięcie jest po tym znaku
        else:
            q=0
            
    return matches

In [5]:
transition_table("ababac")

[{'a': 1, 'c': 0, 'b': 0},
 {'a': 1, 'c': 0, 'b': 2},
 {'a': 3, 'c': 0, 'b': 0},
 {'a': 1, 'c': 0, 'b': 4},
 {'a': 5, 'c': 0, 'b': 0},
 {'a': 1, 'c': 6, 'b': 4},
 {'a': 1, 'c': 0, 'b': 0}]

In [6]:
len(finite_automata("ababbababbvcdaba"*10, "aba"))

30

### KMP algorithm 

In [8]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    return pi

def kmp_string_matching(text, pattern, preprocessing = None):
    if preprocessing is None:
        pi = prefix_function(pattern)
    else:
        pi = preprocessing
    q = 0
    matches = []
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            matches.append(i+1-q)
            q = pi[q-1]
            
    return matches

In [9]:
prefix_function("ababaca")

[0, 0, 1, 2, 3, 0, 1]

In [10]:
print(kmp_string_matching("abaabaaaaba", "aba"))

[0, 3, 8]


#### Timer function

In [11]:
from time import time
def timer(algorithm, text, pattern, preprocessing = None):
    t1 = time()
    algorithm(text, pattern, preprocessing)
    t2 = time()
    print("Czas wykonania algorytmu:" + str(t2-t1) + "s")

### Time comparition

In [19]:
# RUN CELL WITH ARTICLE BEFORE
timer(naive, article, "art")

Czas wykonania algorytmu:0.05187034606933594s


In [20]:
timer(finite_automata, article, "art")

Czas wykonania algorytmu:0.03453850746154785s


In [22]:
timer(kmp_string_matching, article, "art")

Czas wykonania algorytmu:0.04773211479187012s


In [23]:
print(len(naive(article, "art")))
print(len(finite_automata(article, "art")))
print(len(kmp_string_matching(article, "art")))
print(naive(article, "art") == finite_automata(article, "art") == kmp_string_matching(article, "art"))
print(naive(article, "art"))

273
273
273
True
[1153, 1502, 4689, 4731, 4876, 5079, 5145, 5946, 6036, 7263, 7508, 7778, 8041, 8296, 9101, 9956, 10019, 10221, 11119, 11204, 11615, 13191, 15281, 15355, 16089, 16258, 16403, 16544, 16613, 16837, 16853, 23634, 24058, 24149, 24583, 24680, 24777, 24928, 25527, 25686, 26998, 27285, 27476, 27539, 27589, 27854, 28370, 28555, 28763, 30961, 31018, 31093, 31359, 31808, 32606, 32965, 33050, 33265, 33592, 34648, 34734, 35508, 36152, 37140, 37540, 38448, 38592, 39053, 39207, 39433, 39565, 39977, 41149, 41826, 42025, 42195, 42368, 42501, 42715, 42893, 42938, 43444, 43552, 43784, 44587, 44650, 44950, 45007, 45290, 45398, 47316, 47419, 48782, 48817, 48903, 49049, 49256, 49313, 49485, 49556, 49912, 49976, 50099, 50157, 50699, 51047, 51176, 51963, 52068, 52269, 52549, 53005, 53029, 53208, 53785, 53928, 54075, 54134, 54767, 55072, 55276, 55462, 55804, 55988, 56824, 56908, 57161, 57546, 57797, 57929, 57986, 58277, 58375, 58871, 58963, 59392, 59520, 59946, 60293, 60546, 60791, 61259, 6176

### Cases where performance differ a lot

#### >5x improvement over naive

In [18]:
pattern = 'X' * 70000
text = (pattern +'D') * 25
print("Naiwny algorytm:")
timer(naive,text, pattern)
print()

print("Automat skończony:")
timer(finite_automata,text, pattern, transition_table(pattern)) 
print()

print("Algorytm KMP:")
timer(kmp_string_matching, text, pattern, prefix_function(pattern))

Naiwny algorytm:
Czas wykonania algorytmu:6.2361907958984375s

Automat skończony:
Czas wykonania algorytmu:0.3621647357940674s

Algorytm KMP:
Czas wykonania algorytmu:0.5906519889831543s


#### Transition table creation time is  >5x longer than prefix function in KMP

In [24]:
pattern2 = "X"*3000+"D"
t1 = time()
transition_table(pattern2)
t2 = time()
print("Czas wykonania algorytmu:" + str(t2-t1) + "s")

t1 = time()
prefix_function(pattern2)
t2 = time()
print("Czas wykonania algorytmu:" + str(t2-t1) + "s")

Czas wykonania algorytmu:2.5707316398620605s
Czas wykonania algorytmu:0.0030448436737060547s


### Test text

In [18]:
article = """
Dz.U. z 1998 r. Nr 144, poz. 930
                                       
                                       
                                       
                                       
                                    USTAWA
                          z dnia 20 listopada 1998 r.
                                       
         o zryczałtowanym podatku dochodowym od niektórych przychodów
                        osiąganych przez osoby fizyczne
                                       
                                  Rozdział 1
                                Przepisy ogólne
                                       
                                    Art. 1.
Ustawa reguluje opodatkowanie zryczałtowanym podatkiem dochodowym niektórych
przychodów (dochodów) osiąganych przez osoby fizyczne prowadzące pozarolniczą
działalność gospodarczą oraz przez osoby duchowne.
                                       
                                    Art. 2.
1. Osoby fizyczne osiągające przychody z pozarolniczej działalności
  gospodarczej opłacają zryczałtowany podatek dochodowy w formie:
   1) ryczałtu od przychodów ewidencjonowanych,
   2) karty podatkowej.
2. Osoby duchowne, prawnie uznanych wyznań, opłacają zryczałtowany podatek
  dochodowy od przychodów osób duchownych.
3. Wpływy z podatku dochodowego opłacanego w formie ryczałtu od przychodów
  ewidencjonowanych oraz zryczałtowanego podatku dochodowego od przychodów
  osób duchownych stanowią dochód budżetu państwa.
4. Wpływy z karty podatkowej stanowią dochody gmin.
                                       
                                    Art. 3.
Przychodów (dochodów) opodatkowanych w formach zryczałtowanych nie łączy się z
przychodami (dochodami) z innych źródeł podlegającymi opodatkowaniu na
podstawie ustawy z dnia 26 lipca 1991 r. o podatku dochodowym od osób
fizycznych (Dz. U. z 1993 r. Nr 90, poz. 416 i Nr 134, poz. 646, z 1994 r. Nr
43, poz. 163, Nr 90, poz. 419, Nr 113, poz. 547, Nr 123, poz. 602 i Nr 126,
poz. 626, z 1995 r. Nr 5, poz. 25 i Nr 133, poz. 654, z 1996 r. Nr 25, poz.
113, Nr 87, poz. 395, Nr 137, poz. 638, Nr 147, poz. 686 i Nr 156, poz. 776, z
1997 r. Nr 28, poz. 153, Nr 30, poz. 164, Nr 71, poz. 449, Nr 85, poz. 538, Nr
96, poz. 592, Nr 121, poz. 770, Nr 123, poz. 776, Nr 137, poz. 926, Nr 139,
poz. 932-934 i Nr 141, poz. 943 i 945 oraz z 1998 r. Nr 66, poz. 430, Nr 74,
poz. 471, Nr 108, poz. 685 i Nr 117, poz. 756), zwanej dalej "ustawą o podatku
dochodowym".
                                       
                                    Art. 4.
Użyte w ustawie określenia oznaczają:
   1) działalność usługowa - pozarolniczą działalność gospodarczą, której
     przedmiotem są czynności zaliczone do usług, zgodnie z Polską
     Klasyfikacją Wyrobów i Usług, wprowadzoną rozporządzeniem Rady Ministrów
     z dnia 18 marca 1997 r. w sprawie Polskiej Klasyfikacji Wyrobów i Usług
     (PKWiU) (Dz.U. Nr 42, poz. 264), z zastrzeżeniem pkt 2 - 4,
   2) działalność gastronomiczna - usługi związane z przygotowaniem posiłków
     oraz sprzedażą posiłków i towarów,
   3) działalność usługowa w zakresie handlu - sprzedaż w stanie nie
     przetworzonym nabytych uprzednio produktów (wyrobów) i towarów, w tym
     również takich, które zostały przez sprzedawcę zapakowane lub rozważone
     w mniejsze opakowania albo rozlane w butelki, puszki lub mniejsze
     pojemniki,
   4) działalność usługowa w zakresie obnośnego i obwoźnego handlu - sprzedaż
     detaliczną towarów ze skrzyni, walizy, kosza, z samochodu, wozu konnego
     albo w inny sposób przenoszonych lub przewożonych z miejsca na miejsce,
   5) działalność wytwórcza - działalność, w wyniku której powstają nowe
     wyroby, w tym również sprzedaż wyrobów własnej produkcji, prowadzoną
     przez podatnika,
   6) towary - towary handlowe, surowce i materiały podstawowe, z tym że:
       a) towarami handlowymi są towary (wyroby) zakupione z przeznaczeniem do
        dalszej odprzedaży w stanie nie przetworzonym,
       b) surowcami i materiałami podstawowymi są materiały, które w procesie
        produkcji lub przy świadczeniu usług stają się główną substancją
        gotowego wyrobu; do materiałów podstawowych zalicza się również
        materiały stanowiące część składową (montażową) wyrobu lub ściśle z
        wyrobem złączone, z tym że za surowce i materiały podstawowe nie
        uważa się paliw i olejów zużywanych w transporcie,
   7) dowody zakupu - otrzymane faktury, rachunki, paragony, dowody
     wewnętrzne oraz opis otrzymanego towaru; do udokumentowania zakupów
     towarów stosuje się odpowiednio przepisy dotyczące prowadzenia
     podatkowej księgi przychodów i rozchodów,
   8) karty przychodów - indywidualne (imienne) karty przychodów pracowników
     prowadzone przez podatników dokonujących pracownikom wypłat należności
     ze stosunku pracy, o których mowa w art. 12 ustawy o podatku dochodowym,
   9) podatek dochodowy na ogólnych zasadach - podatek dochodowy od osób
     fizycznych opłacany przy zastosowaniu podstawy obliczania podatku, o
     której mowa w art. 26 ustawy o podatku dochodowym i skali, o której mowa
     w art. 27 ustawy o podatku dochodowym,
  10) księgi - księgi rachunkowe albo podatkową księgę przychodów
     i rozchodów, prowadzone na zasadach określonych w odrębnych przepisach,
  11) gospodarstwo rolne - gospodarstwo rolne w rozumieniu przepisów
     o podatku rolnym.
                                       
                                    Art. 5.
Podatnicy opodatkowani na zasadach określonych w ustawie nie mają obowiązku
prowadzenia ksiąg, chyba że przepisy ustawy stanowią inaczej.
                                       
                                  Rozdział 2
                    Ryczałt od przychodów ewidencjonowanych
                                       
                                    Art. 6.
1. Opodatkowaniu ryczałtem od przychodów ewidencjonowanych, z zastrzeżeniem
  art. 8, podlegają przychody z pozarolniczej działalności gospodarczej, o
  których mowa w art. 14 ustawy o podatku dochodowym, w tym również gdy
  działalność ta jest prowadzona w formie spółki cywilnej osób fizycznych,
  zwanej dalej "spółką".
2. Opodatkowaniu ryczałtem od przychodów ewidencjonowanych podlegają również
  przychody:
   1) rolników prowadzących gospodarstwo rolne:
       a) ze sprzedaży piasku, żwiru, kamieni, gliny, glinki, torfu i innych
        minerałów wydobywanych z gruntów wchodzących w skład gospodarstwa
        rolnego,
       b) z usług świadczonych osobiście lub z udziałem członków rodziny
        pozostających we wspólnym gospodarstwie domowym w zakresie przewozu
        mleka do punktów skupu, zrywki i wywozu drewna z lasu do punktów
        przeładunkowych oraz przewozu uczniów do szkół,
       c) z usług o charakterze agrotechnicznym i przewozowym, świadczonych na
        rzecz kół łowieckich,
   2) osób fizycznych z tytułu świadczenia usług hotelarskich, polegających
     na wynajmie pokoi gościnnych i domków turystycznych, w tym również
     takich usług połączonych z wydawaniem posiłków, jeżeli łączna liczba
     pokoi, w tym także w domkach turystycznych, nie przekracza 12,
   3) osób fizycznych z działalności gospodarczej w zakresie wytwórczości
     ludowej i artystycznej, wykonywanej osobiście lub z udziałem członków
     rodziny pozostających we wspólnym gospodarstwie domowym, jeżeli:
       a) sprzedają wytwarzane przez siebie wyroby lub świadczą usługi w
        zakresie wytwórczości ludowej lub artystycznej wyłącznie na rzecz
        osób prawnych i innych jednostek organizacyjnych nie mających
        osobowości prawnej oraz osób fizycznych będących przedsiębiorcami; na
        równi z usługami traktuje się wytwarzanie wyrobów o charakterze
        ludowym i artystycznym z materiałów powierzonych oraz naprawę tych
        wyrobów,
       b) ze sprzedaży wyrobów lub świadczenia usług osiągają przychody
        w kwocie nie przekraczającej 40.000 zł rocznie,
       c) założą książkę zamówień.
3. Wytwórczością ludową i artystyczną, o której mowa w ust. 2 pkt 3, jest
  działalność polegająca na wytwarzaniu wyłącznie w sposób rękodzielniczy,
  przy zastosowaniu obróbki maszynowej jedynie do wstępnej obróbki surowca,
  wyrobów zakwalifikowanych przez komisje etnograficzno-artystyczne
  Stowarzyszenia Twórców Ludowych, Fundacji "Cepelia" - Polska Sztuka i
  Rękodzieło oraz Fundacji Ochrony i Rozwoju Twórczości Ludowej, których
  wykaz stanowi załącznik nr 1 do ustawy.
4. Podatnicy opłacają w roku podatkowym ryczałt od przychodów
  ewidencjonowanych z działalności wymienionej w ust. 1, jeżeli:
   1) w roku poprzedzającym rok podatkowy:
       a) uzyskali przychody z tej działalności prowadzonej wyłącznie
        samodzielnie, w wysokości nie przekraczającej 400.000 zł, lub
       b) uzyskali przychody wyłącznie z działalności prowadzonej w formie
        spółki, a suma przychodów wspólników spółki z tej działalności nie
        przekroczyła kwoty 400.000 zł,
   2) rozpoczną wykonywanie działalności w roku podatkowym i nie korzystają z
     opodatkowania w formie karty podatkowej - bez względu na wysokość
     przychodów.
5. Jeżeli podatnik w roku poprzedzającym rok podatkowy prowadził działalność
  samodzielnie, a także w formie spółki, opłaca w roku podatkowym ryczałt od
  przychodów ewidencjonowanych:
   1) odrębnie z działalności wykonywanej samodzielnie i odrębnie
     z działalności wykonywanej w formie spółki, gdy spełnia warunki
     określone w ust. 4 pkt 1, albo
   2) z działalności wykonywanej samodzielnie, gdy spełnia warunek określony
     w ust. 4 pkt 1 lit. a), a z działalności wykonywanej w formie spółki gdy
spełnia warunek określony w ust. 4 pkt 1 lit. b).
6. Ryczałt od przychodów ewidencjonowanych opłacają również podatnicy, którzy
  w roku poprzedzającym rok podatkowy prowadzili działalność samodzielnie lub
  w formie spółki, z której przychody były opodatkowane wyłącznie w formie
  karty podatkowej, lub za część roku były opodatkowane w formie karty
  podatkowej i za część roku na ogólnych zasadach, a łączne przychody w roku
  poprzedzającym rok podatkowy nie przekroczyły kwoty 400.000 zł; w tych
  przypadkach przychody opodatkowane w formie karty podatkowej oblicza się
  wyłącznie na podstawie wystawionych faktur lub rachunków. Do podatników
  tych stosuje się odpowiednio przepisy ust. 4 i 5.
                                       
                                    Art. 7.
1.  Do  podatników  rozpoczynających działalność w roku podatkowym,  którzy  w
  poprzednim   roku  podatkowym  nie  prowadzili  działalności,  opodatkowanie
  ryczałtem  od  przychodów ewidencjonowanych stosuje się  od  dnia  uzyskania
  pierwszego  przychodu.  W przypadku nowo utworzonej  spółki  za  rozpoczęcie
  działalności uważa się datę uzyskania pierwszego przychodu przez tę spółkę.
2. Nie stanowi rozpoczęcia działalności w roku podatkowym:
   1) całkowita lub częściowa zmiana branży lub przedmiotu działalności,
   2) zmiana umowy spółki lub zmiana wspólnika.
                                       
Art. 8.
1. Przepisów rozdziału 2, z wyjątkiem art. 6 ust. 2, nie stosuje się do
  podatników:
   1) opłacających podatek w formie karty podatkowej na zasadach określonych
     w rozdziale 3,
   2) korzystających, na podstawie odrębnych przepisów, z okresowego
     zwolnienia od podatku dochodowego,
   3) osiągających w całości lub w części przychody z tytułu:
       a) prowadzenia aptek,
       b) działalności w zakresie udzielania pożyczek pod zastaw (prowadzenia
        lombardów),
       c) działalności w zakresie kupna i sprzedaży wartości dewizowych,
       d) wykonywania wolnych zawodów,
       e) świadczenia usług wymienionych w załączniku nr 2 do ustawy, z
        zastrzeżeniem ust. 4,
   4) wytwarzających wyroby opodatkowane podatkiem akcyzowym, na podstawie
     odrębnych przepisów,
   5) podejmujących wykonywanie działalności w roku podatkowym:
       a) po zmianie działalności wykonywanej samodzielnie na działalność
        prowadzoną na imię obojga małżonków, w tym także w formie spółki z
        małżonkiem,
       b) po zmianie działalności wykonywanej na imię obojga małżonków, w tym
        także w formie spółki z małżonkiem, na działalność prowadzoną
        samodzielnie przez jednego lub każdego z małżonków,
       c) po zmianie działalności wykonywanej samodzielnie przez małżonka, na
        działalność prowadzoną samodzielnie przez drugiego małżonka
    - jeżeli małżonek lub małżonkowie, na imię których prowadzona była
     działalność przed zmianą, opłacali z tej działalności podatek dochodowy
     na ogólnych zasadach,
   6) rozpoczynających działalność samodzielnie lub w formie spółki, jeżeli
     podatnik lub co najmniej jeden ze wspólników, przed rozpoczęciem
     działalności w roku podatkowym lub w roku poprzedzającym rok podatkowy,
     wykonywał w ramach stosunku pracy lub spółdzielczego stosunku pracy
     czynności wchodzące w zakres działalności podatnika lub spółki.
2. Jeżeli podatnicy, wymienieni w ust. 1 pkt 6, nie zamierzają wykonywać
  działalności na rzecz byłych lub obecnych pracodawców, mogą do dnia
  rozpoczęcia działalności, o którym mowa w art. 7, zawiadomić urząd skarbowy
  właściwy według miejsca zamieszkania podatnika o korzystaniu z
  opodatkowania w formie ryczałtu od przychodów ewidencjonowanych. W
  przypadku, gdy podatnik lub spółka dokona sprzedaży towarów handlowych lub
  wyrobów, lub uzyska przychody ze świadczenia usług na rzecz byłego lub
  obecnego pracodawcy, traci prawo do ryczałtu od przychodów
  ewidencjonowanych i poczynając od dnia uzyskania tego przychodu do końca
  roku podatkowego opłaca podatek dochodowy na ogólnych zasadach.
3. Jeżeli podatnik w roku poprzedzającym rok podatkowy nie uzyskał przychodu
  z działalności, o której mowa w ust. 1 pkt 3 i 4, traci w roku podatkowym
  prawo do opodatkowania w formie ryczałtu od przychodów ewidencjonowanych
  z dniem uzyskania przychodów z tych rodzajów działalności i od tego dnia
  opłaca podatek dochodowy na ogólnych zasadach.
4. Nie wyłącza opodatkowania w formie ryczałtu od przychodów
  ewidencjonowanych osiąganie dodatkowo przychodów z najmu, podnajmu,
  dzierżawy, poddzierżawy oraz z innych umów o podobnym charakterze.
5. Przychody, o których mowa w ust. 4, podlegają odrębnemu opodatkowaniu
  podatkiem dochodowym na ogólnych zasadach.
                                       
                                    Art. 9.
1. Podatnik może zrzec się opodatkowania w formie ryczałtu od przychodów
  ewidencjonowanych za dany rok podatkowy. Zrzeczenie może być dokonane nie
  później niż do dnia 20 stycznia roku podatkowego lub do dnia
  poprzedzającego dzień rozpoczęcia działalności, jeżeli podatnik rozpoczyna
  działalność w ciągu roku.
2. Zrzeczenia, o którym mowa w ust. 1, podatnik dokonuje przez złożenie
  urzędowi skarbowemu właściwemu według miejsca zamieszkania podatnika
  pisemnego oświadczenia. W wypadku prowadzenia działalności w formie spółki,
  wszyscy wspólnicy składają wspólne oświadczenie urzędom skarbowym właściwym
  według miejsca zamieszkania każdego ze wspólników.
3. Podatnik jest obowiązany w terminie siedmiu dni od dnia złożenia
  oświadczenia o zrzeczeniu złożyć:
   1) urzędowi skarbowemu, o którym mowa w art. 29 ust. 3 lub 4, wniosek
     o zastosowanie opodatkowania w formie karty podatkowej, jeżeli wybierze
     tę formę i spełnia warunki określone w rozdziale 3, albo
   2) urzędowi skarbowemu właściwemu według miejsca zamieszkania podatnika,
     zawiadomienie o założeniu właściwych ksiąg od dnia 1 stycznia roku
     podatkowego lub od dnia rozpoczęcia działalności w ciągu roku
     podatkowego, jeżeli wybierze opłacanie podatku dochodowego na ogólnych
     zasadach.
4. Zawiadomienie, o którym mowa w ust. 3 pkt 2, może być również złożone
  w tym samym oświadczeniu, w którym podatnik zrzeka się opodatkowania w
  formie ryczałtu od przychodów ewidencjonowanych.
                                       
                                   Art. 10.
Zwolnienia od podatku dochodowego, o których mowa w art. 21 ust. 1 pkt 35, 43,
46, 48 ustawy o podatku dochodowym, stosuje się odpowiednio do podatników
opłacających ryczałt od przychodów ewidencjonowanych wymienionych w art. 6
ust. 1.
                                       
                                   Art. 11.
1. Podatnik uzyskujący przychody wymienione w art. 6 ust. 1, opłacający
  podatek w formie ryczałtu od przychodów ewidencjonowanych, może odliczyć od
  przychodów stratę, o której mowa w art. 9 ust. 3 ustawy o podatku
  dochodowym oraz wydatki określone w art. 26 ust. 1 ustawy o podatku
  dochodowym, jeżeli nie zostały odliczone od dochodu na podstawie przepisów
  ustawy o podatku dochodowym.
2. W przypadku dokonywania odliczeń od przychodu stosuje się odpowiednio
  przepisy art. 9 ust. 3 i art. 26 ust. 2-12 ustawy o podatku dochodowym.
3. Za wydatki na cele rehabilitacyjne ponoszone przez podatnika będącego
  osobą niepełnosprawną lub podatnika, na którego utrzymaniu są osoby
  niepełnosprawne, uważa się wydatki poniesione na:
   1) adaptację i wyposażenie mieszkań oraz budynków mieszkalnych stosownie
     do potrzeb wynikających z niepełnosprawności,
   2) przystosowanie pojazdów mechanicznych do potrzeb wynikających z
     niepełnosprawności,
   3) zakup i naprawę indywidualnego sprzętu, urządzeń i narzędzi
     technicznych niezbędnych w rehabilitacji oraz ułatwiających wykonywanie
     czynności życiowych, stosownie do potrzeb wynikających z
     niepełnosprawności,
   4) zakup wydawnictw i materiałów (pomocy) szkoleniowych, stosownie do
     potrzeb wynikających z niepełnosprawności,
   5) odpłatność za pobyt na turnusie rehabilitacyjno-usprawniającym,
   6) odpłatność za pobyt na leczeniu sanatoryjnym, za pobyt w placówkach
     leczniczo-sanatoryjnych, rehabilitacyjo-szkoleniowych, leczniczoopiekuńczych
i szkolno-wychowawczych oraz odpłatność za zabiegi
     rehabilitacyjno-usprawniające,
   7) opłacanie przewodników osób niewidomych I lub II grupy inwalidztwa oraz
     osób z niepełnosprawnością narządu ruchu zaliczonych do I grupy
     inwalidztwa, w kwocie nie przekraczającej w roku podatkowym
     trzykrotności najniższego wynagrodzenia pracowników, określonego w
     odrębnych przepisach, za grudzień roku poprzedzającego rok podatkowy,
   8) utrzymanie przez osoby niewidome, o których mowa w pkt 7, psa
     przewodnika - w wysokości nie przekraczającej w roku podatkowym kwoty
     określonej w pkt 7,
   9) opiekę pielęgniarską w domu nad osobą niepełnosprawną w okresie
     przewlekłej choroby uniemożliwiającej poruszanie się oraz usługi
     opiekuńcze świadczone dla osób niepełnosprawnych zaliczonych do I grupy
     inwalidztwa,
  10) opłacanie tłumacza języka migowego,
  11) kolonie i obozy dla dzieci i młodzieży niepełnosprawnej oraz dzieci
     osób niepełnosprawnych,
  12) leki - w wysokości stanowiącej różnicę pomiędzy faktycznie poniesionymi
     wydatkami w danym miesiącu a kwotą odpowiadającą 20% najniższego
     wynagrodzenia pracowników, określonego w odrębnych przepisach, za
     grudzień roku poprzedzającego rok podatkowy, jeśli lekarz specjalista
     stwierdzi, że osoba niepełnosprawna powinna stosować określone leki
     (stale lub czasowo),
  13) odpłatny, konieczny przewóz na niezbędne zabiegi leczniczorehabilitacyjne:

       a) osoby niepełnosprawnej - karetką transportu sanitarnego,
       b) osoby niepełnosprawnej, zaliczonej do I lub II grupy inwalidztwa,
        oraz dzieci niepełnosprawnych do lat 16 - również innymi środkami
        transportu, niż wymienione w lit. a),
  14) używanie samochodu osobowego, stanowiącego własność osoby
     niepełnosprawnej zaliczonej do I lub II grupy inwalidztwa lub podatnika
     mającego na utrzymaniu osobę niepełnosprawną zaliczoną do I lub II grupy
     inwalidztwa albo dzieci niepełnosprawne do lat 16, dla potrzeb
     związanych z koniecznym przewozem na niezbędne zabiegi leczniczorehabilitacyjne
- w wysokości nie przekraczającej w roku podatkowym
     kwoty określonej w pkt 7,
  15) odpłatne przejazdy środkami transportu publicznego związane z pobytem
     na:
       a) turnusie rehabilitacyjno-usprawniającym,
       b) leczeniu sanatoryjnym w placówkach leczniczo-sanatoryjnych,
        rehabilitacyjno-szkoleniowych, leczniczo-opiekuńczych oraz szkolnowychowawczych,

       c) koloniach i obozach dla dzieci i młodzieży niepełnosprawnej oraz
        dzieci osób niepełnosprawnych.
4. Wydatki na cele rehabilitacyjne wymienione w ust. 3 podlegają odliczeniu
  od przychodu, jeżeli nie były finansowane ze środków zakładowego funduszu
  rehabilitacji osób niepełnosprawnych lub Państwowego Funduszu Rehabilitacji
  Osób Niepełnosprawnych albo nie zostały zwrócone podatnikowi w
  jakiejkolwiek innej formie. W przypadku gdy wydatki były częściowo
  finansowane z tych funduszy, odlicza się różnicę pomiędzy poniesionymi
  wydatkami a kwotą sfinansowaną z zakładowego funduszu albo Państwowego
  Funduszu Rehabilitacji Osób Niepełnosprawnych lub zwróconą z jakichkolwiek
  środków.
5. Podstawą do odliczenia wydatków, o których mowa w ust. 3, jest posiadanie
  dowodu poniesienia tych wydatków, z wyjątkiem wydatków, o których mowa w
  ust. 3, pkt 7, 8 i 14.
6. Warunkiem odliczenia, o którym mowa w ust. 5, jest posiadanie przez osobę,
  której dotyczy wydatek:
   1) orzeczenia o zakwalifikowaniu przez organy orzekające do jednego z
     trzech stopni niepełnosprawności, określonych w odrębnych przepisach,
     lub
   2) decyzji przyznającej rentę z tytułu całkowitej lub częściowej
     niezdolności do pracy albo rentę szkoleniową, albo
   3) orzeczenia o rodzaju i stopniu niepełnosprawności osoby, która nie
     ukończyła 16 roku życia, wydanego na podstawie odrębnych przepisów.
7. Przepisy ust. 3-6 stosuje się odpowiednio do podatników, na których
  utrzymaniu pozostają następujące osoby niepełnosprawne: współmałżonek,
  dzieci własne i przysposobione, dzieci obce przyjęte na wychowanie,
  pasierbowie, rodzice, rodzice współmałżonka, rodzeństwo, ojczym, macocha,
  zięciowie i synowe - jeżeli w roku podatkowym dochody tych osób
  niepełnosprawnych nie przekraczają dwunastokrotności kwoty najniższego
  wynagrodzenia pracowników za grudzień roku poprzedzającego rok podatkowy.
8. Ilekroć w przepisach ust. 3-7 jest mowa o osobach zaliczonych do:
   1) I grupy inwalidztwa - należy przez to rozumieć odpowiednio osoby, w
     stosunku do których, na podstawie odrębnych przepisów, orzeczono:
       a) całkowitą niezdolność do pracy oraz niezdolność do samodzielnej
        egzystencji albo
       b) znaczny stopień niepełnosprawności,
   2) II grupy inwalidztwa - należy przez to rozumieć odpowiednio osoby w
     stosunku do których, na podstawie odrębnych przepisów, orzeczono:
       a) całkowitą niezdolność do pracy albo
       b) umiarkowany stopień niepełnosprawności.
9. Odliczenie, o którym mowa w ust. 5, może być dokonane również w przypadku
  gdy osoba, której dotyczy wydatek, posiada orzeczenie o niepełnosprawności
  wydane przez właściwy organ na podstawie odrębnych przepisów obowiązujących
  do dnia 31 sierpnia 1997 r. lub w odniesieniu do dzieci do lat 16
  orzeczenie o niepełnosprawności wydane przez lekarza specjalistę lub
  właściwą przychodnię specjalistyczną publicznej służby zdrowia.
                                       
                                   Art. 12.
1. Ryczałt od przychodów ewidencjonowanych wynosi:
   1) 8,5%:
       a) przychodów z działalności usługowej, w tym z działalności
        gastronomicznej w zakresie przychodów ze sprzedaży napojów
        o zawartości powyżej 1,5% alkoholu, z zastrzeżeniem pkt 2 i 3,
       b) przychodów z działalności polegającej na wytwarzaniu przedmiotów
        (wyrobów) z materiału powierzonego przez zamawiającego,
       c) prowizji uzyskanej przez komisanta ze sprzedaży na podstawie umowy
        komisu,
       d) prowizji uzyskanej przez kolportera prasy na podstawie umowy
        o kolportaż prasy,
       e) przychodów, określonych w art. 6 ust. 2 pkt 1 lit. b) i c),
       f) przychodów ze świadczenia usług, określonych w art. 6 ust. 2 pkt 3,
   2) 5,5% :
       a) przychodów z działalności wytwórczej, robót budowlanych lub
        w zakresie przewozów ładunków taborem samochodowym o ładowności
        powyżej 2 ton,
       b) uzyskanej prowizji z działalności handlowej w zakresie sprzedaży
        jednorazowych biletów komunikacji miejskiej, znaczków do biletów
        miesięcznych, znaków opłaty skarbowej, znaczków pocztowych, żetonów
        i kart magnetycznych do automatów,
       c)  przychodów z działalności wytwórczej, o której mowa w art. 6 ust. 2
        pkt 1 lit. a),
       d) przychodów ze sprzedaży wyrobów, o których mowa w art. 6 ust. 2 pkt
        3,
   3) 3,0% przychodów:
       a) z działalności gastronomicznej, z wyjątkiem przychodów ze sprzedaży
        napojów o zawartości powyżej 1,5% alkoholu,
       b) z działalności usługowej w zakresie handlu,
       c) ze świadczenia usług związanych z produkcją zwierzęcą, zaliczonych
        zgodnie z Polską Klasyfikacją Wyrobów i Usług do klasy 01.42 - usługi
        związane z chowem i hodowlą zwierząt (z wyjątkiem usług
        weterynaryjnych) i do klasy 85.20 - usługi weterynaryjne,
       d) z działalności rybaków morskich i zalewowych w zakresie sprzedaży
        ryb i innych surowców z własnych połowów, z wyjątkiem konserw oraz
        prezerw z ryb i innych surowców z połowów,
       e) o których mowa w art. 14 ust. 2 pkt 1, 2 i 5-9 ustawy o podatku
        dochodowym,
       f) osób fizycznych z tytułu świadczenia usług hotelarskich, o których
        mowa w art. 6 ust. 2 pkt 2.
2. Podatek zryczałtowany, o którym mowa w ust. 1, pobiera się bez
  pomniejszania przychodu o koszty uzyskania.
3. Jeżeli podatnik obowiązany do prowadzenia ewidencji przychodów prowadzi
  działalność, z której przychody są opodatkowane różnymi stawkami
  określonymi w ust. 1, ryczałt od przychodów ewidencjonowanych ustala się
  według stawki właściwej dla przychodów z każdego rodzaju działalności, pod
  warunkiem, że ewidencja przychodów jest prowadzona w sposób umożliwiający
  określenie przychodów z każdego rodzaju działalności. W razie, gdy
  podatnik, o którym mowa w zdaniu poprzednim, nie prowadzi ewidencji w
  sposób zapewniający ustalenie przychodów dla każdego rodzaju działalności,
  ryczałt od przychodów ewidencjonowanych wynosi 8,5% przychodów.
4. W przypadku, gdy działalność jest wykonywana w formie spółki, przychody
  opodatkowuje się osobno u każdego wspólnika w stosunku do jego udziału w
  zyskach określonego w umowie. W razie braku takiego dowodu przyjmuje się,
  że udziały wspólników w przychodach są równe.
                                       
                                   Art. 13.
1. Ryczałt od przychodów ewidencjonowanych w pierwszej kolejności ulega
  obniżeniu o kwotę składki na powszechne ubezpieczenie zdrowotne podatnika,
  o której mowa w art. 27b ust. 1 ustawy o podatku dochodowym, zapłaconej w
  roku podatkowym, zgodnie z przepisami o powszechnym ubezpieczeniu
  zdrowotnym, o ile nie została odliczona od podatku dochodowego.
2. W przypadku dokonywania obniżki, o której mowa w ust. 1, stosuje się
  odpowiednio przepisy art. 27b ust. 2 i 3 ustawy o podatku dochodowym.
                                       
                                   Art. 14.
1. Ryczałt od przychodów ewidencjonowanych wymienionych w art. 6 ust. 1,
  pomniejszony o kwotę składki, o której mowa w art. 13, ulega obniżeniu o
  wydatki wymienione w art. 27a ust. 1, z wyjątkiem pkt 3 lit. b) ustawy
  o podatku dochodowym, jeżeli nie zostały odliczone od podatku na podstawie
  przepisów ustawy o podatku dochodowym.
2. W przypadku dokonywania odliczeń, o których mowa w ust. 1, stosuje się
  odpowiednio przepisy art. 27a ust. 2, ust. 3 pkt 1 lit. a)-c), e)-h), pkt 2
  i 3, ust. 4-6, ust. 7 pkt 1, ust. 7a, ust. 8-15, ust. 17-19 oraz ust. 21
  ustawy o podatku dochodowym.
                                       
                                   Art. 15.
1. Podatnicy i spółki, których wspólnicy są opodatkowani w formie ryczałtu od
  przychodów ewidencjonowanych, są obowiązani, z zastrzeżeniem ust. 2,
  prowadzić ewidencję przychodów odrębnie za każdy rok podatkowy, zwaną dalej
  "ewidencją", wykaz środków trwałych oraz wartości niematerialnych i
  prawnych oraz posiadać i przechowywać dowody zakupu towarów.
2. Obowiązek prowadzenia ewidencji nie dotyczy podatników uzyskujących
  przychody wymienione w art. 6 ust. 2 pkt 1 i 2 wyłącznie od osób prawnych i
  innych jednostek organizacyjnych nie mających osobowości prawnej oraz osób
  fizycznych będących przedsiębiorcami, a także przychody, o których mowa w
  art. 6 ust. 2 pkt 3.
3. Obowiązek prowadzenia ewidencji, z zastrzeżeniem ust. 4, powstaje od dnia,
  od którego ma zastosowanie opodatkowanie w formie ryczałtu od przychodów
  ewidencjonowanych. Podatnicy są obowiązani zawiadomić w formie pisemnej
  urząd skarbowy właściwy według miejsca zamieszkania podatnika o założeniu
  ewidencji do dnia 20 stycznia roku podatkowego, w którym będą opłacali
  ryczałt od przychodów ewidencjonowanych.
4. Podatnicy rozpoczynający prowadzenie działalności w ciągu roku podatkowego
  są obowiązani złożyć zawiadomienie, o którym mowa w ust. 3, w terminie
  siedmiu dni od dnia rozpoczęcia działalności.
5. Jeżeli działalność jest prowadzona w formie spółki, zawiadomienie składa
  jeden ze wspólników. W przypadku gdy jeden ze wspólników ma miejsce
  zamieszkania na terenie objętym właściwością miejscową urzędu skarbowego,
  na którym również znajduje się siedziba spółki, obowiązek złożenia
  zawiadomienia, o którym mowa w ust. 3 i 4, ciąży na tym wspólniku.
6. Ewidencję oraz dowody, na podstawie których dokonywane są zapisy, a także
  dowody zakupu, o których mowa w ust. 1, należy przechowywać w miejscu
  wykonywania działalności lub w miejscu wskazanym przez podatnika jako jego
  siedziba, albo w biurze rachunkowym, któremu zostało powierzone prowadzenie
  ewidencji. Jeżeli na zlecenie podatnika prowadzenie ewidencji zostało
  powierzone biuru rachunkowemu, podatnik jest obowiązany w terminie siedmiu
  dni od dnia zawarcia umowy z biurem rachunkowym zawiadomić o tym urząd
  skarbowy, w którym zostało złożone zawiadomienie o prowadzeniu ewidencji,
  wskazując nazwę i adres biura oraz miejsce (adres) przechowywania ewidencji
  i dowodów związanych z jej prowadzeniem.
                                       
                                   Art. 16.
Minister właściwy do spraw finansów publicznych określi, w drodze
rozporządzenia:
   1) sposób prowadzenia ewidencji, szczegółowe warunki jakim powinna
     odpowiadać ewidencja, aby stanowiła dowód w postępowaniu podatkowym,
     sposób dokumentowania przychodów oraz obliczania należnego ryczałtu od
     przychodów ewidencjonowanych,
   2) sposób prowadzenia wykazu środków trwałych oraz wartości
     niematerialnych i prawnych, o którym mowa w art. 15 ust. 1,
   3) sposób prowadzenia książki zamówień, o której mowa w art. 6 ust. 2 pkt
     3 lit.c).
                                       
                                   Art. 17.
W razie nieprowadzenia lub prowadzenia ewidencji bez zachowania warunków do
uznania jej za dowód w postępowaniu podatkowym, organ podatkowy określi
wartość niezaewidencjonowanego przychodu, w tym również w drodze oszacowania i
ustali od tej kwoty ryczałt od przychodów ewidencjonowanych w wysokości 20%.
                                       
                                   Art. 18.
1. Osoby prawne i inne jednostki organizacyjne nie mające osobowości prawnej
  oraz osoby fizyczne będące przedsiębiorcami, zwani dalej "płatnikami", na
  których rzecz jest wykonywana działalność wymieniona w art. 6 ust. 2, są
  obowiązane do obliczania i poboru ryczałtu od przychodów ewidencjonowanych
  przy każdorazowej wypłacie należności.
2. Płatnicy nie są obowiązani do poboru ryczałtu od przychodów
  ewidencjonowanych, jeżeli podatnik przed pierwszą wypłatą należności w roku
  podatkowym zawiadomi  w formie pisemnej płatnika, że:
   1) nie spełnia warunków do opodatkowania w formie ryczałtu od przychodów
     ewidencjonowanych, lub
   2) zrzekł się opodatkowania w formie ryczałtu od przychodów
     ewidencjonowanych za dany rok podatkowy.
                                       
                                   Art. 19.
1. Płatnicy przekazują kwoty pobranego ryczałtu od przychodów
  ewidencjonowanych pomniejszone o kwotę składki na powszechne ubezpieczenie
  zdrowotne, o której mowa w art. 13, w terminie do siódmego dnia miesiąca
  następującego po miesiącu, w którym na rzecz podatnika dokonano wypłaty
  należności - na rachunek urzędu skarbowego właściwego według siedziby
  płatnika.
2. W terminie, o którym mowa w ust. 1, płatnicy są obowiązani przesłać
  właściwym urzędom skarbowym w odniesieniu do przychodów, o których mowa:
   1)  w art. 6 ust. 2 pkt 1 i 2 - zbiorcze deklaracje według ustalonego
     wzoru,
   2)  w art. 6 ust. 2 pkt 3 - imienne informacje według ustalonego wzoru.
3. W terminie do dnia 15 marca roku następującego po roku podatkowym płatnicy
  są obowiązani przesłać podatnikom osiągającym przychody wymienione w art. 6
  ust. 2 pkt 3 oraz urzędowi skarbowemu właściwemu według miejsca
  zamieszkania podatnika imienne informacje o wysokości przychodu i pobranym
  ryczałcie od przychodów ewidencjonowanych.
                                       
                                   Art. 20.
1. Podatnicy, uzyskujący przychody wymienione w art. 6 ust. 1, są obowiązani
  sporządzić spis z natury towarów handlowych, materiałów (surowców)
  podstawowych i pomocniczych, półwyrobów, wyrobów gotowych, braków i
  odpadków, zwany dalej "spisem z natury" na dzień zaprowadzenia ewidencji
  oraz na koniec każdego roku podatkowego. Spis z natury należy sporządzić
  również w razie zmiany wspólnika lub zmiany umowy spółki, a także na dzień
  likwidacji działalności. W razie zawiadomienia urzędu skarbowego o
  likwidacji działalności gospodarczej podatnicy są obowiązani sporządzić
  również spis rzeczowych składników majątku związanego z wykonywaną
  działalnością, nie będących środkami trwałymi, zwanych dalej
  "wyposażeniem".
2. Spis z natury powinien zawierać co najmniej następujące dane: nazwisko
  i imię właściciela zakładu (nazwę firmy), datę sporządzenia spisu, numer
  kolejny pozycji arkusza spisu z natury, szczegółowe określenie towaru
  i innych składników wymienionych w ust. 1, jednostkę miary, ilość
  stwierdzoną w czasie spisu, cenę w złotych i groszach za jednostkę miary,
  wartość wynikającą z przemnożenia ilości towaru przez jego cenę
  jednostkową, łączną wartość spisu z natury oraz klauzulę "Spis zakończono
  na pozycji...", podpisy osób sporządzających spis oraz podpis właściciela
  zakładu (wspólników).
3. Podatnik wycenia towary i inne składniki majątku objęte spisem z natury
  według cen zakupu albo według cen rynkowych z dnia sporządzenia spisu,
  jeżeli są one niższe od cen zakupu, a także gdy nie jest możliwe ustalenie
  ceny zakupu. Przez cenę zakupu rozumie się cenę, jaką nabywca płaci za
  zakupione składniki majątku, pomniejszoną o podatek od towarów i usług,
  podlegający odliczeniu zgodnie z odrębnymi przepisami, a przy imporcie powiększoną
o należne cło i podatek akcyzowy, opłaty wyrównawcze oraz
  opłaty celne dodatkowe, zaś w przypadku otrzymania składnika majątku w
  drodze darowizny lub spadku - wartość odpowiadającą cenie zakupu takiego
  samego lub podobnego składnika.
4. Podatnik jest obowiązany dokonać wyceny najpóźniej w terminie 14 dni od
  dnia zakończenia spisu z natury.
5. Podatnik jest obowiązany wpisać do ewidencji spis z natury według
  poszczególnych rodzajów jego składników lub w jednej pozycji (sumie),
  jeżeli na podstawie spisu zostało sporządzone odrębne szczegółowe
  zestawienie poszczególnych jego składników. Zestawienie to powinno być
  przechowywane łącznie z ewidencją.
                                       
                                   Art. 21.
1. Podatnicy, z wyjątkiem podatników, o których mowa w art. 15 ust. 2, są
  obowiązani za każdy miesiąc obliczać ryczałt od przychodów
  ewidencjonowanych i wpłacać go w terminie do dnia 20 następnego miesiąca, a
  za miesiąc grudzień w terminie złożenia zeznania - na rachunek urzędu
  skarbowego właściwego według miejsca zamieszkania podatnika.
2. Podatnicy, o których mowa w ust. 1, są obowiązani złożyć w urzędzie
  skarbowym właściwym według miejsca zamieszkania podatnika:
   1) informację według ustalonego wzoru o wysokości uzyskanego przychodu i
     wysokości dokonanych odliczeń za I półrocze roku podatkowego - w
     terminie do dnia 31 lipca każdego roku,
   2) zeznanie według ustalonego wzoru o wysokości uzyskanego przychodu,
     wysokości dokonanych odliczeń i należnego ryczałtu od przychodów
     ewidencjonowanych - w terminie do dnia 31 stycznia następnego roku.
3. Przy obliczaniu ryczałtu od przychodów ewidencjonowanych za poszczególne
  miesiące, podatnicy mogą uwzględniać odliczenia i obniżki, o których mowa w
  art. 11, 13 i 14.
4. Ryczałt od przychodów ewidencjonowanych wynikający z zeznania jest
  podatkiem należnym za dany rok podatkowy, chyba że urząd skarbowy wyda
  decyzję, w której ustali inną wysokość podatku.
5. W razie niezłożenia zeznania o wysokości osiągniętego przychodu,
  dokonanych odliczeniach i kwocie należnego ryczałtu od przychodów
  ewidencjonowanych, urząd skarbowy, z zastrzeżeniem art. 17, wyda decyzję
  określającą wysokość zobowiązania w formie ryczałtu od przychodów
  ewidencjonowanych.
                                       
                                   Art. 22.
1. W razie utraty warunków do opodatkowania ryczałtem od przychodów
  ewidencjonowanych podatnik jest obowiązany, poczynając od dnia, w którym
  nastąpiła utrata tych warunków, zaprowadzić właściwe księgi - chyba, że
  jest zwolniony z tego obowiązku - i opłacać podatek dochodowy na ogólnych
  zasadach.
2. W stosunku do podatników, o których mowa w ust. 1, podlegających za część
  roku opodatkowaniu na ogólnych zasadach, za podstawę do określenia podatku
  dochodowego przyjmuje się dochód osiągnięty po utracie warunków do
  opodatkowania w formie ryczałtu od przychodów ewidencjonowanych.
                                       
                                  Rozdział 3
                                Karta podatkowa
                                       
                                   Art. 23.
1. Zryczałtowany podatek dochodowy w formie karty podatkowej mogą płacić
  podatnicy prowadzący działalność:
   1) usługową lub wytwórczo - usługową, określoną w części I tabeli
     stanowiącej załącznik nr 3 do ustawy, zwanej dalej "tabelą", w zakresie
     wymienionym w załączniku nr 4 do ustawy - przy zatrudnieniu nie
     przekraczającym stanu określonego w tabeli,
   2) usługową w zakresie handlu detalicznego żywnością, napojami, wyrobami
     tytoniowymi oraz kwiatami, z wyjątkiem napojów o zawartości powyżej 1,5%
     alkoholu - w warunkach określonych w części II tabeli,
   3) usługową w zakresie obnośnego i obwoźnego handlu detalicznego
     artykułami nieżywnościowymi - w warunkach określonych w części III
     tabeli, z wyjątkiem handlu paliwami silnikowymi, środkami transportu
     samochodowego, ciągnikami rolniczymi i motocyklami oraz z wyjątkiem
     handlu artykułami nieżywnościowymi objętego koncesjonowaniem,
   4) gastronomiczną - jeżeli nie jest prowadzona sprzedaż napojów o
     zawartości powyżej 1,5% alkoholu - w warunkach określonych w części IV
     tabeli,
   5) w zakresie usług transportowych wykonywanych przy użyciu jednego
     pojazdu - w warunkach określonych w części V tabeli,
   6) w zakresie usług rozrywkowych - w warunkach określonych w części VI
     tabeli,
   7) w zakresie sprzedaży posiłków domowych w mieszkaniach, jeżeli nie jest
     prowadzona sprzedaż napojów o zawartości powyżej 1,5% alkoholu - w
     warunkach określonych w części VII tabeli,
   8) w wolnych zawodach, polegającą na świadczeniu usług w zakresie ochrony
     zdrowia ludzkiego - w warunkach określonych w części VIII tabeli,
   9) w wolnych zawodach, polegającą na świadczeniu usług weterynaryjnych, w
     tym również sprzedaż preparatów weterynaryjnych wymienionych w
     pozycjach: 24.42.13-70, 24.42.21-65, 24.42.21-69, 24.42.23-40 ex oraz
     24.41.51-90.30 Polskiej Klasyfikacji Wyrobów i Usług, w związku ze
     świadczonymi usługami - w warunkach określonych w części IX tabeli,
  10) w zakresie opieki domowej nad dziećmi i osobami chorymi  - w warunkach
     określonych w części X tabeli,
  11) w zakresie usług edukacyjnych, polegającą na udzielaniu lekcji na
     godziny - w warunkach określonych w części XI tabeli.
2. Wyróżnik "ex", o którym mowa w ust. 1 pkt 9, oznacza, że zakres
  dopuszczonych do sprzedaży wyrobów jest węższy niż określony w danym
  grupowaniu Polskiej Klasyfikacji Wyrobów i Usług.
                                       
                                   Art. 24.
1. Podatnicy prowadzący działalność opodatkowaną w formie karty podatkowej są
  zwolnieni od obowiązku prowadzenia ksiąg, składania zeznań podatkowych,
  deklaracji o wysokości uzyskanego dochodu oraz wpłacania zaliczek na
  podatek dochodowy. Podatnicy ci są jednak obowiązani wydawać na żądanie
  klienta rachunki i faktury, o których mowa w odrębnych przepisach,
  stwierdzające sprzedaż wyrobu, towaru lub wykonanie usługi oraz
  przechowywać w kolejności numerów kopie tych rachunków i faktur w okresie
  pięciu lat podatkowych, licząc od końca roku, w którym wystawiono rachunek
  lub fakturę.
2. Do podatników, którzy w roku poprzedzającym rok podatkowy, opłacali
  podatek dochodowy na ogólnych zasadach lub w innej formie niż karta
  podatkowa, albo którzy w roku podatkowym rozpoczęli działalność, nie
  stosuje się zwolnień, o których mowa w ust. 1, w okresie przed doręczeniem
  decyzji ustalającej opodatkowanie w formie karty podatkowej, nie dłuższym
  jednak niż cztery miesiące. Kwoty podatku dochodowego zapłacone w okresie
  przed doręczeniem decyzji ustalającej opodatkowanie w formie karty
  podatkowej zalicza się na poczet podatku wynikającego z tej decyzji.
3. Podatnicy, którzy w roku poprzedzającym rok podatkowy, opłacali podatek
  dochodowy w formie karty podatkowej, są obowiązani do dnia otrzymania
  decyzji ustalającej wysokość tego podatku, opłacać podatek dochodowy w
  formie karty podatkowej, w wysokości stawki określonej w decyzji
  ustalającej jej wysokość na rok poprzedni. Kwoty podatku dochodowego
  zapłacone w okresie przed doręczeniem decyzji ustalającej opodatkowanie w
  formie karty podatkowej zalicza się na poczet podatku wynikającego z tej
  decyzji.
                                       
Art. 25.
1. Podatnicy prowadzący działalność, o której mowa w art. 23, podlegają
  opodatkowaniu w formie karty podatkowej, jeżeli:
   1) złożą wniosek o zastosowanie opodatkowania w tej formie,
   2) we wniosku, o którym mowa w pkt 1, zgłoszą prowadzenie działalności
     wymienionej w jednej z 11 części tabeli,
   3) przy prowadzeniu działalności nie korzystają z usług osób nie
     zatrudnionych przez siebie na podstawie umowy o pracę, oraz z usług
     innych przedsiębiorstw i zakładów, chyba że chodzi o usługi
     specjalistyczne,
   4) nie prowadzą, poza jednym z rodzajów działalności wymienionej w art.
     23, innej pozarolniczej działalności gospodarczej, z wyjątkiem
     działalności, o której mowa w art. 6 ust. 2,
   5) małżonek podatnika nie prowadzi działalności w tym samym zakresie,
   6) nie wytwarzają wyrobów opodatkowanych, na podstawie odrębnych
     przepisów, podatkiem akcyzowym.
2. Nie wyłącza opodatkowania w formie karty podatkowej:
   1) prowadzenie przez podatnika działalności w zakresie wytwarzania energii
     elektrycznej, w elektrowniach wodnych i wiatrowych, o mocy oddawanej do
     5000 kilowatów oraz wytwarzanie biogazu,
   2) osiąganie przychodów z najmu, podnajmu, dzierżawy, poddzierżawy oraz z
     innych umów o podobnym charakterze
 - z których przychody (dochody) podlegają odrębnemu opodatkowaniu podatkiem
  dochodowym na ogólnych zasadach.
3. Za usługi specjalistyczne w rozumieniu ust. 1 pkt 3 uważa się czynności i
  prace wchodzące w inny niż zgłoszony zakres działalności, niezbędne do
  całkowitego wykonania wyrobu lub świadczonej usługi, w tym również
  czynności i prace towarzyszące, o których mowa w załączniku nr 4.
4. Jeżeli podatnik we wniosku o zastosowanie opodatkowania w formie karty
  podatkowej zgłosi prowadzenie działalności określonej w art. 23 ust. 1 pkt
  1, 2 i 4 w różnym zakresie lub prowadzi działalność w odrębnych zakładach,
  łączne zatrudnienie nie może przekroczyć stanu zatrudnienia określonego dla
  zakresu działalności, dla którego przewidziano najniższy stan zatrudnienia.
5. Podatnicy prowadzący działalność określoną w art. 23 ust. 1 pkt 1-6 mogą
  być opodatkowani w formie karty podatkowej również wtedy, gdy działalność
  ta jest prowadzona w formie spółki, pod warunkiem, że łączna liczba
  wspólników oraz zatrudnionych pracowników nie przekracza stanu zatrudnienia
  określonego w tabeli.
6. Przy ocenie warunków uzasadniających opodatkowanie w formie karty
  podatkowej i ustalaniu wysokości podatku dochodowego w odniesieniu do
  podatników, o których mowa w art. 23 ust. 1 pkt 1, do liczby pracowników:
   1) wlicza się również osoby zatrudnione na podstawie umowy o pracę
     nakładczą oraz członków rodziny mających inne niż podatnik miejsce
     pobytu stałego lub czasowego,
   2) nie wlicza się:
       a) członków rodziny pozostających we wspólnym gospodarstwie domowym z
        podatnikiem, a w razie gdy działalność prowadzona jest przez
        wspólników - tylko członków rodziny jednego ze wspólników,
       b) osób zatrudnionych w celu nauki zawodu lub przyuczenia do
        wykonywania określonej pracy, zgodnie z odrębnymi przepisami o nauce
        zawodu lub przyuczaniu do wykonywania określonej pracy - w okresie
        nauki zawodu lub przyuczania oraz nie więcej niż trzech osób
        zatrudnionych w okresie pierwszych dwunastu miesięcy po złożeniu
        przez nie egzaminu; na równi z osobami zatrudnionymi w celu nauki
        zawodu lub przyuczenia do wykonywania określonej pracy traktuje się
        uczniów szkół średnich oraz studentów, zatrudnionych w okresie
        wakacji letnich i zimowych,
       c) osób pobierających emerytury lub renty - łącznie nie więcej niż
        dwóch,
       d) pracowników zatrudnionych wyłącznie przy sprzedaży wyrobów,
        przyjmowaniu zleceń na usługi, utrzymywaniu czystości w zakładzie,
        prowadzeniu kasy i księgowości, kierowców i konwojentów - pod
        warunkiem, że podatnik określił na piśmie zakres czynności tych
        pracowników,
       e)  osób, na których rachunek jest prowadzona działalność po śmierci
        podatnika, jeżeli osoby te nie biorą udziału w prowadzeniu
        działalności,
       f)  jednego bezrobotnego absolwenta skierowanego, na podstawie
        odrębnych przepisów, przez właściwy urząd pracy do odbywania stażu u
        pracodawcy przez okres nie przekraczający dwunastu miesięcy.
7. Przy ocenie warunków uzasadniających opodatkowanie w formie karty
  podatkowej i ustalaniu wysokości stawek podatku w odniesieniu do
  podatników, o których mowa w art. 23 ust. 1 pkt 2-6 i 11, do liczby
  pracowników nie wlicza się małżonka, z zastrzeżeniem pkt 2 objaśnień do
  części V tabeli oraz pkt 1 objaśnień do części XI tabeli, i niepełnoletnich
  członków rodziny pozostających we wspólnym gospodarstwie domowym z
  podatnikiem. W razie prowadzenia działalności przez wspólników - do liczby
  pracowników nie wlicza się małżonka i niepełnoletnich członków rodziny
  tylko jednego wspólnika. Przy prowadzeniu działalności usługowej w zakresie
  handlu detalicznego i gastronomii - do liczby pracowników nie wlicza się
  ponadto osób zatrudnionych w celu praktycznej nauki zawodu, zgodnie z
  odrębnymi przepisami o nauce zawodu, oraz pracowników zatrudnionych
  wyłącznie do utrzymywania czystości w zakładzie, pod warunkiem, że podatnik
  określił na piśmie zakres czynności tych pracowników.
8. W przypadku powołania osób, o których mowa w ust. 6 pkt 2 lit. b), do
  odbycia zasadniczej służby wojskowej w okresie pierwszych dwunastu miesięcy
  po złożeniu przez nie egzaminu, okres nie wliczania ich do stanu
  zatrudnienia przedłuża się o czas trwania służby, jeżeli podatnik zawiadomi
  w formie pisemnej o tym fakcie urząd skarbowy w terminie siedmiu dni od
  dnia powołania do służby.
                                       
                                   Art. 26.
1. Wysokość podatku dochodowego w formie karty podatkowej, z zastrzeżeniem
  art. 27 ust. 4, ustala się:
   1) dla podatników prowadzących działalność określoną w art. 23 ust. 1 pkt
     1 i pkt 5-11 - według stawek miesięcznych określonych w tabeli,
   2) dla podatników prowadzących działalność określoną w art. 23 ust. 1 pkt
     2-4 - w granicach stawek miesięcznych określonych w tabeli, biorąc pod
     uwagę wszystkie okoliczności mające wpływ na rozmiary tej działalności.
2. W przypadkach, o których mowa w art. 25 ust. 5, wysokość podatku
  dochodowego w formie karty podatkowej ustala się według stawki określonej
  dla stanu zatrudnienia odpowiadającego łącznej liczbie wspólników i
  pracowników.
3. W przypadkach, o których mowa w art. 25 ust. 6 pkt 2 lit. e), wysokość
  podatku dochodowego w formie karty podatkowej ustala się według stawki
  bezpośrednio poprzedzającej stawkę określoną dla stanu zatrudnienia
  odpowiadającego łącznej liczbie wspólników i pracowników, z tym że stawka
  ta nie może być niższa od określonej dla działalności wykonywanej bez
  zatrudnienia pracowników.
4. Jeżeli podatnik we wniosku o zastosowanie opodatkowania w formie karty
  podatkowej zgłosi prowadzenie działalności wymienionej w art. 23 ust. 1 pkt
  1 w różnych zakresach lub różnych miejscowościach, dla których przewidziano
  różne wysokości stawek karty podatkowej, wysokość podatku dochodowego
  w formie karty podatkowej ustala się według stawki właściwej dla tego
  zakresu działalności i miejscowości, dla których przewidziana jest stawka
  najwyższa. Przepis ten ma odpowiednie zastosowanie do podatników
  prowadzących działalność usługową w zakresie handlu detalicznego oraz
  działalność gastronomiczną.
5. Liczbę mieszkańców miejscowości określonych w tabeli przyjmuje się według
  stanu na dzień 31 grudnia roku poprzedzającego rok podatkowy.
                                       
                                   Art. 27.
1.  Stawki karty podatkowej:
   1) określone w części I tabeli obniża się o 20% dla podatników, którzy
     ukończyli do dnia 1 stycznia roku podatkowego 60 lat życia lub są
     osobami, w stosunku do których, na podstawie odrębnych przepisów
     orzeczono co najmniej lekki stopień niepełnosprawności, jeżeli przy
     prowadzeniu działalności określonej w art. 23 ust. 1 pkt 1, z wyjątkiem
     prowadzenia parkingów, nie zatrudniają pracowników, w tym również osób
     określonych w art. 25 ust. 6 pkt 2 lit. a) i c), nie licząc małżonka,
   2) określone w części V tabeli obniża się o 20% dla podatników, którzy
     ukończyli do dnia 1 stycznia roku podatkowego 60 lat życia lub są
     osobami w stosunku do których, na podstawie odrębnych przepisów,
     orzeczono co najmniej umiarkowany lub lekki stopień niepełnosprawności,
     jeżeli przy prowadzeniu działalności w zakresie usług transportowych nie
     zatrudniają pracowników i członków rodziny,
   3) określone w części I tabeli obniża się dla podatników, którzy
     zatrudniają pracowników będących osobami, w stosunku do których, na
     podstawie odrębnych przepisów, orzeczono co najmniej lekki stopień
     niepełnosprawności, o 10% na każdą osobę; obniżka stawki nie dotyczy
     osób wymienionych w art. 25 ust. 6 pkt 2 lit. c) i d).
2. Na wniosek podatnika urząd skarbowy może dodatkowo obniżyć stawkę karty
  podatkowej określoną w części I tabeli w przypadkach szczególnie
  uzasadnionych - w stosunku do podatników, o których mowa w ust. 1.
3. Na wniosek podatnika urząd skarbowy może obniżyć stawkę karty podatkowej
  określoną w części I tabeli, jeżeli rozmiar działalności podatnika, przy
  zatrudnieniu nie przekraczającym jednego pracownika, wskazuje na to, że
  określona w tabeli stawka byłaby oczywiście za wysoka.
4. W razie gdy podatnik prowadzi działalność wymienioną w art. 23 ust. 1 pkt
  2-5 i pkt 8-11 w rozmiarach wskazujących na to, iż określone w częściach IIV
i VIII-XI tabeli stawki byłyby oczywiście nieodpowiednie, stawki te mogą
  być na wniosek podatnika lub z urzędu obniżone lub podwyższone, nie więcej
  jednak niż o 50%.
5. Podatnikom, którzy przy równoczesnym zatrudnieniu na podstawie umowy
  o pracę w pełnym wymiarze czasu pracy wykonują usługi i nie zatrudniają
  pracowników oraz osób, o których mowa w art. 25 ust. 6, stawki karty
  podatkowej określone w części I tabeli obniża się o 80%; podatnicy ci nie
  mogą równocześnie korzystać z obniżek przewidzianych w ust. 1-4.
6. W przypadkach określonych w art. 25 ust. 5 obniżkę stawki, o której mowa w
  ust. 1 pkt 1 i w ust. 5, stosuje się, jeżeli wszyscy wspólnicy spełniają
  warunki uprawniające do tej obniżki.
7. Przepisy ust. 1 i 4 stosuje się odpowiednio do podatników, którzy przed
  dniem 1 września 1997 r., uzyskali orzeczenia o zakwalifikowaniu do jednej
  z trzech grup inwalidzkich przez organy działające na podstawie przepisów o
  zaopatrzeniu emerytalnym i o ubezpieczeniu społecznym.
                                   Art. 28.
1. W razie zatrudnienia przez podatnika przy prowadzeniu działalności
  określonej w art. 23 ust. 1 pkt 1, z wyjątkiem prowadzenia parkingów,
  pełnoletnich członków rodziny, nie licząc małżonka, a w przypadkach
  określonych w art. 25 ust. 5 - pełnoletnich członków rodziny jednego ze
  wspólników, nie licząc małżonka, a także w razie zatrudnienia osób, o
  których mowa w art. 25 ust. 6 pkt 2 lit. c) i d), podwyższa się stawkę
  karty podatkowej z tytułu zatrudnienia każdej z tych osób za miesiące, w
  których była zatrudniona:
   1) o 40% stawki określonej dla działalności prowadzonej bez zatrudnienia
     pracowników,
   2) o 30% stawki określonej dla działalności prowadzonej przy zatrudnieniu
     jednego pracownika,
   3) o 20% stawki określonej dla działalności prowadzonej przy zatrudnieniu
     dwóch pracowników,
   4) o 15% stawki określonej dla działalności prowadzonej przy zatrudnieniu
     trzech pracowników,
   5) o 10% stawki określonej dla działalności prowadzonej przy zatrudnieniu
     czterech i pięciu pracowników.
2. Wysokość stawki  karty podatkowej ustalonej na podstawie ust. 1 nie może
  być wyższa od stawki bezpośrednio następującej po stawce właściwej dla
  łącznej liczby wspólników i pracowników.
                                       
                                   Art. 29.
1. Wniosek o zastosowanie opodatkowania w formie karty podatkowej podatnik
  składa właściwemu urzędowi skarbowemu w deklaracji według ustalonego wzoru,
  w terminie do dnia 31 grudnia roku poprzedzającego rok podatkowy, od
  którego ma być zastosowana karta, a jeżeli podatnik rozpoczyna działalność
  w ciągu roku podatkowego - przed rozpoczęciem działalności. Jeżeli do końca
  każdego roku podatkowego objętego opodatkowaniem w formie karty podatkowej
  podatnik nie zgłosił likwidacji działalności, uważa się, że prowadzi nadal
  działalność opodatkowaną w tej formie.
2. Podatnicy, którzy są zatrudnieni na podstawie umowy o pracę w pełnym
  wymiarze czasu pracy w służbie weterynaryjnej organów administracji
  publicznej i wykonują równocześnie działalność, o której mowa w art. 23
  ust. 1 pkt 9, są obowiązani dołączyć do wniosku oświadczenie w formie
  pisemnej o tym zatrudnieniu.
3. Właściwy miejscowo w sprawie podatku dochodowego opłacanego w formie karty
  podatkowej jest urząd skarbowy według miejsca położenia zorganizowanego
  zakładu, wskazanego w zgłoszeniu do ewidencji działalności gospodarczej, a
  w razie niedopełnienia zgłoszenia działalności do ewidencji lub gdy
  działalność jest wykonywana bez posiadania zorganizowanego zakładu - według
  miejsca zamieszkania podatnika lub siedziby spółki, a jeżeli nie można
  ustalić siedziby spółki - według miejsca zamieszkania jednego ze
  wspólników.
4. Jeżeli działalność jest prowadzona w kilku zorganizowanych zakładach na
  terenie objętym właściwością miejscową kilku urzędów skarbowych właściwość
  ustala się według miejsca wskazanego przez podatnika jako siedzibę
  działalności.
                                       
                                   Art. 30.
1. Urząd skarbowy, uwzględniając wniosek o zastosowanie karty podatkowej,
  wydaje decyzję ustalającą wysokość podatku dochodowego w formie karty
  podatkowej, odrębnie na każdy rok podatkowy. Jeżeli działalność jest
  prowadzona w formie spółki, w decyzji tej wymienia się wszystkich jej
  wspólników.
2. Jeżeli urząd skarbowy stwierdzi brak warunków do zastosowania
  opodatkowania w formie karty podatkowej, wydaje decyzję odmowną. W tym
  przypadku podatnik jest obowiązany płacić:
   1) ryczałt od przychodów ewidencjonowanych, jeżeli spełnia warunki
     określone w rozdziale 2, albo
   2) podatek dochodowy na ogólnych zasadach.
3. Podatnicy, o których mowa w ust. 2, korzystający dotychczas ze zwolnienia
  od obowiązku prowadzenia ksiąg albo ewidencji, o której mowa w art. 15 ust.
  1, są obowiązani do założenia i prowadzenia właściwych ksiąg albo
  ewidencji, poczynając od pierwszego dnia miesiąca następującego po
  miesiącu, w którym została doręczona decyzja odmawiająca im zastosowania
  opodatkowania w formie karty podatkowej.
                                       
                                   Art. 31.
1. Podatek dochodowy w formie karty podatkowej, wynikający z decyzji, o
  której mowa w art. 30, podatnik obniża o kwotę składki na powszechne
  ubezpieczenie zdrowotne opłaconej w roku podatkowym, zgodnie z przepisami o
  powszechnym ubezpieczeniu zdrowotnym, o ile nie została odliczona od
  podatku dochodowego. Zasadę tę stosuje się również do podatników, o których
  mowa w art. 25 ust. 5.
2. Kwota składki na powszechne ubezpieczenie zdrowotne, o którą pomniejsza
  się kartę podatkową, nie może przekroczyć 7,5% podstawy wymiaru tej
  składki, określonej w odrębnych przepisach.
3. Wysokość wydatków na cele, o których mowa w ust. 1, ustala się na
  podstawie dokumentów stwierdzających ich poniesienie.
4. Po upływie roku podatkowego, w terminie do dnia 31 stycznia, podatnik jest
  obowiązany złożyć we właściwym urzędzie skarbowym roczną informację według
  ustalonego wzoru o wysokości składki na powszechne ubezpieczenie zdrowotne
  zapłaconej i odliczonej od karty podatkowej w poszczególnych miesiącach.
5. Podatnicy płacą podatek dochodowy w formie karty podatkowej, pomniejszony
  o zapłaconą składkę na powszechne ubezpieczenie zdrowotne, bez wezwania w
  terminie do dnia siódmego każdego miesiąca za miesiąc ubiegły, a za
  grudzień - w terminie do dnia 28 grudnia roku podatkowego, na rachunek
  właściwego urzędu skarbowego.
                                       
                                   Art. 32.
1. Podatnik może zrzec się zastosowania opodatkowania w formie karty
  podatkowej w terminie 14 dni od dnia doręczenia decyzji ostatecznej
  ustalającej wysokość podatku dochodowego w formie karty podatkowej.
2. Podatnik, o którym mowa w ust. 1, jest obowiązany od pierwszego dnia
  miesiąca następującego po zrzeczeniu zaprowadzić:
   1) ewidencję i płacić ryczałt od przychodów ewidencjonowanych, jeżeli
     spełnia warunki określone w rozdziale 2, albo
   2) właściwe księgi i płacić podatek dochodowy na ogólnych zasadach.
3. Do końca miesiąca, w którym podatnik zrzekł się zastosowania opodatkowania
  w formie karty podatkowej, jest obowiązany opłacać podatek dochodowy na
  ogólnych zasadach. W tym przypadku podatnik jest obowiązany złożyć
  właściwemu urzędowi skarbowemu deklarację według ustalonego wzoru w ustawie
  o podatku dochodowym o wysokości dochodu osiągniętego od początku roku do
  końca miesiąca, w którym zrzekł się opodatkowania w formie karty
  podatkowej, w terminie przewidzianym dla składania tej deklaracji. W tym
  samym terminie podatnik jest obowiązany wpłacić różnicę pomiędzy kwotą
  podatku dochodowego wynikającego z deklaracji a kwotą podatku dochodowego
  zapłaconego w formie karty podatkowej, w okresie przed zrzeczeniem się
  opodatkowania w tej formie.
                                       
                                   Art. 33.
1. W razie rozpoczęcia działalności w ciągu roku podatkowego, podatek w
  formie karty podatkowej za okres od dnia rozpoczęcia działalności do końca
  pierwszego miesiąca pobiera się w wysokości 1/30 miesięcznej należności za
  każdy dzień.
2. Przepis ust. 1 stosuje się odpowiednio do podatników, którzy korzystali ze
  zwolnienia od podatku dochodowego i którym okres tego zwolnienia upłynął z
  pierwszym dniem danego miesiąca.
3. W razie likwidacji działalności w ciągu roku podatkowego i zgłoszenia tego
  urzędowi skarbowemu, podatek w formie karty podatkowej za ostatni miesiąc
  prowadzenia działalności pobiera się za okres do dnia zaprzestania
  działalności w wysokości 1/30 miesięcznej należności za każdy dzień.
                                       
                                   Art. 34.
1. W razie zgłoszenia przez podatnika przerwy w prowadzeniu działalności w
  danym roku podatkowym, z wyjątkiem podatników prowadzących działalność
  usługową w zakresie obnośnego i obwoźnego handlu detalicznego, nie pobiera
  się podatku w formie karty podatkowej za pełny okres przerwy trwającej
  nieprzerwanie co najmniej dziesięć dni w wysokości 1/30 miesięcznej
  należności za każdy dzień przerwy, jeżeli podatnik zawiadomi o tej przerwie
  w terminie trzech dni od dnia jej rozpoczęcia i dnia jej zakończenia.
2. Zasadę, o której mowa w ust. 1, stosuje się do podatników prowadzących
  działalność usługową w zakresie obnośnego i obwoźnego handlu detalicznego,
  jeżeli przerwa została spowodowana chorobą, służbą wojskową, wyjazdem za
  granicę oraz urlopem wypoczynkowym w wymiarze co najmniej jednego miesiąca.
                                       
                                   Art. 35.
1. Podatnicy opodatkowani w formie karty podatkowej, z wyjątkiem prowadzących
  parkingi, jeżeli przy prowadzeniu działalności zatrudniają pracowników, w
  tym również wymienionych w art. 25 ust. 6 pkt 1, oraz osoby, o których mowa
  w art. 25 ust. 6 pkt 2 lit. a), c), d) i f), są obowiązani prowadzić
  książkę ewidencji zatrudnienia tych osób, zwaną dalej "ewidencją
  zatrudnienia".
2. Podatnik jest obowiązany przechowywać ewidencję zatrudnienia w miejscu
  wykonywania działalności albo w miejscu wskazanym przez podatnika jako jego
  siedziba, z zastrzeżeniem ust. 3, jeżeli działalność jest wykonywana bez
  posiadania zorganizowanego zakładu.
3. Podatnicy prowadzący działalność usługową w zakresie obnośnego i obwoźnego
  handlu są obowiązani przechowywać ewidencję zatrudnienia w miejscu
  wykonywania działalności.
4. Minister właściwy do spraw finansów publicznych określi, w drodze
  rozporządzenia, sposób prowadzenia ewidencji zatrudnienia.
                                       
                                   Art. 36.
1. Podatnicy opodatkowani w formie karty podatkowej są obowiązani zawiadomić
  urząd skarbowy:
   1) o zmianach, jakie zaszły w stosunku do stanu faktycznego podanego w
     złożonym wniosku o zastosowanie opodatkowania w formie karty podatkowej,
     które:
       a) powodują utratę warunków do opodatkowania w formie karty podatkowej,
       b) mają wpływ na wysokość podatku dochodowego w formie karty
        podatkowej, a w szczególności o zmianach: w stanie zatrudnienia,
        miejsca prowadzenia działalności, rodzaju i zakresu prowadzonej
        działalności, liczby stanowisk na parkingu oraz liczby i rodzaju
        urządzeń przy prowadzeniu usług rozrywkowych, liczby godzin
        przeznaczonych na wykonywanie wolnego zawodu w zakresie ochrony
        zdrowia ludzkiego oraz wolnego zawodu w zakresie usług
        weterynaryjnych, liczby sprzedawanych posiłków domowych, liczby
        godzin sprawowania opieki domowej nad dziećmi i osobami chorymi,
        liczby godzin przeznaczonych na udzielanie lekcji,
   2) o likwidacji prowadzonej działalności.
2. Nie uważa się za zwiększenie stanu zatrudnienia czasowego zatrudnienia
  pracownika lub pełnoletniego członka rodziny w miejsce nieobecnego stałego
  pracownika, którego nieobecność trwa nieprzerwanie co najmniej dziesięć dni
  i jest spowodowana szczególnymi względami, w tym chorobą, urlopem,
  powołaniem na ćwiczenia wojskowe.
3. W przypadkach wymienionych w ust. 2, jeżeli w miejsce nieobecnego stałego
  pracownika podatnik nie zatrudni innego pracownika lub pełnoletniego
  członka rodziny, urząd skarbowy obniża stawkę karty podatkowej w sposób
  określony w art. 37.
4. Nie stanowi przeszkody do opodatkowania w formie karty podatkowej
  przekroczenie stanu zatrudnienia określonego w części I, II i IV tabeli:
   1) nie więcej niż o trzech pracowników - jeżeli podatnik prowadzi
     działalność na terenie gmin wymienionych w odrębnych przepisach,
     określających wykaz gmin zagrożonych szczególnie wysokim bezrobociem
     strukturalnym oraz zawiadomi o tym urząd skarbowy, albo
   2) nie więcej niż o dwóch pracowników - jeżeli podatnik prowadzący
     gospodarstwo rolne wykonuje równocześnie pozarolniczą działalność
     gospodarczą w miejscowości o liczbie mieszkańców do 5000 oraz zawiadomi
     o tym urząd skarbowy.
5. Podatnikowi spełniającemu równocześnie warunki określone w ust. 4 pkt 1 i
  2 przysługuje prawo do przekroczenia stanu zatrudnienia nie więcej niż o
  trzech pracowników.
6. Z tytułu zatrudnienia osób, o których mowa w ust. 4, stawka karty
  podatkowej nie ulega zmianie.
7. O zmianach, o których mowa w ust. 1, 2 i 4, podatnik jest obowiązany
  zawiadomić w formie pisemnej urząd skarbowy, najpóźniej w terminie siedmiu
  dni od powstania okoliczności powodujących zmiany. Jeżeli zawiadomienie
  dotyczy zatrudnienia osoby, o której mowa w art. 25 ust. 6 pkt 2 lit. d),
  należy w nim również podać szczegółowy zakres czynności, które osoba ta ma
  wykonywać.
8. Jeżeli w trakcie roku podatkowego gmina zostanie wyłączona z wykazu gmin
  zagrożonych szczególnie wysokim bezrobociem strukturalnym, preferencje, o
  których mowa w ust. 4 pkt 1 i w ust. 5, podatnik zachowuje do dnia 31
  grudnia danego roku podatkowego.
9. Liczbę mieszkańców, o której mowa w ust. 4 pkt 2, przyjmuje się według
  stanu na dzień 31 grudnia roku poprzedzającego rok podatkowy.
                                       
                                   Art. 37.
W razie zawiadomienia o zmianach, o których mowa w art. 36 ust. 1 pkt 1 lit.
b) i ust. 3, zmiana wysokości podatku w formie karty podatkowej następuje
poczynając od miesiąca, w którym zaszły okoliczności powodujące zmianę.
Należność za ten miesiąc pobiera się w wysokości 1/30 dotychczasowej
miesięcznej należności za każdy dzień części miesiąca przed dniem, w którym
nastąpiła zmiana, zwiększonej lub zmniejszonej o 1/30 nowej miesięcznej
należności za każdy dzień pozostałej części tego miesiąca.
                                       
                                   Art. 38.
1. Podatnik może zrzec się zastosowania opodatkowania w formie karty
  podatkowej w terminie 14 dni od dnia doręczenia decyzji zmieniającej
  wysokość podatku dochodowego w formie karty podatkowej. Przepis art. 32
  ust. 2 stosuje się odpowiednio.
2. Do końca miesiąca, w którym podatnik zrzekł się zastosowania opodatkowania
  w formie karty podatkowej, opłaca on za każdy miesiąc podatek dochodowy w
  formie karty podatkowej, w wysokości stawki określonej w decyzji w sprawie
  ustalenia jej wysokości na dany rok.
                                       
                                   Art. 39.
1. Podatnik, który zawiadomi w formie pisemnej urząd skarbowy o utracie
  warunków do opodatkowania w formie karty podatkowej, jest obowiązany,
  poczynając od dnia, w którym nastąpiła utrata tych warunków, zaprowadzić:
   1) ewidencję i płacić ryczałt od przychodów ewidencjonowanych, jeżeli
     spełnia warunki określone w rozdziale 2, albo
   2) właściwe księgi - chyba, że jest zwolniony od tego obowiązku - i płacić
     podatek dochodowy na ogólnych zasadach.
2. W stosunku do podatników, o których mowa w ust. 1 pkt 2, podlegających za
  część roku opodatkowaniu na ogólnych zasadach, za podstawę do ustalenia
  podatku dochodowego za ten okres przyjmuje się dochód osiągnięty od dnia
  utraty warunków do opodatkowania w formie karty podatkowej.
                                       
                                   Art. 40.
1. W przypadku, gdy podatnik:
   1) poda we wniosku o zastosowanie opodatkowania w formie karty podatkowej
     dane niezgodne ze stanem faktycznym, powodujące nieuzasadnione
     zastosowanie opodatkowania w formie karty podatkowej, bądź ustalenie
     wysokości podatku dochodowego w formie karty podatkowej w kwocie niższej
     od należnej, lub
   2) prowadzi nierzetelnie (niezgodnie ze stanem faktycznym) ewidencję
     zatrudnienia bądź, mimo obowiązku, nie prowadzi tej ewidencji, a
     stwierdzony stan zatrudnienia jest wyższy od zgłoszonego urzędowi
     skarbowemu, lub
   3) nie zawiadomi urzędu skarbowego w terminie, o którym mowa w art. 36
     ust. 7, o zmianach powodujących utratę warunków do opodatkowania w
     formie karty podatkowej albo mających wpływ na podwyższenie wysokości
     podatku dochodowego w formie karty podatkowej, bądź w zawiadomieniu poda
     dane w tym zakresie niezgodne ze stanem faktycznym, lub
   4) w rachunku lub fakturze stwierdzających sprzedaż wyrobu, towaru lub
     wykonanie usługi poda dane istotnie niezgodne ze stanem faktycznym
 - urząd skarbowy stwierdza wygaśnięcie decyzji, o której mowa w art. 30 ust.
  1.
2. W przypadkach określonych w ust. 1:
   1) podatnicy są obowiązani płacić podatek dochodowy na ogólnych zasadach
     za cały rok podatkowy,
   2) obowiązek założenia właściwych ksiąg powstaje od pierwszego dnia
     miesiąca następującego po miesiącu, w którym została doręczona decyzja
     stwierdzająca wygaśnięcie decyzji ustalającej wysokość podatku
     dochodowego w formie karty podatkowej.
                                       
                                   Art. 41.
W razie zaistnienia okoliczności, o których mowa w art. 40, kwoty wpłacone z
tytułu karty podatkowej zalicza się na podatek dochodowy określony na ogólnych
zasadach za dany rok podatkowy.
                                       
                                   Rozdział 4
         Zryczałtowany podatek dochodowy od przychodów osób duchownych
                                       
                                   Art. 42.
1. Osoby duchowne osiągające przychody z opłat otrzymywanych w związku z
  pełnionymi funkcjami o charakterze duszpasterskim, opłacają od tych
  przychodów podatek dochodowy w formie ryczałtu od przychodów osób
  duchownych, zwany dalej "ryczałtem".
2. Osoby, o których mowa w ust. 1, zawiadamiają właściwy urząd skarbowy o
  rozpoczęciu pełnienia funkcji o charakterze duszpasterskim w terminie 14
  dni od dnia objęcia funkcji.
                                       
                                   Art. 43.
1. Kwartalne stawki ryczałtu od przychodów proboszczów i od przychodów
  wikariuszy określone są odpowiednio w załącznikach nr 5 i 6 do ustawy.
2. Kwartalne stawki ryczałtu od przychodów proboszczów i od przychodów
  wikariuszy, o których mowa w ust. 1, stosuje się odpowiednio do osób
  duchownych wszystkich wyznań, sprawujących porównywalne funkcje.
3. Liczbę mieszkańców przyjmuje się według stanu na dzień  31 grudnia roku
  poprzedzającego rok podatkowy.
                                       
                                   Art. 44.
1. Kwartalna stawka ryczałtu ulega obniżeniu o kwotę składki na powszechne
  ubezpieczenie zdrowotne, opłaconej zgodnie z przepisami o powszechnym
  ubezpieczeniu zdrowotnym, bezpośrednio przez osobę duchowną w kwartale za
  jaki uiszczany jest ryczałt, o ile składka ta nie została odliczona od
  podatku dochodowego.
2. Kwota składki na powszechne ubezpieczenie zdrowotne, o którą zmniejsza się
  stawkę ryczałtu od przychodów osób duchownych, nie może przekroczyć 7,5%
  podstawy wymiaru tej składki, określonej w odrębnych przepisach.
3. Wysokość wydatków na cele, o których mowa w ust. 1, ustala się na
  podstawie dokumentów stwierdzających ich poniesienie.
4. Do dnia 31 stycznia roku następującego po roku podatkowym, osoba duchowna
  jest obowiązana złożyć w urzędzie skarbowym właściwym według miejsca
  wykonywania funkcji o charakterze duszpasterskim roczną informację, według
  ustalonego wzoru, o wysokości składki na ubezpieczenie zdrowotne,
  zapłaconej i odliczonej od ryczałtu w poszczególnych kwartałach.
                                       
                                   Art. 45.
1. Wikariusze lub inni duchowni pełniący czasowo funkcje proboszczów opłacają
  ryczałt według stawek określonych w załączniku nr 5.
2. Osoby duchowne kierujące jednostkami kościelnymi posiadającymi samodzielną
  administrację w wydzielonej części parafii (wikariaty eksponowane, kuracje,
  lokacje itd.) opłacają ryczałt według stawek określonych w załączniku nr 5,
  właściwych dla liczby mieszkańców tej części parafii. Proboszcz parafii
  opłaca w takim przypadku ryczałt według stawki odpowiadającej liczbie
  mieszkańców, zmniejszonej o liczbę mieszkańców wydzielonej części parafii.
3. Rektorzy i inne osoby duchowne kierujące jednostkami kościelnymi
  posiadającymi samodzielną administrację bez wydzielonej części parafii
  opłacają ryczałt według stawek określonych w załączniku nr 6, dla parafii o
  liczbie mieszkańców powyżej 1.000 do 3.000, w zależności od siedziby
  kierowanej jednostki lub miejsca zatrudnienia.
4. Osoby duchowne nie pełniące funkcji proboszczów, wikariuszy i rektorów,
  które osiągają przychody wymienione w art. 42 ust. 1, z misji, rekolekcji
  oraz innych posług religijnych, jeżeli zawiadomią urząd skarbowy o
  osiąganiu takich przychodów, opłacają ryczałt według stawek określonych w
  załączniku nr 6 dla parafii o liczbie mieszkańców powyżej 1.000 do 3.000, w
  zależności od miejsca zamieszkania.
                                       
                                   Art. 46.
1. Urząd skarbowy właściwy według miejsca wykonywania funkcji o charakterze
  duszpasterskim, zwany dalej "właściwym urzędem skarbowym" wydaje decyzję
  ustalającą wysokość ryczałtu, odrębnie na każdy rok podatkowy.
2. Właściwy urząd skarbowy, na wniosek osoby duchownej, odpowiednio obniża
  stawki ryczałtu określone w załącznikach nr 5 i 6, jeżeli liczba wyznawców
  na danym terenie stanowi mniejszość w ogólnej liczbie mieszkańców. Osoba
  duchowna jest obowiązana dołączyć do wniosku oświadczenie o liczbie
  wyznawców. Przepis ust. 1 stosuje się odpowiednio.
                                       
                                   Art. 47.
Osoby duchowne opłacają ryczałt, pomniejszony o zapłaconą w kwartale składkę
na powszechne ubezpieczenie zdrowotne, bez wezwania - w terminie do dnia 20
następnego miesiąca po upływie kwartału, a za czwarty kwartał - do dnia 28
grudnia roku podatkowego, na rachunek właściwego urzędu skarbowego.
                                       
                                   Art. 48.
1. W razie rozpoczęcia wykonywania funkcji w ciągu kwartału, stawkę ryczałtu
  za okres od dnia rozpoczęcia wykonywania funkcji do końca kwartału ustala
  się za każdy dzień w wysokości 1/90 części stawki kwartalnej.
2. W razie zaprzestania na stałe wykonywania funkcji i zgłoszenia tego faktu
  właściwemu urzędowi skarbowemu, odpowiednio zmniejsza się stawkę ryczałtu w
  sposób określony w ust. 1.
3.  W razie przerwy w wykonywaniu funkcji trwającej dłużej niż jeden miesiąc
  i zawiadomienia o tym właściwego urzędu skarbowego w ciągu trzech dni od
  dnia rozpoczęcia przerwy, kwartalną stawkę ryczałtu zmniejsza się w sposób
  określony w ust. 1 o każdy dzień przerwy trwającej ponad miesiąc.
                                       
                                   Art. 49.
1. W razie zmiany miejsca wykonywania funkcji, osoba duchowna jest obowiązana
  w terminie siedmiu dni zawiadomić o tym urzędy skarbowe właściwe według
  dotychczasowego i nowego miejsca wykonywania funkcji oraz podać dane
  niezbędne do ustalenia wysokości ryczałtu według nowego miejsca wykonywania
  funkcji. Obowiązek zawiadomienia dotyczy również odpowiednio osób, których
  charakter wykonywanej funkcji ulega zmianie.
2. W przypadkach określonych w ust. 1 właściwy urząd skarbowy, ustalając
  wysokość ryczałtu za okres do końca roku podatkowego, stosuje odpowiednio
  przepis art. 48 ust. 1.
                                       
                                   Art. 50.
Osoby duchowne opłacające ryczałt są zwolnione od obowiązku składania zeznań
podatkowych o wysokości osiągniętego dochodu ze źródła przychodu objętego
ryczałtem.
                                       
                                   Art. 51.
1. Osoby duchowne mogą zrzec się opodatkowania w formie ryczałtu za dany rok
  podatkowy, nie później niż do dnia 31 grudnia roku poprzedzającego rok
  podatkowy lub do dnia poprzedzającego dzień rozpoczęcia pełnienia funkcji o
  charakterze duszpasterskim w przypadku rozpoczęcia pełnienia tych funkcji w
  ciągu roku podatkowego i opłacać za ten rok podatek dochodowy na ogólnych
  zasadach, prowadząc podatkową księgę przychodów i rozchodów na zasadach
  określonych w odrębnych przepisach.
2. Zrzeczenia, o którym mowa w ust. 1, osoba duchowna dokonuje poprzez
  złożenie, właściwemu urzędowi skarbowemu, pisemnego oświadczenia lub w
  zawiadomieniu, o którym mowa w art. 42 ust. 2, jeżeli osoba duchowna zrzeka
  się opodatkowania w formie ryczałtu począwszy od dnia rozpoczęcia
  wykonywania funkcji o charakterze duszpasterskim.
3. Osoby duchowne mogą zrzec się opłacania ryczałtu i opłacać podatek
  dochodowy na ogólnych zasadach, prowadząc podatkową księgę przychodów i
  rozchodów na zasadach określonych w odrębnych przepisach, zawiadamiając o
  tym na piśmie  właściwy urząd skarbowy w terminie 14 dni po otrzymaniu
  decyzji ustalającej wysokość ryczałtu.
4. Przepisy art. 9 ust. 3 pkt 2 stosuje się odpowiednio.
                                  Rozdział 5
                               Przepisy wspólne
                                       
                                   Art. 52.
1. Podatnicy dokonujący pracownikom wypłat należności ze stosunku pracy,
  o których mowa w art. 12 ustawy o podatku dochodowym, są obowiązani
  prowadzić karty przychodów, o których mowa w art. 4 pkt 8.
2. Minister właściwy do spraw finansów publicznych określi, w drodze
  rozporządzenia, sposób prowadzenia kart przychodów.
3. Minister właściwy do spraw finansów publicznych określi, w drodze
  rozporządzenia, wzory:
   1) zbiorczych deklaracji i imiennych informacji, o których mowa w art. 19
     ust. 2 i 3,
   2) zeznania i informacji, o których mowa w art. 21 ust. 2,
   3) wniosku o zastosowanie opodatkowania w formie karty podatkowej, o
     którym mowa w art. 29 ust. 1,
   4) rocznych informacji o wysokości składki zapłaconej na powszechne
     ubezpieczenie zdrowotne, o których mowa w art. 31 ust. 4 i 44 ust. 4,
 wraz z objaśnieniami co do sposobu prawidłowego ich wypełnienia, terminu i
  miejsca składania.
                                       
                                   Art. 53.
1. Podatnicy, o których mowa w art. 2 ust. 1 pkt 2, oraz uzyskujący przychody
  wymienione w art. 6 ust. 1, mogą korzystać z ulgi z tytułu szkolenia
  uczniów polegającej na obniżeniu zryczałtowanego podatku dochodowego,
  zwanej dalej "ulgą uczniowską".
2. Ulga uczniowska, o której mowa w ust. 1, przysługuje osobom fizycznym
  prowadzącym działalność gospodarczą, w tym również w formie spółki prawa
  cywilnego, spółki komandytowej lub spółki jawnej, uprawnionym na mocy
  odrębnych przepisów do szkolenia uczniów i zatrudniającym w ramach
  prowadzonej działalności pracowników w celu nauki zawodu.
3. Ulga uczniowska przysługuje, jeżeli szkolenie zostało zakończone
  pozytywnym wynikiem egzaminu.
4. Ulga uczniowska nie przysługuje za wyszkolenie pracownika będącego już
  wykwalifikowanym pracownikiem (czeladnikiem) w tym samym lub innym
  zawodzie.
5. Wysokość ulgi uczniowskiej za wyszkolenie jednego pracownika zależy od
  okresu szkolenia wynikającego z umowy o pracę.
6. Ulga uczniowska z tytułu wyszkolenia jednego pracownika stanowi
  równowartość:
   1) sześciokrotności najniższego wynagrodzenia miesięcznego pracowników,
     określonego za miesiąc, w którym zakończono szkolenie pozytywnym
     wynikiem egzaminu - przy okresie szkolenia do 24 miesięcy,
   2) dziesięciokrotności miesięcznego wynagrodzenia, o którym mowa w pkt 1 przy
okresie szkolenia ponad 24 miesiące.
7. Ulgę uczniowską stosuje się od miesiąca następującego po miesiącu, w
  którym została wydana decyzja o przyznaniu ulgi.
8. Kwoty ulg uczniowskich, określone w ust. 6, podwyższa się o 20%
  podatnikom, którzy prowadzą działalność gospodarczą w miejscowościach o
  liczbie mieszkańców do 5.000 oraz w gminach o szczególnym zagrożeniu
  wysokim bezrobociem strukturalnym, określonych w odrębnych przepisach.
  Podwyżka ulgi uczniowskiej o 20% przysługuje także wtedy, gdy po
  zakończeniu szkolenia gmina, w której prowadzona jest działalność,
  wykreślona została z wykazu gmin zagrożonych szczególnie wysokim
  bezrobociem strukturalnym.
9. Kwoty ulg uczniowskich, określone w ust. 6 i 8 dodatkowo podwyższa się o
  20% podatnikom, którzy w danym roku podatkowym zakończyli szkolenie więcej
  niż jednego pracownika; podwyżka przysługuje z tytułu zakończenia szkolenia
  pozytywnym wynikiem egzaminu każdego następnego pracownika.
10. Ulga uczniowska przysługuje, jeżeli zatrudnienie pracownika w celu nauki
  zawodu trwało przez cały okres szkolenia wynikający z umowy o pracę w celu
  nauki zawodu.
11. Jeżeli umowa o pracę w celu przygotowania zawodowego została rozwiązana z
  przyczyn niezależnych od podatnika, a pracownik podjął naukę zawodu na
  podstawie umowy o pracę w celu przygotowania zawodowego u innego podatnika
  - przysługującą z tytułu wyszkolenia ulgę uczniowską dzieli się między
  obydwu podatników, proporcjonalnie do liczby miesięcy prowadzonego przez
  nich szkolenia. Jeżeli rozwiązanie umowy nastąpiło z winy szkolącego
  podatnika, ulga uczniowska przysługuje w wysokości proporcjonalnej do
  okresu szkolenia tylko temu podatnikowi, u którego pracownik ukończył naukę
  zawodu.
12. Jeżeli działalność jest prowadzona w formie spółki, ulga uczniowska
  przysługuje wspólnikom proporcjonalnie do ich udziału w dochodach spółki. W
  przypadku zmiany wspólnika w ciągu okresu szkolenia, przepis ust. 11
  stosuje się odpowiednio.
13. Przepisy ust. 1-12 stosuje się odpowiednio do podatników, którzy na
  podstawie umowy zawartej ze szkołą prowadzą zajęcia praktyczne lub praktyki
  zawodowe uczniów zasadniczych szkół zawodowych, techników, liceów
  zawodowych i policealnych szkół zawodowych dla niepracujących, zwane dalej
  "praktyczną nauką zawodu", pod warunkiem że praktyczna nauka zawodu trwała
  nie krócej niż 10 miesięcy.
14. Wniosek o przyznanie ulg uczniowskich, o których mowa w ust. 1-13,
  podatnik jest obowiązany złożyć w urzędzie skarbowym właściwym według
  miejsca zamieszkania podatnika, w terminie miesiąca od daty złożenia przez
  pracownika egzaminu kończącego naukę zawodu lub od daty zakończenia przez
  ucznia odbywania u podatnika praktycznej nauki zawodu. Podatnik jest
  obowiązany dołączyć do wniosku dokument stwierdzający datę złożenia przez
  pracownika egzaminu kończącego naukę zawodu wynikiem pozytywnym, a w
  przypadku, o którym mowa w ust. 11, również oświadczenie stwierdzające
  przyczyny krótszego okresu szkolenia. Wniosek o ulgę uczniowską, o której
  mowa w ust. 13, powinien zawierać następujące dane: datę zawarcia ze szkołą
  umowy o praktyczną naukę zawodu, imię i nazwisko ucznia, datę rozpoczęcia i
  zakończenia przez ucznia odbywania u podatnika praktycznej nauki zawodu.
                                       
                                   Art. 54.
1. Poczynając od roku podatkowego 2000 kwoty określone w art. 6 ust. 2 pkt 3
  lit. b), ust. 4 pkt 1 i ust. 6 oraz stawki określone w załącznikach nr 3, 5
  i 6, ulegają corocznie podwyższeniu w stopniu odpowiadającym wskaźnikowi
  wzrostu cen konsumpcyjnych towarów i usług w okresie pierwszych trzech
  kwartałów roku poprzedzającego rok podatkowy w stosunku do tego samego
  okresu roku ubiegłego, ogłaszanemu przez Prezesa Głównego Urzędu
  Statystycznego w Dzienniku Urzędowym Rzeczypospolitej Polskiej "Monitor
  Polski".
2. Przy obliczaniu kwot i stawek w sposób określony w ust. 1, z zastrzeżeniem
  ust. 3, pomija się końcówki wyrażone w groszach.
3. Przy obliczaniu stawek określonych w części I tabeli w zakresie usług
  parkingowych, części VI oraz częściach VIII-XI tabeli, stawki zaokrągla się
  do pełnych dziesiątek groszy w ten sposób, że kwoty wynoszące:
    1) mniej niż 5 groszy pomija się,
    2) 5 i więcej groszy podwyższa się do pełnych dziesiątek groszy.
                                       
                                   Art. 55.
Minister właściwy do spraw finansów publicznych w terminie do dnia 30
listopada roku poprzedzającego rok podatkowy ogłasza, w drodze obwieszczenia w
Dzienniku Urzędowym Rzeczypospolitej Polskiej "Monitor Polski", kwoty i
stawki, obliczone zgodnie z art. 54.
                                       
                                       
                                  Rozdział 6
      Zmiany w przepisach obowiązujących, przepisy przejściowe i końcowe
                                       
                                   Art. 56.
W ustawie z dnia 26 lipca 1991 r. o podatku dochodowym od osób fizycznych
(Dz.U. z 1993 r. Nr 90, poz. 416, Nr 134, poz. 646, z 1994 r. Nr 43, poz. 163,
Nr 90, poz. 419, Nr 113, poz. 547, Nr 123, poz. 602, z 1995 r. Nr 5, poz. 25,
Nr 133, poz. 654, z 1996 r. Nr 25, poz. 113, Nr 87, poz. 395, Nr 137, poz.
638, Nr 147, poz. 686, Nr 156, poz. 776, z 1997 r. Nr 28, poz. 153, Nr 30,
poz. 164, Nr 71, poz. 449, Nr 85, poz. 538, Nr 96, poz. 592, Nr 121, poz. 770,
Nr 123, poz. 776, Nr 137, poz. 926, Nr 139, poz. 932, 933 i 934, Nr 141, poz.
943 i 945, z 1998 r. Nr 66, poz. 430, Nr 74, poz. 471, Nr 108, poz. 685, Nr
117, poz. 756) wprowadza się następujące zmiany:
1) w art. 9 w ust. 3 wyrazy "w art. 30 ust. 1 pkt 6" zastępuje się wyrazami
  "w ustawie z dnia 20 listopada 1998 r. o zryczałtowanym podatku dochodowym
  od niektórych przychodów osiąganych przez osoby fizyczne (Dz.U. Nr ...,
  poz. ...), zwanej dalej "ustawą o zryczałtowanym podatku dochodowym"";
2) w art. 26:
   a) w ust. 5 wyrazy "zryczałtowanym podatkiem dochodowym na podstawie art.
     30 ust. 1 pkt 6" zastępuje się wyrazami "ryczałtem od przychodów
     ewidencjonowanych na podstawie ustawy o zryczałtowanym podatku
     dochodowym",
   b) w ust. 13 wyrazy "art. 30 ust. 4" zastępuje się wyrazami "art. 11
     ustawy o zryczałtowanym podatku dochodowym",
   c) dodaje się ust. 13a w brzmieniu:
      "13a. Wydatki na cele określone w ust. 1 podlegają odliczeniu od
        dochodu, jeżeli nie zostały odliczone od przychodów na podstawie art.
        11 ustawy o zryczałtowanym podatku dochodowym.";
3) w art. 27a:
   a) w ust. 7 w pkt 2 kropkę na końcu zastępuje się przecinkiem i dodaje pkt
     3 w brzmieniu:
      "3) wydatki na cele określone w ust. 1 nie zostały odliczone od ryczałtu
        od przychodów ewidencjonowanych na podstawie art. 14 ustawy o
        zryczałtowanym podatku dochodowym.",
   b) w ust. 16 wyrazy "art. 30 ust. 4" zastępuje się wyrazami "art. 14
     ustawy o zryczałtowanym podatku dochodowym";
4) w art. 30:
   a) w ust. 1 skreśla się pkt 6 i 8,
   b) ust. 2 skreśla się,
   c) w ust. 3 wyrazy "pkt 1-6 i 8" zastępuje się wyrazami "pkt 1-5 oraz 9 i
     11",
   d) ust. 4 skreśla się,
   e) w ust. 5 skreśla się pkt 2,
   f) ust. 6 i 7 skreśla się;
5) w art. 41 w ust. 4 w pkt 2 wyrazy "oraz w pkt 8, jeżeli tak stanowią
  odrębne przepisy" skreśla się.
                                       
                                       
                                   Art. 57.
1. Wniosek o zastosowanie opodatkowania w formie karty podatkowej od 1
  stycznia 1999 r. może być złożony nie później niż do dnia 20 stycznia 1999
  r.
2. Obowiązek złożenia wniosku, o którym mowa w ust. 1, nie dotyczy
  podatników, którzy do końca 1998 r. prowadzili działalność gospodarczą i
  byli opodatkowani w formie karty podatkowej.
3. Zrzeczenie, z opodatkowania w formie ryczałtu od 1 stycznia 1999 r., o
  którym mowa w art. 51 ust. 1, może nastąpić nie później niż do dnia 20
  stycznia 1999 r.
                                       
                                       
                                   Art. 58.
Ustawa wchodzi w życie z dniem 1 stycznia 1999 r., z tym że przepisy art. 16,
art. 35 ust. 4, art. 52 ust. 2 oraz art. 56 pkt 4 lit. e) wchodzą w życie z
dniem ogłoszenia.
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                          Załączniki do ustawy
                                   z dnia 20 listopada 1998 r. (poz. ........)
                                                                              
                                                                Załącznik nr 1
 Wykaz wyrobów rękodzieła ludowego i artystycznego (atestowanych przez komisje
         artystyczne i etnograficzne) objętych ryczałtem od przychodów
                               ewidencjonowanych
Objaśnienie: wyróżnik "ex" oznacza, że zakres wyrobów objętych ryczałtem od
        przychodów ewidencjonowanych jest węższy niż określony w danym
        grupowaniu Polskiej Klasyfikacji Wyrobów i Usług.
Część I. Wyroby rękodzieła ludowego i artystycznego z metalu
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |               Nazwa wyrobu              |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                  - 3 -                  |
+----+---------------+-----------------------------------------+
| 1. | 28.61.1 ex    | Wyroby nożownicze                       |
| 2. | 28.63.14 ex   | Zawiasy, zamocowania, obsady i podobne  |
|    |               | wyroby do pojazdów mechanicznych, drzwi,|
|    |               | okien, mebli, itp., wykonane z metali   |
|    |               | nieszlachetnych                         |
| 3. | 28.75.24 ex   | Statuetki i pozostałe wyroby dekoracyjne|
|    |               | oraz ramki do fotografii, obrazków itp.,|
|    |               | ramki z lusterkami, wykonane z metali   |
|    |               | pospolitych                             |
| 4. | 28.75.27 ex   | Wyroby z metali pospolitych pozostałe,  |
|    |               | gdzie indziej nie sklasyfikowane        |
| 5. | 28.75.27-43.3 | Wyroby kowalstwa artystycznego          |
| 6. | 28.75.27-59.8 | Wyroby brązownicze i kowalstwa          |
|    |               | artystycznego                           |
| 7. | 31.50.2 ex    | Lampy i osprzęt oświetleniowy           |
| 8. | 33.50.26 ex   | Koperty do zegarków, obudowy do zegarów |
|    |               | i ich części                            |
| 9. | 36.21.10-20.0 | Medale                                  |
|    | ex            |                                         |
| 10.| 36.22.13 ex   | Wyroby jubilerskie oraz ich części;     |
|    |               | wyroby jubilerskie ze złota lub srebra  |
|    |               | oraz ich części                         |
| 11.| 36.22.14 ex   | Wyroby wykonane z metali szlachetnych   |
|    |               | pozostałe; wyroby z pereł naturalnych   |
|    |               | lub hodowlanych, kamieni szlachetnych   |
|    | z wyjątkiem:  | lub półszlachetnych                     |
|    | 36.22.14-70.0 | Katalizatory, w postaci drucianej       |
|    |               | tkaniny lub siatki, z platyny           |
| 12.| 36.61.10 ex   | Biżuteria sztuczna                      |
+----+---------------+-----------------------------------------+
Część II. Wyroby rękodzieła ludowego i artystycznego z ceramiki
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 26.21.11 ex   | Zastawy stołowe, naczynia kuchenne i    |
|    |               | inne artykuły gospodarstwa domowego oraz|
|    |               | artykuły toaletowe z porcelany          |
| 2. | 26.21.12 ex   | Zastawy stołowe, naczynia kuchenne i    |
|    |               | inne artykuły gospodarstwa domowego     |
|    |               | (poza porcelaną)                        |
| 3. | 26.21.13 ex   | Statuetki i inne dekoracyjne wyroby     |
|    |               | ceramiczne                              |
| 4. | 26.25.12 ex   | Wyroby ceramiczne pozostałe, poza       |
|    |               | budowlanymi, gdzie indziej nie          |
|    |               | sklasyfikowane                          |
| 5. | 31.50.2 ex    | Lampy i osprzęt oświetleniowy           |
| 6. | 33.50.26 ex   | Koperty do zegarków, obudowy do zegarów |
|    |               | i ich części                            |
| 7. | 36.61.10 ex   | Biżuteria sztuczna                      |
| 8. | 36.63.61-39 ex| Fajki i cybuchy z innych materiałów     |
| 9. | 36.63.61-50 ex| Cygarniczki do papierosów i cygar oraz  |
|    |               | ich części, części fajek                |
| 10.| 92.31.10-00.22| Rzeźby, posągi z dowolnego materiału,   |
|    | ex            | oryginalne, współczesne                 |
| 11.| 92.31.10-00.42| Wyroby artystyczne z ceramiki,          |
|    | ex            | współczesne                             |
+----+---------------+-----------------------------------------+
Część III. Wyroby rękodzieła ludowego i artystycznego ze szkła
       
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 26.13.12      | Kieliszki i szklanki poza szklano-      |
|    |               | ceramicznymi                            |
| 2. | 26.13.13      | Wyroby ze szkła w rodzaju używanych do  |
|    |               | celów stołowych, kuchennych,            |
|    |               | toaletowych, biurowych, do dekoracji    |
|    |               | wnętrz itp.                             |
| 3. | 26.15.24      | Części szklane do lamp i osprzętu       |
|    |               | oświetleniowego, podświetlane płyty z   |
|    |               | nazwami firm itp.                       |
| 4. | 26.15.26      | Wyroby szklane, gdzie indziej nie       |
|    |               | sklasyfikowane                          |
| 5. | 36.61.10-90.1 | Biżuteria sztuczna szklana              |
| 6. | 92.31.10-00.43| Wyroby artystyczne ze szkła, współczesne|
+----+---------------+-----------------------------------------+
Część IV. Wyroby rękodzieła ludowego i artystycznego z drewna, wikliny, słomy,
        rogożyny, trzciny, łuby oraz papieru
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 20.40.12-50.1 | Beczki i komplety beczkowe z drewna     |
|    |               | iglastego                               |
| 2. | 20.40.12-50.2 | Beczki i komplety beczkowe z drewna     |
|    |               | liściastego                             |
| 3. | 20.40.12-50.3 | Baryłki do płynów drewniane             |
| 4. | 20.51.12      | Przybory stołowe i kuchenne drewniane   |
| 5. | 20.51.13      | Intarsje i inkrustacje drewniane, kasety|
|    |               | drewniane na biżuterię lub sztućce,     |
|    |               | statuetki i inne ozdoby drewniane       |
| 6. | 20.51.14      | Ramy do obrazów, fotografii, luster lub |
|    |               | podobnych przedmiotów drewniane i       |
|    |               | pozostałe artykuły z drewna             |
| 7. | 20.52.15      | Wyroby ze słomy, lub innych materiałów  |
|    |               | używanych do wyplatania; wyroby         |
|    | z wyjątkiem:  | koszykarskie i plecionkarskie           |
|    | 20.52.15-     |                                         |
|    | 30.31,        | Maty z trzciny pospolitej izolacyjne    |
|    | 20.52.15-     | Maty z trzciny pospolitej podtynkowe    |
|    | 30.32,        | Maty ze słomy izolacyjne dla budownictwa|
|    | 20.52.15-     | Wyroby z materiałów do wyplatania       |
|    | 30.40,        | pozostałe (z wyjątkiem roślinnych),     |
|    | 20.52.15-59.00| gdzie indziej nie sklasyfikowane        |
| 8. | 21.22.11-50   | Serwetki i obrusy z papieru, waty       |
|    |               | celulozowej lub bibułki tissue          |
| 9. | 21.22.12-90.12| Wyroby do użytku domowego z papieru,    |
|    |               | waty celulozowej, lub bibułki tissue -  |
|    |               | firanki                                 |
| 10.| 21.25.14-77.10| Koronki i hafty z papieru lub tektury   |
| 11.| 22.22.20-17.00| Pamiętniki                              |
| 12.| 22.22.20-80   | Albumy na próbki lub do kolekcji, z     |
|    |               | papieru lub tektury                     |
| 13.| 36.11.12      | Meble do siedzenia o konstrukcji głównie|
|    |               | drewnianej                              |
| 14.| 36.13.10-90.30| Meble kuchenne drewniane regionalne i   |
|    |               | artystyczne                             |
| 15.| 36.14.12      | Meble drewniane do sypialni, jadalni    |
|    |               | oraz drewniane meble salonowe           |
| 16.| 36.14.13      | Meble drewniane, gdzie indziej nie      |
|    |               | sklasyfikowane                          |
| 17.| 36.14.14      | Meble z tworzyw sztucznych oraz meble   |
|    |               | wykonane z innych materiałów, łącznie z |
|    |               | meblami trzcinowymi, wiklinowymi,       |
|    |               | bambusowymi lub wykonanymi z podobnych  |
|    |               | materiałów                              |
| 18.| 36.61.10-90.91| Biżuteria sztuczna, pozostała - z drewna|
| 19.| 36.63.61-35-10| Fajki i cybuchy z drewna lub korzenia - |
|    |               | o charakterze pamiątkarskim             |
| 20.| 36.63.61-50   | Cygarniczki do papierosów i cygar oraz  |
|    |               | ich części, części fajek                |
+----+---------------+-----------------------------------------+
Część V. Wyroby rękodzieła ludowego i artystycznego włókiennicze
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 17.20.10 ex   | Tkaniny (z wyjątkiem tkanin specjalnych)|
|    |               | z włókien naturalnych innych niż        |
|    |               | bawełniane                              |
| 2. | 17.20.20 ex   | Tkaniny bawełniane (z wyjątkiem tkanin  |
|    |               | specjalnych)                            |
| 3. | 17.20.40 ex   | Plusz, welwet, inne tkaniny strzyżone,  |
|    |               | tkaniny ręcznikowe i inne specjalne     |
|    | z wyjątkiem:  |                                         |
|    | 17.20.40-70   | Gaza (bez zaliczonej do 17.54.11.30)    |
|    | 17.20.40-90   | Tkaniny z włókien szklanych (łącznie z  |
|    |               | tkaninami wąskimi)                      |
| 4. | 17.40.11 ex   | Koce (z wyjątkiem kocy elektrycznych) i |
|    |               | pledy turystyczne                       |
| 5. | 17.40.12 ex   | Bielizna pościelowa                     |
| 6. | 17.40.13 ex   | Bielizna stołowa                        |
| 7. | 17.40.14 ex   | Bielizna do celów toaletowych i         |
|    |               | kuchennych                              |
| 8. | 17.40.15 ex   | Zasłony (łącznie z draperiami), rolety; |
|    |               | zasłony lub lambrekiny łóżkowe          |
| 9. | 17.40.16 ex   | Wyroby wyposażenia wnętrz pozostałe,    |
|    |               | gdzie indziej nie sklasyfikowane;       |
|    |               | zestawy tkanin i przędzy do wykonywania |
|    |               | dywaników, gobelinów itp.               |
| 10.| 17.51.11 ex   | Dywany i inne włókiennicze wykładziny   |
|    |               | podłogowe, wiązane                      |
| 11.| 17.51.12 ex   | Dywany i inne włókiennicze wykładziny   |
|    |               | podłogowe, tkane                        |
| 12.| 17.51.13 ex   | Dywany i inne włókiennicze wykładziny   |
|    |               | podłogowe, z nitkami powiązanymi w      |
|    |               | pęczki                                  |
| 13 | 17.51.14 ex   | Dywany i pozostałe włókiennicze         |
|    |               | wykładziny podłogowe, gdzie indziej nie |
|    |               | sklasyfikowane (w tym z filcu)          |
| 14.| 17.54.11 ex   | Tkaniny wąskie; wyroby wykończeniowe    |
|    |               | itp.                                    |
| 15.| 17.54.12 ex   | Tiul i inne tkaniny siatkowe, tkaniny o |
|    |               | ściegu dzianinowym lub szydełkowym;     |
|    |               | koronki w sztukach, w taśmach lub w     |
|    |               | postaci motywu                          |
| 16.| 17.54.13 ex   | Hafty w sztukach, taśmach lub w postaci |
|    |               | motywu                                  |
+----+---------------+-----------------------------------------+
Część VI. Wyroby rękodzieła ludowego i artystycznego odzieżowe
        
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 17.40.12 ex   | Bielizna pościelowa                     |
| 2. | 17.40.13 ex   | Bielizna stołowa                        |
| 3. | 17.71 ex      | Trykotaże                               |
| 4. | 17.72 ex      | Pulowery, swetry i podobne wyroby dziane|
|    |               | lub szydełkowe                          |
| 5. | 18.22ex       | Ubiory wierzchnie pozostałe             |
|    | z wyjątkiem:  |                                         |
|    | 18.22.4       | Odzież używana i inne używane wyroby    |
|    |               | włókiennicze                            |
| 6. | 18.23 ex      | Bielizna                                |
|    | z wyjątkiem:  |                                         |
|    | 18.23.25      | Biustonosze, pasy, gorsety, szelki,     |
|    |               | podwiązki i podobne artykuły oraz ich   |
|    |               | części                                  |
| 7. | 18.24 ex      | Odzież i dodatki odzieżowe pozostałe,   |
|    | z wyjątkiem:  | gdzie indziej nie sklasyfikowane        |
|    | 18.24.12      |                                         |
|    |               | Dresy, ubiory narciarskie, stroje       |
|    |               | kąpielowe i inna odzież, z materiałów o |
|    |               | splocie dzianinowym lub szydełkowym     |
+----+---------------+-----------------------------------------+
Część VII. Wyroby rękodzieła ludowego i artystycznego skórzane
        
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 18.10.10 ex   | Odzież ze skóry lub skóry wtórnej       |
| 2. | 18.30.12 ex   | Odzież, dodatki odzieżowe oraz inne     |
|    |               | wyroby wykonane ze skór futerkowych (z  |
|    |               | wyjątkiem nakryć głowy)                 |
| 3. | 19.20.11 ex   | Wyroby rymarskie przeznaczone dla       |
|    |               | każdego rodzaju zwierząt, wykonane z    |
|    |               | dowolnego materiału                     |
| 4. | 19.20.14-50.00| Wyroby ze skóry lub skóry wtórnej, gdzie|
|    | ex            | indziej nie sklasyfikowane              |
| 5. | 19.30 ex      | Obuwie                                  |
| 6. | 92.31.10-00.46| Wyroby artystyczne ze skóry, współczesne|
|    | ex            |                                         |
+----+---------------+-----------------------------------------+
Część VIII. Wyroby rękodzieła ludowego i artystycznego osobno nie wymienione
         
+----+-------------------+-----------------------------------------+       
| Lp.|   Symbol PKWiU    |              Nazwa wyrobu               |
+----+-------------------+-----------------------------------------+
- 1 -|     - 2 -         |                 - 3 -                   |
+----+-------------------+-----------------------------------------+
| 1. | 33.20.32-55.00 ex | Przyrządy traserskie                    |
| 2. | 36.30.12-90.20 ex | Mandoliny i mandole                     | 
| 3. | 36.30.12-90.30 ex | Banjo                                   |
| 4. | 36.30.12-90.90 ex | Instrumenty muzyczne strunowe pozostałe,|
|    |                   | gdzie indziej nie wymienione            |
| 5. | 92.31.10-00.12 ex | Obrazy, rysunki, pastele, grafiki,      |
|    |                   | plakaty, druki reklamowe wykonane       |
|    |                   | ręcznie, współczesne                    |
| 6. | 92.31.10-00.22 ex | Rzeźby, posągi z dowolnego materiału,   |
|    |                   | oryginalne, współczesne                 |
+----+-------------------+-----------------------------------------+
                                                                              
                                                                Załącznik nr 2
                                                                              
                                                                              
 Wykaz usług, których świadczenie wyłącza podatnika z opodatkowania ryczałtem
   od przychodów ewidencjonowanych, oznaczonych według Polskiej Klasyfikacji
                                Wyrobów i Usług
                                       
Objaśnienie: wyróżnik "ex" oznacza, że zakres usługi wymieniony w załączniku
        jest węższy niż określony w grupowaniu Polskiej Klasyfikacji Wyrobów
        i Usług.
+----+---------------+-----------------------------------------+       
| Lp.| Symbol PKWiU  |              Nazwa wyrobu               |
+----+---------------+-----------------------------------------+
- 1 -|     - 2 -     |                 - 3 -                   |
+----+---------------+-----------------------------------------+
| 1. | 22.33.10      | Usługi reprodukcji komputerowych        |
|    |               | nośników informacji                     |
| 2. | 50.10.30      | Usługi pośrednictwa w sprzedaży pojazdów|
|    |               | mechanicznych                           |
| 3. | 50.30.30      | Usługi pośrednictwa w sprzedaży części i|
|    |               | akcesoriów do pojazdów mechanicznych    |
| 4. | 50.40.30      | Usługi pośrednictwa w sprzedaży         |
|    |               | motocykli oraz części i akcesoriów do   |
|    |               | nich                                    |
| 5. | grupa 51.1    | Usługi pośrednictwa w handlu hurtowym   |
| 6. | grupa 55.1    | Usługi hoteli                           |
| 7. | grupa 55.2    | Usługi świadczone przez obiekty         |
|    |               | noclegowe turystyki oraz inne miejsca   |
|    |               | krótkotrwałego zakwaterowania           |
| 8. | 63.12.12      | Usługi magazynowania i przechowywania   |
|    |               | cieczy i gazów                          |
|  9 | 63.12.14 -    | Usługi magazynowania i przechowywania   |
|    | 00.30         | towarów w strefach wolnocłowych         |
| 10.| 63.21.24      | Usługi parkingowe                       |
| 11.| 63.21.25 -    | Usługi obsługi centrali wzywania radio- |
|    | 00.10         | taxi                                    |
| 12.| 63.22.12      | Usługi pilotowania                      |
| 13.| grupa 63.3    | Usługi organizatorów i pośredników      |
|    |               | turystycznych                           |
| 14.| sekcja J      | Usługi pośrednictwa finansowego         |
| 15.| sekcja K      | Usługi związane  z nieruchomościami,    |
|    |               | wynajem, nauka i prowadzenie            |
|    | z wyjątkiem:  | działalności gospodarczej               |
|    | 71.3          | Usługi wynajmu pozostałych maszyn i     |
|    |               | urządzeń                                |
|    | 71.4          | Usługi wypożyczania artykułów użytku    |
|    |               | osobistego i domowego, gdzie indziej nie|
|    |               | sklasyfikowane                          |
|    | 72.40.10 ex   | Usługi baz danych - dotyczy wyłącznie   |
|    |               | baz danych dotyczących informacji       |
|    |               | handlowej i usługowej                   |
|    \ 72.5          | Usługi konserwacji i napraw maszyn      |
|    |               | biurowych, księgujących i liczących     |
|    | 74.40.12 ex   | Usługi planowania, opracowywania        |
|    |               | i zamieszczania reklam - dotyczy        |
|    |               | wyłącznie reklamy handlowej             |
|    | 74.70.12      | Usługi mycia okien                      |
|    | 74.70.13      | Usługi sprzątania tradycyjnego          |
|    | 74.70.14      | Usługi sprzątania specjalistycznego     |
|    | 74.70.15      | Usługi czyszczenia pieców i kominów     |
|    | 74.70.16      | Usługi sprzątania i czyszczenia         |
|    |               | obiektów, pozostałe                     |
|    | 74.82         | Usługi związane z pakowaniem            |
|    | 74.84.15      | Usługi organizowania wystaw, targów     |
|    |               | i kongresów                             |
| 16.| sekcja L      | Usługi w zakresie administracji         |
|    |               | publicznej, obrony narodowej,           |
|    | z wyjątkiem:  | obowiązkowych ubezpieczeń społecznych   |
|    | 75.25.11      | Usługi związane ze zwalczaniem pożarów  |
|    |               | i zapobieganiem pożarom                 |
|    | 75.25.12 ex   | Usługi ochrony przeciwpożarowej         |
|    |               | pozostałe - dotyczy wyłącznie pozostałej|
|    |               | działalności straży pożarnej            |
| 17.| sekcja M.     | Usługi w zakresie edukacji              |
|    | z wyjątkiem:  |                                         |
|    | 80.10.11 ex   | Usługi wychowania przedszkolnego -      |
|    |               | dotyczy wyłącznie przygotowania dzieci  |
|    |               | do nauki w szkole                       |
| 18.| 85.1          | Usługi w zakresie ochrony zdrowia       |
|    |               | ludzkiego                               |
| 19.| 85.32.13      | Usługi poradnictwa dla dzieci, gdzie    |
|    |               | indziej nie sklasyfikowane              |
|    | 85.32.14      | Usługi opieki społecznej świadczone     |
|    |               | przez instytucje o charakterze doraźnym |
|    | 85.32.15      | Usługi opieki społecznej związane z     |
|    |               | przystosowaniem zawodowym               |
| 20.| dział 91      | Usługi świadczone przez organizacje     |
|    |               | członkowskie, gdzie indziej nie         |
|    |               | sklasyfikowane                          |
| 21.| dział 92      | Usługi związane z rekreacją, kulturą i  |
|    | z wyjątkiem:  | sportem                                 |
|    |  92.53        |                                         |
|    |               | Usługi związane z ogrodami botanicznymi |
|    |               | i zoologicznymi oraz obszarami z        |
|    |               | obiektami chronionej przyrody           |
| 22.| 93.05 ex      | Usługi pozostałe, gdzie indziej nie     |
|    |               | sklasyfikowane - dotyczy wyłącznie biur |
|    |               | matrymonialnych, opieki doraźnej,       |
|    |               | robienia zakupów, działalności          |
|    |               | wróżbitów, astrologów, działalności w   |
|    |               | zakresie usług towarzyskich.            |
+----+---------------+-----------------------------------------+
                                                                              
                                                                Załącznik nr 3
                                       
                                       
                TABELA MIESIĘCZNYCH STAWEK PODATKU DOCHODOWEGO
                                       
                     W FORMIE KARTY PODATKOWEJ (w złotych)
                                       
                                       
            Część I. Działalność usługowa oraz wytwórczo - usługowa
                                       
                                       
Objaśnienia:
 1. Stawki oznaczone w rubryce 3:
 - "0" dotyczą działalności prowadzonej bez zatrudnienia pracowników,
 - "1" dotyczą działalności prowadzonej przy zatrudnieniu 1 pracownika,
 - "2" dotyczą działalności prowadzonej przy zatrudnieniu 2 pracowników,
 - "3" dotyczą działalności prowadzonej przy zatrudnieniu 3 pracowników,
 - "4" dotyczą działalności prowadzonej przy zatrudnieniu 4 pracowników,
 - "5" dotyczą działalności prowadzonej przy zatrudnieniu 5 pracowników.
 2. Za pracowników nie uważa się osób wymienionych w art. 25 ust. 6 pkt 2
   ustawy.
 3. Przy prowadzeniu działalności wymienionej w tej części tabeli
   dopuszczalne jest - poza usługami, o których mowa w art. 4 pkt 1 ustawy wytwarzanie:

  a) przedmiotów (wyrobów) z materiału powierzonego przez zamawiającego,
  b) przedmiotów (wyrobów) na indywidualne zamówienie z materiałów własnych
    wykonawcy, w tym także połączone z czynnościami ich instalowania,
    uwzględniających indywidualne cechy przedmiotów (wyrobów) lub życzenia
    zamawiającego,
 - jeżeli przedmioty (wyroby) te nie są przeznaczone przez zamawiającego do
 dalszej odprzedaży lub nie służą zamawiającemu do celów wytwórczych lub
 usługowych jako surowce, materiały, półfabrykaty, elementy, części.
 4. Przy prowadzeniu działalności wymienionej w poz. 70 - 76 dopuszczalne
   jest - poza świadczeniami dla ludności - wykonywanie świadczeń dla innych
   odbiorców w rozmiarze nie przekraczającym kwoty 48.000 zł przychodu
   rocznie.
 5. Za świadczenia dla ludności uważa się opłacane ze środków pieniężnych
   ludności świadczenia określone w ust. 3 oraz usługi, o których mowa w art.
   4 pkt 1 ustawy. Przez pojęcie "ludność" rozumie się osoby fizyczne i
   gospodarstwa domowe oraz inne podmioty, tj. indywidualne gospodarstwa
   rolne i ogrodnicze, zakłady wytwórcze, budowlane, handlowe i usługowe
   prowadzone przez osoby fizyczne i spółki nie mające osobowości prawnej
   oraz zakłady prowadzone przez agentów. Pojęcie "ludność" obejmuje również
   organizacje wyznaniowe i zakonne, placówki państw obcych (wyłącznie w
   rozumieniu służb dyplomatycznych i konsularnych) oraz rady szkół i
   placówek, rady rodziców i inaczej nazwane reprezentacje rodziców.
 6. Pojęcie świadczeń dla ludności nie obejmuje czynności związanych
   z wytwarzaniem wyrobów (w tym półfabrykatów, elementów, części, obróbki
   elementów) przeznaczonych do celów produkcyjnych lub do dalszej odprzedaży
   oraz wykonywania robót budowlano-montażowych w ramach tzw. podwykonawstwa,
   jeżeli inwestorem są inne osoby i jednostki niż wymienione w ust. 5.
 7. Zakres działalności oznaczonej w rubryce 2 tabeli symbolem "*" obejmuje
   działalność usługową i wytwórczo - usługową.
 8. Przy prowadzeniu działalności wymienionej w poz. 79 w zakresie zbiorowego
   wypasu owiec w rejonach górskich w sezonie letnim, stawki określone w
   rubrykach 4-6 dotyczą działalności przy zatrudnieniu nie przekraczającym 4
   pracowników.
+---+--------------------------+-------+-------------------------------+
|Lp.|   Zakres działalności    | Ozna- |   W miejscowości o liczbie    |
|   |                          | czenie|          mieszkańców          |
|   |                          | stawek+---------+----------+----------+
|   |                          |       |do 5000  |ponad 5000| powyżej  |
|   |                          |       |         | do 50000 |  50000   |
+---+--------------------------+-------+---------+----------+----------+
| 1 |           2              |   3   |   4     |    5     |    6     |
+---+--------------------------+-------+---------+----------+----------+
| 1   usługi ślusarskie            0      225       262        287     |
|                                  1      386       418        481     |
|                                  2      492       566        642     |
|                                  3      616       704        784     |
+----------------------------------------------------------------------+
| 2  usługi w zakresie wyrobu      0      163       176        200     |                         
|    i naprawy naczyń              1      308       333        355     |                         
|    blaszanych                    2      379       418        468     |                         
+----------------------------------------------------------------------+                                   
| 3  usługi rusznikarskie          0      114       128        137     |                         
|                                  1      212       250        275     |
+----------------------------------------------------------------------+                                 
| 4  usługi  w zakresie            0      262       287        287     |                          
|    złotnictwa - jubilerstwa      1      443       507        507     |                          
|    - dla ludności                2      666       741        741     |                          
+----------------------------------------------------------------------+                                  
| 5  usługi grawerskie             0      236       262        299     |                          
|                                  1      431       492        556     |
+----------------------------------------------------------------------+
| 6  usługi w zakresie             0      212       250        275     |                          
|    wyrobu pieczątek              1      395       431        492     |                          
+----------------------------------------------------------------------+                                
| 7  usługi pobielania kotłów      0      468       468        468     |                          
|    i naczyń                      1      912       912        912     |
+----------------------------------------------------------------------+
| 8  usługi kowalskie              0      114       128        137     |                          
|                                  1      225       262        287     |
|                                  2      308       333        355     |
|                                  3      340       371        402     |
+----------------------------------------------------------------------+
| 9  usługi w zakresie             0      236       275        299     |                          
|    elektromechaniki              1      431       492        556     |                          
|    chłodniczej                   2      654       747        819     |                          
|                                  3      819       903        983     |
+----------------------------------------------------------------------+
|10  usługi w zakresie             0      212       250        275     |                          
|    kotlarstwa                    1      379       418        468     |
+----------------------------------------------------------------------+
|11  usługi w zakresie             0      187       212        236     |                          
|    mechaniki maszyn              1      340       364        395     |                          
|                                  2      431       492        556     |
|                                  3      518       591        666     |
|                                  4      630       716        794     |
+----------------------------------------------------------------------+
|12  usługi w zakresie             0      114       137        152     |                         
|    mechaniki maszyn i            1      236       262        299     |                          
|    urządzeń rolniczych           2      317       348        371     |                          
|                                  3      364       395        431     |
|                                  4      402       456        518     |
+----------------------------------------------------------------------+
|13  usługi w zakresie             0      137       163        176     |                          
|    elektromechaniki sprzętu      1      308       340        364     |                          
|    medycznego                    2      395       431        492     |                          
|    i laboratoryjnego             3      481       543        616     |                          
+----------------------------------------------------------------------+                                
|14  usługi w zakresie             0      152       163        187     |                          
|    mechaniki precyzyjnej         1      308       333        355     |                          
|                                  2      379       418        468     |
|                                  3      443       507        566     |
+----------------------------------------------------------------------+
|15  usługi w zakresie             0      152       163        187     |                          
|    mechaniki maszyn              1      317       348        371     |                          
|    biurowych                                                         | 
+----------------------------------------------------------------------+                                
|16  usługi w zakresie wyrobu      0      187       212        236     |                          
|    i naprawy                     1      355       386        418     |                          
|    wag                                                               | 
+----------------------------------------------------------------------+                                
|17  usługi zegarmistrzowskie      0      114       137        152     |                          
|                                  1      250       275        308     |
|                                  2      324       355        379     |
|                                  3      371       411        456     |
+----------------------------------------------------------------------+
|18  usługi w zakresie             0      333       355        386     |                          
|    antykorozyjnego               1      642       730        801     |                          
|    zabezpieczania pojazdów       2      921      1011       1111     |                         
|                                  3     1178      1348       1518     |
+----------------------------------------------------------------------+
|19  usługi w zakresie mycia       0      200       225        250     |                         
|    oraz smarowania               1      379       418        468     |                          
|    samochodów                    2      507       581        654     |                          
|    i motocykli oraz inne         3      666       766        830     |                          
|    usługi nie związane z         4      766       848        921     |                          
|    ich naprawą                   5      893       983       1075     |                          
+----------------------------------------------------------------------+                                  
|20  usługi w zakresie             0      250       275        308     |                          
|    elektromechaniki, z           1      443       518        581     |                          
|    wyjątkiem dźwigowej,          2      654       754        819     |                          
|    chłodniczej i pojazdowej      3      784       856        939     |                          
+----------------------------------------------------------------------+                                                                
|21  naprawa i konserwacja         0      163       187        212     |                          
|    urządzeń                      1      333       355        386     |                          
|    elektronicznych               2      411       468        531     |
+----------------------------------------------------------------------+                                
|22  usługi w zakresie tele-       0      163       176        200     |                          
|    i radiomechaniki              1      317       348        371     |
|                                  2      395       431        492     |
|                                  3      518       591        666     |
|                                  4      630       716        794     |
+----------------------------------------------------------------------+
|23  usługi w zakresie             0      324       324        324     |                          
|    bieżnikowania opon            1      556       556        556     |                          
|                                  2      741       741        741     |
|                                  3      838       838        838     |
+----------------------------------------------------------------------+
|24  usługi wulkanizatorskie,      0      225       262        287     |                          
|    z wyjątkiem bieżnikowania     1      395       431        492     |                          
|    opon                          2      556       642        716     |                          
|                                  3      691       775        848     |
+----------------------------------------------------------------------+
|25  szklarstwo "*"                0      200       200        200     |
|                                  1      386       386        386     |
|                                  2      492       492        492     |
|                                  3      591       591        591     |
+----------------------------------------------------------------------+
|26  witrażownictwo "*"            0      152       163        187     |
|                                  1      308       340        364     |
|                                  2      379       418        468     |
|                                  3      386       492        556     |
+----------------------------------------------------------------------+
|27  usługi stolarskie, z          0      200       225        250     |                          
|    wyjątkiem wyrobu trumien      1      355       386        418     |                          
|                                  2      468       531        604     |
|                                  3      591       666        754     |
+----------------------------------------------------------------------+
|28  kołodziejstwo "*"             0      152       152        152     |
|                                  1      299       299        299     |
|                                  2      364       364        364     |
+----------------------------------------------------------------------+
|29  usługi w zakresie             0      128       152        163     |                          
|    pozłotnictwa                  1      262       299        324     |
|                                  2      333       355        386     |
|                                  3      364       395        431     |
+----------------------------------------------------------------------+
|30  usługi tapicerskie            0      137       163        176     |
|                                  1      308       340        364     |
|                                  2      386       418        481     |
|                                  3      443       518        581     |
+----------------------------------------------------------------------+
|31  bednarstwo "*"                0      102       102        102     |
|                                  1      212       212        212     |
|                                  2      287       287        287     |
+----------------------------------------------------------------------+
|32  koszykarstwo "*"              0      250       250        287     |                          
|                                  1      566       566        566     |
|                                  2      801       801        801     |
+----------------------------------------------------------------------+
|33  wytwarzanie mioteł,           0      137       137        137     |                          
|    wyrobów                       1      299       299        299     |                          
|    z gałęzi i słomy "*"                                              | 
+----------------------------------------------------------------------+                               
|34  gręplarstwo "*"               0       77        89        102     |
|                                  1      200       225        250     |
|                                  2      262       299        324     |
+----------------------------------------------------------------------+                               
|35  kilimiarstwo "*"              0      176       176        176     |
|                                  1      355       355        355     |
|                                  2      443       443        443     |
+----------------------------------------------------------------------+                               
|36  koronkarstwo,                 0       89        89         89     |                          
|    z wyjątkiem maszynowego "*"   1      200       200        200     |
+----------------------------------------------------------------------+                                 
|37  usługi hafciarskie            0      128       137        137     |
|                                  1      250       275        275     |
|                                  2      333       355        355     |
|                                  3      379       411        411     |
|                                  4      411       456        456     |
+----------------------------------------------------------------------+                               
|38  dziewiarstwo na drutach       0       53        53         53     |                          
|    i szydełkach "*"              1      137       137        137     |
|                                  2      187       187        187     |
+----------------------------------------------------------------------+                               
|39  repasacja pończoch            0       53        53         53     |
|                                  1      137       137        137     |
+----------------------------------------------------------------------+                               
|40  usługi w zakresie             0       89       102        114     |                          
|    bieliźniarstwa                1      212       236        262     |
|                                  2      287       317        340     |
+----------------------------------------------------------------------|                               
|41  usługi w zakresie             0       89       102        114     |                          
|    gorseciarstwa                 1      212       236        262     |
|                                  2      287       317        340     |
+----------------------------------------------------------------------+                               
|42  czapnictwo i                  0      137       137        137     |                          
|    kapelusznictwo męskie         1      275       275        275     |
|    "*"                           2      348       348        348     |
+----------------------------------------------------------------------+                                 
|43  modniarstwo"*"                0      152       152        152     |
|                                  1      299       299        299     |
|                                  2      371       371        371     |
+----------------------------------------------------------------------+
|44  usługi krawieckie             0       77        89        102     |
|                                  1      163       187        212     |
|                                  2      262       299        324     |
|                                  3      333       355        386     |
|                                  4      355       386        418     |
|                                  5      402       456        518     |
+----------------------------------------------------------------------+
|45  cerowanie i naprawa           0       53        53         53     |                          
|    odzieży, obciąganie           1      114       128        137     |                          
|    guzików, okrętkowanie,        2      152       163        187     |                          
|    mereżkowanie, plisowanie                                          | 
+----------------------------------------------------------------------+
|46  usługi w zakresie             0      176       200        200     |                          
|    kożuszkarstwa                 1      355       386        386     |
|                                  2      468       531        531     |
|                                  3      616       691        691     |
+----------------------------------------------------------------------+
|47  cholewkarstwo "*"             0      114       114        114     |
|                                  1      250       250        250     |
|                                  2      333       333        333     |
+----------------------------------------------------------------------+
|48  szewstwo miarowe              0      128       152        163     |
|                                  1      287       317        340     |
|                                  2      355       386        418     |
|                                  3      402       443        507     |
+----------------------------------------------------------------------+
|49  szewstwo ortopedyczne         0       89       102        114     |                          
|    "*"                           1      225       262        287     |
|                                  2      308       333        355     |
|                                  3      355       386        418     |
+----------------------------------------------------------------------+
|50  szewstwo naprawkowe           0       65        77         89     |
|                                  1      163       176        200     |
|                                  2      225       262        287     |
+----------------------------------------------------------------------+
|51  odświeżanie i renowacja       0      128       152        163     |                          
|    wyrobów skórzanych i          1      250       275        308     |                          
|    futrzarskich,                 2      333       355        386     |                          
|    z wyjątkiem mechanicznego     3      371       411        456     |                          
|    czyszczenia i farbowania                                          |
|    wyrobów futrzarskich                                              |
+----------------------------------------------------------------------+
|52  usługi w zakresie             0      187       212        236     |                          
|    mechanicznego                 1      340       371        402     |                          
|    czyszczenia i farbowania      2      431       492        556     |                          
|    wyrobów futrzarskich                                              |
+----------------------------------------------------------------------+
|53  usługi w zakresie             0      187       212        212     |                          
|    kuśnierstwa                   1      364       395        395     |
|                                  2      492       556        556     |
|                                  3      630       704        704     |
+----------------------------------------------------------------------+
|54  rymarstwo,                    0       89       102        114     |                          
|    z wyjątkiem produkcji         1      200       225        262     |                          
|    plandek "*"                   2      275       308        333     |
+----------------------------------------------------------------------+
|55  rękawicznictwo - wyrób        0      137       137        137     |                          
|    rękawiczek skórzanych i       1      275       275        275     |                          
|    skóropodobnych,               2      348       348        348     |                          
|    z wyjątkiem roboczych,                                            |
|    ochronnych i sportowych                                           |
|    "*"                                                               |
+----------------------------------------------------------------------+
|56  młynarstwo "*"                0       77        77         77     |
|                                  1      137       137        137     |
|                                  2      200       200        200     |
+----------------------------------------------------------------------+
|57  przemiał i przerób zbóż       0       53        53         53     |                          
|    w wiatrakach oraz             1       65        65         65     |                          
|    śrutowanie zboża "*"          2      102       102        102     |
+----------------------------------------------------------------------+
|58  wytwarzanie waty z cukru      0       53        65         65     |
|    "*", prażenie kukurydzy       1      128       137        137     |
|    "*"                                                               |
+----------------------------------------------------------------------+
|59  usługi w zakresie             0      137       163        176     |                          
|    introligatorstwa              1      308       333        355     |
|                                  2      379       418        468     |
+----------------------------------------------------------------------+
|60  usługi w zakresie             0      114       128        137     |                          
|    lutnictwa                     1      212       250        275     |
|                                  2      308       333        355     |
+----------------------------------------------------------------------+
|61  usługi w zakresie             0      137       163        176     |                          
|    organmistrzostwa              1      275       308        333     |                          
|                                  2      348       379        411     |
+----------------------------------------------------------------------+
|62  usługowy wyrób i naprawa      0      114       137        152     |                          
|    instrumentów dętych           1      236       262        299     |                          
|                                  2      317       348        371     |
+----------------------------------------------------------------------+
|63  usługowy wyrób i naprawa      0      137       163        176     |                          
|    instrumentów                  1      275       308        333     |                          
|    perkusyjnych                  2      340       371        402     |                          
+----------------------------------------------------------------------+
|64  usługowy wyrób i naprawa      0      114       137        152     |                          
|    instrumentów                  1      250       275        308     |                          
|    harmoniowych                  2      333       355        386     |                          
|    i fortepianowych                                                  |
+----------------------------------------------------------------------+
|65  usługowy wyrób                0      137       163        176     |                         
|    i naprawa instrumentów        1      299       324        348     |                          
|    muzycznych elektrycznych                                          |
|    i elektronicznych                                                 |
+----------------------------------------------------------------------+
|66  ortopedyka "*"                0       65        65         77     |
|                                  1      137       163        176     |
|                                  2      212       236        262     |
|                                  3      236       262        299     |
+----------------------------------------------------------------------+
|67  usługi w zakresie             0      163       163        163     |                          
|    lakiernictwa                  1      340       340        340     |
+----------------------------------------------------------------------+
|68  usługowe wytwarzanie          0      137       163        176     |                          
|    szyldów                       1      308       333        355     |
|                                  2      379       418        468     |
+----------------------------------------------------------------------+
|69  konserwacja i naprawa         0      152       163        187     |                          
|    przedmiotów zabytkowych       1      308       333        355     |                          
|    i artystycznych               2      379       418        468     |                          
|                                  3      443       518        581     |
+----------------------------------------------------------------------+
|70  usługi w zakresie robót       0      200       225        250     |                          
|    budowlanych: murarskich,      1      340       364        395     |                          
|    ciesielskich,                 2      468       531        604     |                          
|    dekarskich,                   3      654       741        819     |                          
|    posadzkarskich,               4      766       838        921     |                          
|    malarskich, związanych z      5      884       976       1066     |                          
|    wykładaniem                                                       |
|    i tapetowaniem ścian,                                             |
|    izolatorskich,                                                    |
|    związanych ze                                                     |
|    wznoszeniem                                                       |
|    i montażem konstrukcji                                            |
|    stalowych                                                         |
|    - dla ludności                                                    |
+----------------------------------------------------------------------+
|71  usługi w zakresie             0      152       163        187     |                          
|    sztukatorstwa                 1      262       299        324     |                          
|     - dla ludności               2      348       379        411     |
|                                  3      411       468        531     |
+----------------------------------------------------------------------+
|72  usługi w zakresie             0      200       225        250     |                          
|    odgrzybiania budynków         1      340       364        395     |                          
|     - dla ludności                                                   |
+----------------------------------------------------------------------+
|73  usługi w zakresie             0      152       163        187     |                          
|    cyklinowania                  1      308       333        355     |                          
|     - dla ludności                                                   |
+----------------------------------------------------------------------+
|74  usługi w zakresie             0      152       163        176     |                          
|    zduństwa                      1      287       317        340     |                          
|    - dla ludności                2      364       395        431     |
|                                  3      468       531        604     |
+----------------------------------------------------------------------+
|75  usługi w zakresie robót       0      262       299        324     |                          
|    budowlanych                   1      468       531        604     |                          
|    instalacyjnych                2      691       775        847     |                          
|    związanych z zakładaniem      3      819       912        994     |                          
|    instalacji: grzewczych,                                           |
|    wentylacyjnych,                                                   |
|    klimatyzacyjnych,                                                 |
|    wodnokanalizacyjnych,                                             |
|    gazowych, elektrycznych                                           |
|    i osprzętu elektrycznego                                          |
|     - dla ludności                                                   |
+----------------------------------------------------------------------+
|76  studniarstwo, z               0      250       250        250     |                          
|    wyjątkiem wiercenia           1      317       317        317     |                          
|    studni o głębokości           2      616       616        616     |                          
|    ponad 30 m                                                        | 
|     - dla ludności                                                   |
+----------------------------------------------------------------------+
|77  obsługa maszynowa             0      262       262        262     |                            
|    produkcji rolniczej i         1      431       431        431     |                          
|    ogrodniczej                   2      630       630        630     |                          
|                                  3      775       775        775     |
+----------------------------------------------------------------------+
|78  usługi zwalczania             0      137       137        137     |                          
|    szkodników roślin i           1      287       287        287     |                          
|    zwierząt                                                          |
+----------------------------------------------------------------------+
|79  usługi utrzymania i           0      225       225        225     |                          
|    pielęgnacji zwierząt          1      402       402        402     |                          
|                                                                      |
|    z czego                                                           |
|    usługi w zakresie                    176       176        176     |
|    zbiorowego wypasu owiec                                           |
|    w rejonach górskich w                                             |
|    sezonie letnim                                                    |
+----------------------------------------------------------------------+
|80  usługi w zakresie             0      163       176        200     |                          
|    prania, prasowania i          1      308       340        364     |                          
|    prężenia firanek              2      386       418        481     |
+----------------------------------------------------------------------+
|81  usługi w zakresie             0       65        77         89     |                          
|    maglowania                    1      163       176        200     |
|                                  2      212       250        275     |
+----------------------------------------------------------------------+
|82  usługi w zakresie             0      152       163        187     |                          
|    chemicznego czyszczenia       1      299       324        348     |                          
|    i farbowania, z               2      364       395        431     |                          
|    wyjątkiem czyszczenia         3      443       456        518     |                          
|    i farbowania wyrobów          4      468       518        581     |                          
|    futrzarskich                                                      |
+----------------------------------------------------------------------+
|83  pranie pierza i puchu         0      114       137        152     |
|                                  1      250       275        308     |
|                                  2      333       355        386     |
+----------------------------------------------------------------------+
|84  kominiarstwo                  0      102       114        128     |
|                                  1      236       262        299     |
|                                  2      317       348        371     |
+----------------------------------------------------------------------+
|85  usługi odkażania,             0      187       212        236     |                          
|    tępienia owadów               1      364       395        431     |                          
|    i odszczurzania               2      481       543        616     |                          
+----------------------------------------------------------------------+
|86  usługi fryzjerskie dla        0       77        89        102     |                          
|    kobiet                        1      200       225        250     |                          
|     i dziewcząt                  2      275       308        333     |
|                                  3      333       355        386     |
|                                  4      355       386        418     |
|                                  5      431       492        556     |
+----------------------------------------------------------------------+
|87  usługi fryzjerskie dla        0       77        77         89     |                          
|    mężczyzn                      1      163       187        212     |                          
|    i chłopców                    2      250       275        308     |
|                                  3      308       333        355     |
|                                  4      317       348        371     |
|                                  5      395       431        492     |
+----------------------------------------------------------------------+
|88  usługi kosmetyczne            0       77        89        102     |
|                                  1      200       225        250     |
|                                  2      287       317        340     |
|                                  3      340       371        402     |
|                                  4      364       395        431     |
|                                  5      395       431        492     |
+----------------------------------------------------------------------+
|89  manicure, pedicure            0       53        65         65     |
|                                  1      128       137        137     |
|                                  2      152       163        187     |
|                                  3      212       250        275     |
|                                  4      236       262        299     |
|                                  5      317       348        371     |
+----------------------------------------------------------------------+
|90  usługi fotograficzne:         0      340       371        402     |                       
|    świadczone z                  1      666       754        819     |                          
|    wykorzystaniem urządzeń       2      866       957       1048     |                          
|    do samoczynnego               3      976      1080       1202     |                          
|    automatycznego                                                    |
|    naświetlania odbitek                                              |
|    (samoczynnych printerów)                                          |
|                                                                      |
|    usługi pozostałe              0      137       163        176     |
|                                  1      299       324        348     |
|                                  2      364       395        431     |
|                                  3      411       468        531     |
|                                  4      468       543        602     |
+----------------------------------------------------------------------+
|91  wyświetlanie rysunków,        0      308       333        355     |                          
|    planów i tekstów              1      591       666        754     |                          
|    techniką kserografii                                              |
|    i diazotypii                                                      |
+----------------------------------------------------------------------+
|92  sprzątanie wnętrz             0      137       163        176     |
|                                  1      275       308        333     |
|                                  2      348       379        411     |
+----------------------------------------------------------------------+
|93  ważenie osób                  0       40        40         40     |
+----------------------------------------------------------------------+
|94  usługi w zakresie             0       77        77         77     |
|    radiestezji                                                       |
+----------------------------------------------------------------------+
|95  usługi świadczone w           0      137       163        176     |
|    gospodarstwach domowych                                           |
+----------------------------------------------------------------------+
                                       
                               Usługi parkingowe
                                       
   Objaśnienie: Stawki dotyczą działalności bez względu na stan zatrudnienia
+---+---------------------+-------------------------------------------------+                               
|Lp.|     Parkingi        |                 Stawka                          |
|   | o liczbie stanowisk |                                                 |
+---+---------------------+-------------------------------------------------+    
| 1 |          2          |                   3                             |
+---+---------------------+-------------------------------------------------+
|   | powyżej       do    |                                                 |
| 1 |              140    | 4 zł od każdego stanowiska                      |
| 2 |  140         180    | 560 zł + 6 zł od każdego stanowiska powyżej 140 |
| 3 1  180                | 800 zł + 7 zł od każdego stanowiska powyżej 180 |
+---+---------------------+-------------------------------------------------+
                                       
     Część II. Usługi w zakresie handlu detalicznego żywnością, napojami,
                      wyrobami tytoniowymi oraz kwiatami
Objaśnienia:
1. Stawki oznaczone w rubryce 3:
 - "0" dotyczą działalności prowadzonej bez zatrudnienia pracowników,
 - "1" dotyczą działalności prowadzonej przy zatrudnieniu 1 pracownika,
 - "2" dotyczą działalności prowadzonej przy zatrudnieniu 2 pracowników,
 - "3" dotyczą działalności prowadzonej przy zatrudnieniu 3 pracowników.
2. Za pracownika uważa się również pełnoletniego członka rodziny, z wyjątkiem
  małżonka.
3. W ramach detalicznej sprzedaży artykułów żywnościowych podatnicy mogą
  również dokonywać detalicznej sprzedaży wydawnictw prasowych (PKWiU 22.12,
  22.13), biletów komunikacji miejskiej (PKWiU 22.22.13-00.20 ex), opakowań z
  tworzyw sztucznych (toreb i torebek) (PKWiU 25.22.11 ex, 25.22.12 ex),
  zapałek (PKWiU 36.63.63-00), zapalniczek (PKWiU 36.63.61-10, 36.63.62-30) i
  papieru toaletowego (PKWiU 21.22.11-10).
4.  W ramach detalicznej sprzedaży kwiatów podatnicy mogą również dokonywać
 detalicznej sprzedaży ziemi i odżywek do kwiatów (bez względu na symbol
 PKWiU), doniczek (bez względu na symbol PKWiU), kwiatów i wieńców sztucznych
 (PKWiU 36.63.76-50, 36.63.76-90), nasion roślin kwiatowych (PKWiU 01.12.2300.10),
sadzonek roślin kwiatowych (PKWiU 01.12.21-00.33), zieleni ciętej
 spod szkła (PKWiU 01.12.22-00.2, 01.12.22-00.30), świec i zniczy (PKWiU
 36.63.75-00.1, 36.63.75-00.30), kart okolicznościowych (bez względu na
 symbol PKWiU) oraz maskotek (bez względu na symbol PKWiU).
5.  Wyróżnik "ex", o którym mowa w ust. 3, oznacza, że zakres dopuszczonych do
 sprzedaży wyrobów jest węższy niż określony w danym grupowaniu Polskiej
 Klasyfikacji Wyrobów i Usług.
+---+-------------------------+---------------------------------------+
|   |                         | W miejscowości o liczbie mieszkańców  |
|   |                         +-----------+-------------+-------------+
|Lp.| Usługi w zakresie handlu|  do 5.000 | ponad 5.000 |  powyżej    |
|   |      detalicznego       |           |  do 50.000  |   50.000    |
|   |                         +-----+-----+------+------+------+------+
|   |                         | od  | do  |  od  | do   | od   | do   |
+---+---------------------+---+-----+-----+------+------+------+------+
| 1 |          2          | 3 |  4  |  5  |  6   |  7   |  8   |  9   |
+---+---------------------+---+-----+-----+------+------+------+------+                                                                 
| 1 | żywnością, w tym    | 0 | 225 | 340 |  308 | 418  | 340  | 531  |
|   | owocami             | 1 | 308 | 418 |  340 | 531  | 418  | 679  |
|   | i warzywami,        | 2 | 340 | 536 |  418 | 679  | 531  | 775  |
|   | napojami            | 3 | 418 | 679 |  531 | 775  | 714  | 965  |
|   | i wyrobami          |   |     |     |      |      |      |      |
|   | tytoniowymi         |   |     |     |      |      |      |      |
+---+---------------------+---+-----+-----+------+------+------+------+                                                                
| 2 | kwiatami            | 0 | 308 | 418 |  340 | 531  | 418  | 679  |
|   |                     | 1 | 340 | 531 |  418 | 679  | 531  | 775  |
|   |                     | 2 | 418 | 679 | 531  | 775  | 714  | 965  |
+---+---------------------+---+-----+-----+------+------+------+------+
                                       
                                       
    Część III. Usługi w zakresie obnośnego i obwoźnego handlu detalicznego
                          artykułami nieżywnościowymi
Objaśnienia:
1. Stawki oznaczone w rubryce 3:
 - "0" dotyczą działalności prowadzonej bez zatrudnienia pracowników,
 - "1" dotyczą działalności prowadzonej przy zatrudnieniu 1 pracownika,
 - "2" dotyczą działalności prowadzonej przy zatrudnieniu 2 pracowników.
2. Za pracownika uważa się również pełnoletniego członka rodziny, z wyjątkiem
  małżonka.
+---+----------------------+-------------------------------------------+
|   |                      | W miejscowości o liczbie mieszkańców      |
|   |                      +--------------+--------------+-------------+
|   | Działalność handlowa |              |  ponad 5.000 |  powyżej    |
|Lp.| w zakresie obnośnej  | do 5.000     |  do 50.000   |  50.000     |
|   | i obwoźnej sprzedaży +------+-------+-------+------+------+------+
|   |    detalicznej       |  od  |   do  |  od   |  do  |  od  |  do  |
+---+------------------+---+------+-------+-------+------+------+------+                           
| 1 |         2        | 3 |  4   |   5   |   6   |   7  |   8  |   9  |
+---+------------------+---+------+-------+-------+------+------+------+                                                                 
| 1 | artykułów        | 0 |  308 |  679  |  340  |  775 |  418 | 1264 |
|   | nieżywnościowych | 1 |  340 |  775  |  418  | 1264 |  531 | 1870 |
|   |                  | 2 |  411 |  965  |  568  | 1697 |  714 | 2482 |
+---+------------------+---+------+-------+-------+------+------+------+                                                              
                                       
                             Część IV. Gastronomia
                                       
                                       
Objaśnienia:
1. Stawki oznaczone w rubryce 3:
 - "0" dotyczą działalności prowadzonej bez zatrudnienia pracowników,
 - "1" dotyczą działalności prowadzonej przy zatrudnieniu 1 pracownika,
 - "2" dotyczą działalności prowadzonej przy zatrudnieniu 2 pracowników,
 - "3" dotyczą działalności prowadzonej przy zatrudnieniu 3 pracowników,
  -"4" dotyczą działalności prowadzonej przy zatrudnieniu 4 pracowników.
2. Za pracownika uważa się również pełnoletniego członka rodziny,
  z wyjątkiem małżonka.
+---+--------------------------------------------+--------------+
|Lp.| Prowadzenie działalności gastronomicznej   |              |
|   | bez sprzedaży napojów o zawartości powyżej |     Stawka   |
|   |          1,5% alkoholu                     |              |
+---+----------------------------------------+---+--------------+
| 1 |                   2                    | 3 |       4      |
+---+----------------------------------------|---+--------------+
| 1 | w zakresie sprzedaży lodów z automatów | 0 |  299 - 395   |
|   |                                        | 1 |  395 - 518   |
|   |                                        | 2 |  468 - 591   |
|   |                                        | 3 |  543 - 666   |
|   |                                        | 4 |  730 - 857   |
+---+----------------------------------------+---+--------------+                                                      
| 2 | w zakresie pozostałej działalności     | 0 |  250 - 348   |
|   | gastronomicznej                        | 1 |  348 - 481   |
|   |                                        | 2 |  411 - 543   |
|   |                                        | 3 |  492 - 616   |
|   |                                        | 4 |  663 - 809   |
+---+----------------------------------------+---+--------------+                                   
                                       
                         Część V. Usługi transportowe
                                       
                                       
Objaśnienia:
1. Stawki określone pod lp. 1 i 2 mogą być podwyższone, nie więcej jednak niż
  o 50%, w razie wykonywania działalności ze zmiennikiem.
2. Za zmiennika uważa się również pełnoletniego członka rodziny, nie
  wyłączając małżonka.
3. Stawki określone pod lp. 2 dotyczą działalności z użyciem samochodu
  ciężarowego o ładowności do 2 t.
4.  Stawki określone pod lp. 4 dotyczą działalności prowadzonej przy
 zatrudnieniu nie przekraczającym 1 pracownika lub 1 pełnoletniego członka
 rodziny, z wyjątkiem małżonka.
+---+---------------------------+---------------------------------------+
|   |                           | W miejscowości o liczbie mieszkańców  |
|   |                           +---------+---------+---------+---------+          
|Lp |     Rodzaj usługi         | do      |  ponad  |  ponad  | powyżej |
|   |                           | 25.000  |  25.000 | 100.000 | 500.000 |
|   |                           |         |  do     |  do     |         |
|   |                           |         | 100.000 | 500.000 |         |
+---+---------------------------+---------+---------+---------+---------+
| 1 |            2              |    3    |   4     |   5     |   6     |
+---+---------------------------+---------+---------+---------+---------+
| 1 | Usługi taksówkowe w       |   129   |  136    |  146    |  157    |
|   | zakresie                  |         |         |         |         |
|   | przewozu osób             |         |         |         |         |
+---+---------------------------+---------+---------+---------+---------+
| 2 | Usługi taksówkowe w       |   194   |  212    |  229    |  254    |  
|   | zakresie                  |         |         |         |         |
|   | przewozu ładunków         |         |         |         |         |
+---+---------------------------+---------+---------+---------+---------+
| 3 | Przewozy pasażerskie      |    81   |   81    |   81    |  81     |
|   | z wykorzystaniem ludzkiej |         |         |         |         |
|   | i zwierzęcej siły         |         |         |         |         |
|   | pociągowej                |         |         |         |         |
+---+---------------------------+---------+---------+---------+---------+
| 4 | Usługi transportu         |    90   |   90    |   90    |   90    |
|   | wodnego śródlądowego      |         |         |         |         |
|   | pasażerskiego             |         |         |         |         |
|   | i towarowego, z           |         |         |         |         |
|   | wyjątkiem                 |         |         |         |         |
|   | przewozu jednostkami      |         |         |         |         |
|   | o napędzie mechanicznym   |         |         |         |         |
|   | z czego:                  |         |         |         |         |
|   | usługi w zakresie         | 40 - 65 | 40 - 65 | 40 - 65 | 40 - 65 |
|   | transportu osób na rzece  |         |         |         |         |
|   | Dunajec przez flisaków    |         |         |         |         |
|   | pienińskich               |         |         |         |         |
+---+---------------------------+---------+---------+---------+---------+                                      
  
                                     
                          Część VI. Usługi rozrywkowe
 
Objaśnienia:
1. Stawki dotyczą działalności przy zatrudnieniu nie przekraczającym 2
  pracowników lub 2 pełnoletnich członków rodziny, z wyjątkiem małżonka.
2. Znak "*" oznacza, że w razie prowadzenia więcej niż jednego urządzenia,
  stawkę karty podatkowej określa się dla każdego urządzenia oddzielnie, a
  następnie sumę tych stawek podwyższa się o 50%.
3. W razie świadczenia dwu lub więcej rodzajów usług, stawkę karty podatkowej
  określa się dla każdego rodzaju usług oddzielnie, a następnie sumę tych
  stawek podwyższa się o 30% niezależnie od podwyżki określonej w ust. 2.
+---+---------------------+----------------+-----------------------------+
|Lp.| Rodzaj świadczonych |    Liczba      |           Stawka            |
|   |     usług           |   urządzeń     |                             |
|   |                     |  (samochodów)  |                             |
+---+---------------------+----------------+-----------------------------+
| 1 |        2            |       3        |               4             |
+---+---------------------+----------------+-----------------------------+
| 1 | Automaty            |     do  2      | 26 zł od każdego urządzenia |
|   | zręcznościowe, na   | powyżej 2 do 4 | 52 zł + 61 zł od każdego    |
|   | których nie są      |                |       urządzenia ponad 2    |
|   | prowadzone gry o    | powyżej 4 do 6 | 174 zł +75 zł od każdego    |
|   | wygrane pieniężne   |                |       urządzenia ponad 4    |
|   | lub rzeczowe        | powyżej 6 do 8 | 324 zł + 80 zł od każdego   |
|   | (bilardy            |                |       urządzenia ponad 6    |
|   | elektromechaniczne, | powyżej 8 do 10| 484 zł + 92 zł od każdego   |
|   | gry telewizyjne, gry|                |       urządzenia ponad 8    |
|   | elektroniczne),     | powyżej 10     | 668 zł + 105 zł od każdego  |
|   | zabawki bujane      |                |       urządzenia ponad 10   |
|   | (ruchome figury)    |                |                             |
|   | oraz szafy grające  |                |                             |
+---+---------------------+----------------+-----------------------------+
| 2 | Karuzele o obrocie  |                |                             |
|   | prostym (poziome z  |                |                             |
|   | wiszącymi lub       |                |                             |
|   | stojącymi sylwetkami|                |                             |
|   | dla dzieci, poziome |                |                             |
|   | gondolowe lub       |                |                             |
|   | krzesełkowe, pionowe|                |                             |
|   | gondolowe) i o      |                |                             |
|   | obrocie złożonym    |                |                             |
|   | (elektrohydrau -    |                |                             |
|   | liczne gondolowe)   |                |                             |
|   | dla dzieci - o      |                |                             |
|   | liczbie miejsc:     |                |                             |
|   |         do 16       |     1*         | 5 zł od każdego miejsca     |
|   |     powyżej 16      |     1*         | 80 zł + 5 zł od każdego     |
|   |                     |                |     miejsca ponad 16        |
+---+---------------------+----------------+-----------------------------+
| 3 |  Karuzele o obrocie |                |                             |
|   | złożo-nym           |                |                             |
|   | (elektrohydrauliczne|                |                             |
|   | lub                 |                |                             |
|   | elektropneumatyczne |                |                             |
|   | gondolowe) - o      |                |                             |
|   | liczbie miejsc:     |                |                             |
|   |       do 24         |      1*        | 16 zł od każdego miejsca    |
|   |   powyżej 24        |      1*        | 384 zł + 13 zł od każdego   |
|   |                     |                |    miejsca ponad 24         |
+---+---------------------+----------------+-----------------------------+
| 4 | Imprezy samochodowe |       do  6    | 60 zł od każdego samochodu  |
|   | (pawilony, tory,    | powyżej 6 do 12| 360 zł + 94 zł od każdego   |
|   | miasteczka          |                |      samochodu ponad 6      |
|   | samochodowe) - o    |powyżej 12 do 24| 924 zł + 120 zł od każdego  |
|   | liczbie samochodów  |                |      samochodu ponad 12     |
|   |                     | powyżej 24     | 2364 zł + 150 zł od każdego |
|   |                     |                |      samochodu ponad 24     |
+---+---------------------+----------------+-----------------------------+
| 5 | Strzelnice          |         1      | 187 zł                      |
|   |                     |         2      | 380 zł                      |
|   |                     | powyżej 2      | 380 zł + 380 zł od każdej   |
|   |                     |                |       strzelnicy ponad 2    |
+---+---------------------+----------------+-----------------------------+
| 6 | Imprezy             |                |                             |
|   | zręcznościowe inne: |                |                             |
|   | a) rzuty różne,    |          1      | 49 zł                       |
|   |   tasiemkarnie,     |         2      | 123 zł                      |
|   | tarcze obrotowe     |         3      | 288 zł                      |
|   |                     | powyżej 3      | 288 zł + 198 zł od każdego  |
|   |                     |                |        urządzenia ponad 3   |
|   |                     |                |                             |
|   | b)  siłomierze,     |      do 4      | 13 zł od każdego urządzenia |
|   |   bajazzo,          | powyżej 4 do 6 | 52 zł + 15 zł od każdego    |
|   | bilardy ręczne      |                |        urządzenia ponad 4   |
|   |                     | powyżej 6      | 82 zł + 30 zł od każdego    |
|   |                     |                |        urządzenia ponad 6   |
+---+---------------------+----------------+-----------------------------+
| 7 | Huśtawki            |         1      | 51 zł                       |
|   | (wahadłowe i kołowe)|         2      | 262 zł                      |
|   |                     | powyżej 2      | 262 zł + 262 zł od każdego  |
|   |                     |                |        urządzenia ponad 2   |
+---+---------------------+----------------+-----------------------------+
| 8 | Kolejki torowe      |        1*      | 8 zł                        |
+---+---------------------+----------------+-----------------------------+
| 9 | Zjeżdżalnie:        |                |                             |
|   | a) zjeżdżalnie      |        1*      | 47 zł                       |
|   | "Gigant"            |                |                             |
|   | b) zjeżdżalnie      |        1*      | 31 zł                       |
|   | przewoźne           |                |                             |
+---+---------------------+----------------+-----------------------------+
|10 | Imprezy pozostałe:  |                |                             |
|   | a) pałac strachów,  |        1*      | 942 zł                      |
|   | labirynt            |                |                             |
|   | b) ściana emocji    |        1*      | 767 zł                      |
|   | c) ściana lalkowa   |        1*      | 262 zł                      |
|   | d) głowa medium,    |        1*      | 152 zł                      |
|   | pokazy gadów        |                |                             |
|   | e) gabinet złudzeń, |        1*      | 65 zł                       |
|   | pokazy              |                |                             |
|   | iluzjonistyczne     |                |                             |
|   | f) gabinet luster   |        1*      | 34 zł                       |
|   | g) hipodrom - kuce  |        1*      | 21 zł                       |
+---+---------------------+----------------+-----------------------------+
                                       
                                       
             Część VII. Sprzedaż posiłków domowych w mieszkaniach
                                       
                                       
+---+-------------------------------------+----------------------------+
|   |                                     | W miejscowości o liczbie   |
|   |                                     |        mieszkańców         |
|Lp.|   Rodzaj wykonywanej czynności      +--------------+-------------+
|   |                                     |   do 25.000  |   powyżej   |
|   |                                     |              |   25.000    |
+---+-------------------------------------+--------------+-------------+
| 1 |                 2                   |      3       |     4       |
+---+-------------------------------------+--------------+-------------+
|   | Sprzedaż śniadań, obiadów i kolacji |              |             |
|   | w liczbie posiłków każdego rodzaju, |              |             |
|   | licząc w skali średniej tygodniowej |              |             |
| 1 | do 10 na dobę                       |    227       |    241      |
| 2 | do 20 na dobę                       |    312       |    329      |
| 3 | do 30 na dobę                       |    358       |     -       |
|   |                                     |              |             |
|   | Sprzedaż jednodaniowych posiłków i  |              |             |
|   | napojów bezalkoholowych na          |              |             |
|   | podstawie umowy zawartej z          |              |             |
|   | jednostką właściwą do prowadzenia   |              |             |
|   | działalności gastronomicznej lub    |              |             |
|   | turystycznej w liczbie:             |              |             |   
| 4 | do 50 posiłków na dobę              |    273       |    298      |
| 5 | do 100 posiłków na dobę             |    358       |    394      |
+---+-------------------------------------+--------------+-------------+
 
                                       
                                       
    Część VIII. Wolne zawody - świadczenie usług w zakresie ochrony zdrowia
                                   ludzkiego
                                       
Objaśnienia:
1. Stawka dotyczy również wykonywania zawodu przy zatrudnieniu jednej pomocy
  fachowej lub przyuczonej.
2. Stawka nie dotyczy działalności prowadzonej w formie spółki cywilnej osób
  fizycznych.
+---+-----------------------------+--------------------------+
|Lp.|    Rodzaj wykonywanego      |          Stawka          |
|   |         zawodu              |                          |
+---+-----------------------------+--------------------------+
| 1 |             2               |             3            |
+---+-----------------------------+--------------------------+
|   |                             |                          |
| 1 |  Pielęgniarka, położna      |  1  zł  za każdą godzinę |
|   |                             |                          |
+---+-----------------------------+--------------------------+
                                               
                                               
Część IX. Wolne zawody - świadczenie usług weterynaryjnych
Objaśnienia:
1. Stawka dotyczy również wykonywania zawodu przy zatrudnieniu jednej pomocy
  fachowej lub przyuczonej.
2. Za pomoc fachową lub przyuczoną nie uważa się lekarza weterynarii.
3. Stawki nie dotyczą działalności prowadzonej w formie spółki cywilnej osób
  fizycznych.
4. Stawka określona pod lit. b) dotyczy wyłącznie podatników, którzy wykonują
  usługi weterynaryjne i są równocześnie zatrudnieni na podstawie umowy o
  pracę w pełnym wymiarze czasu pracy w służbie weterynaryjnej organów
  administracji publicznej.
+--------------+------------------+--------------------------------+
|    Rodzaj    |  Liczba godzin   |         Stawka                 |
| wykonywanego |  przeznaczonych  |                                |
|    zawodu    |  na wykonywanie  |                                |
|              |zawodu miesięcznie|                                |
+--------------+------------------+--------------------------------+
|      1       |        2         |              3                 |
+--------------+------------------+--------------------------------+
| Lekarz       |                  |                                |
| weterynarii: |                  |                                |
| a)           |       do 48      | 5 zł za każdą godzinę          |
|              | powyżej 48 do 96 | 240 zł + 6 zł za każdą godzinę |
|              |                  |            ponad 48            |
|              | powyżej 96       | 528 zł                         |
|              |                  |                                |
| b)           |        -         | 260 zł                         |
+--------------+------------------+--------------------------------+
                                       
                                       
             Część X. Opieka domowa nad dziećmi i osobami chorymi
                                       
Objaśnienie:
Stawki   dotyczą   działalności  prowadzonej  bez  zatrudnienia   pracowników,
z wyjątkiem małżonka.
+----------------------+-----------------------------------------+
|    Liczba godzin     |                 Stawka                  |
|   przeznaczonych     |                                         |
| na wykonywanie usług |                                         |
|     miesięcznie      |                                         |
+----------------------+-----------------------------------------+         
|          1           |                   2                     |
+----------------------+-----------------------------------------+
|      do 48           |   1 zł za każdą godzinę                 |
| powyżej 48 do 96     |  48 zł + 2 zł za każdą godzinę ponad 48 |
| powyżej 96           | 144 zł                                  |
+----------------------+-----------------------------------------+
                                       
                                       
      Część XI. Usługi edukacyjne w zakresie udzielania lekcji na godziny
                                       
Objaśnienia:
1. Stawki dotyczą działalności prowadzonej bez zatrudnienia pracowników, nie
  wyłączając małżonka.
2. Za udzielanie lekcji na godziny uważa się pomoc w nauce, polegającą głównie
  na odrabianiu (przygotowaniu) z uczniem zadanych lekcji.
+----------------------+-----------------------------------------+
|    Liczba godzin     |                 Stawka                  |
|   przeznaczonych     |                                         |
| na udzielanie lekcji |                                         |
|     miesięcznie      |                                         |
+----------------------+-----------------------------------------+
|          1           |                    2                    |
+----------------------+-----------------------------------------+
|      do 48           |   3 zł za każdą godzinę                 |
| powyżej 48 do 96     | 144 zł + 6 zł za każdą godzinę ponad 48 |
| powyżej 96           | 432 zł                                  |
+----------------------+-----------------------------------------+
                                                               
                                                 Załącznik nr 4
            CHARAKTERYSTYKA DZIAŁALNOŚCI USŁUGOWEJ
         I WYTWÓRCZO-USŁUGOWEJ OBJĘTEJ OPODATKOWANIEM
                   W FORMIE KARTY PODATKOWEJ
Objaśnienie:
Zakres działalności oznaczonej w rubryce 2 tabeli symbolem "*"
obejmuje działalność usługową i wytwórczo - usługową.
                               
Lp.      Zakres                    Przedmiot działalności
      działalności
 1         2                                 3
 1 usługi ślusarskie   Wyrób, naprawa i konserwacja wyrobów z metali
                       oraz przy użyciu elementów z innych surowców,
                       a w szczególności: narzędzi, prostych
                       narzędzi pomiarowych, mebli, sprzętu
                       gospodarstwa domowego i rolno-ogrodniczego;
                       sprzętu i urządzeń sportowo-rekreacyjnych;
                       konstrukcji, okuć, części armatur i innych
                       elementów na potrzeby budownictwa; kas
                       pancernych i  kaset; wózków dziecięcych i
                       gospodarczych, rowerów, części zamiennych i
                       akcesoriów do pojazdów jednośladowych i
                       maszyn, ogrodzeń metalowych, wraz z ich
                       montażem, galanterii, artykułów powszechnego
                       użytku z grupy 1001 drobiazgów oraz naprawa
                       zabawek itp.
                       Naprawa części mechanicznych maszyn i
                       urządzeń. Naprawa i konserwacja sprzętu
                       przeciwpożarowego. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
2  usługi w zakresie   Wyrób i naprawa naczyń blaszanych. Wyrób
   wyrobu i naprawy    przedmiotów z cyny (np. dzbanów, kufli,
   naczyń blaszanych   kielichów, mis, świeczników). Wyrób rur i
   blaszanych          kolanek do pieców. Renowacja przedmiotów
                       zabytkowych z blachy. Obudowa blachą
                       wszelkiego rodzaju sprzętu. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
                               
3 usługi rusznikarskie Wyrób, naprawa i konserwacja broni palnej
                       (myśliwskiej, sportowej krótkiej i gazowej).
                       Dorabianie części zamiennych do broni i
                       montowanie na niej lunet. Renowacja broni,
                       np. oksydowanie. Ocena stanu technicznego
                       broni i jej przestrzeliwanie. Zdobienie
                       broni. Wyrób amunicji do broni myśliwskiej i
                       sportowej. Wyrób kopii dawnej broni palnej.
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia wyrobu lub świadczonej usługi.
4  usługi w zakresie   Wyrób, naprawa, przeróbka i konserwacja
   złotnictwa-         biżuterii i innych przedmiotów użytkowych i
   jubilerstwa         ozdobnych (np. wyrobów korpusowych) z metali
   - dla ludności      szlachetnych i innych metali. Obróbka i
                       oprawa kamieni naturalnych i syntetycznych.
                       Zdobienie wyrobów różnymi technikami.
                       Rytowanie w kamieniach. Usługowa oprawa
                       bursztynów. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
5  usługi grawerskie   Zdobienie ręczne i maszynowe przedmiotów
                       użytkowych i dekoracyjnych z metali
                       szlachetnych, kolorowych i żelaznych
                       technikami grawerskimi. Grawerowanie wklęsłe,
                       płaskie i wypukłe w metalu, kamieniach
                       szlachetnych i ozdobnych oraz innych
                       materiałach. Wyrób stempli i matryc. Wyrób
                       medali, orderów, plakietek, znaczków,
                       monogramów, wizytówek, szyldzików,
                       numeratorów, cechowników, wzorników itp. oraz
                       różnych ozdób technikami grawerskimi.
                       Rytowanie walców. Cyzelowanie. Emaliowanie,
                       inkrustowanie i intarsjowanie. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
6  usługi w zakresie   Wykonywanie wszelkiego rodzaju pieczątek.
   Wyrobu pieczątek    Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia świadczonej usługi.
7  usługi pobielania   Nakładanie powłoki cynowej na powierzchnię
   kotłów i naczyń     powierzonych kotłów, naczyń i urządzeń, wraz
                       z pracami przygotowawczymi i wykończeniowymi,
                       np. trawienie, czyszczenie, polerowanie.
                       Naprawa pobielanych naczyń i przedmiotów.
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania
                       świadczonej usługi.
                               
8  usługi kowalskie    Naprawa i konserwacja narzędzi, sprzętu i
                       maszyn rolniczych, przedmiotów gospodarstwa
                       domowego z metali żelaznych, podwozi,
                       przyczep do pojazdów mechanicznych, pojazdów
                       konnych i wózków ręcznych. Wyrób części do
                       maszyn rolniczych i pojazdów techniką
                       plastycznej obróbki metali. Wyrób kutych okuć
                       budowlanych. Wyrób i naprawa kutych parkanów,
                       bram, balustrad i krat ozdobnych oraz innych
                       zdobniczych przedmiotów kutych, wykonywanie
                       odkuwek do dalszej obróbki. Podkuwanie koni.
                       Wyrób, naprawa i konserwacja resorów. Inne
                       prace i czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
9  usługi w zakresie   Montaż, demontaż i naprawa podzespołów,
   elektromechaniki    zespołów oraz członów funkcjonalnych urządzeń
   chłodniczej         i aparatury chłodniczej oraz zamrażającej.
                       Wykonywanie, naprawa i konserwacja instalacji
                       elektrycznej do urządzeń chłodniczych i
                       zamrażających. Wykonywanie części zamiennych
                       do urządzeń chłodniczych i zamrażających,
                       wraz z obróbką metali. Napełnianie agregatów
                       środkami chłodniczymi. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia świadczonej usługi.
10 usługi w zakresie   Wyrób, naprawa, konserwacja oraz instalowanie
   kotlarstwa          wszelkiego rodzaju: kotłów centralnego
                       ogrzewania (parowych i wodnych, nisko-,
                       średnio- i wysokoprężnych), kotłów warzelnych
                       i o innym przeznaczeniu, aparatów grzewczowentylacyjnych,
wymienników ciepła różnych
                       typów i innych zbiorników ciśnieniowych i
                       bezciśnieniowych. Montowanie na kotłach i
                       zbiornikach osprzętu i aparatury kontrolnopomiarowej;
czyszczenie kotłów i zbiorników.
                       Wyrób elementów do kotłów, zbiorników,
                       wymienników ciepła itp. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
11 usługi w zakresie   Wyrób, naprawa, konserwacja, montaż i
   mechaniki           demontaż maszyn i urządzeń mechanicznych
   maszyn              przemysłowych, hydraulicznych i gospodarstwa
                       domowego oraz pomp. Wyrób części zamiennych
                       do maszyn. Obróbka ręczna i maszynowa części
                       maszyn. Regulacja nowych, naprawianych i
                       konserwowanych maszyn, urządzeń oraz kontrola
                       ich sprawnego działania. Projektowanie
                       wytwarzanych maszyn i urządzeń mechanicznych
                       i hydraulicznych. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
                               
12 usługi w zakresie   Wyrób, naprawa i konserwacja maszyn
   mechaniki maszyn    rolniczych i urządzeń zmechanizowanego
   i urządzeń          sprzętu rolniczego. Wyrób części zamiennych.
   rolniczych          Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania
                       i wykończenia wyrobu lub świadczonej usługi.
13 usługi w zakresie   Wyrób, instalowanie, naprawa i konserwacja
   elektromechaniki    aparatury i urządzeń elektromedycznych.
   sprzętu medycznego  Wykonywanie, naprawa i konserwacja instalacji
   i laboratoryjnego   elektromedycznych i uziemień dla urządzeń
                       elektromedycznych. Wyrób aparatury
                       elektromedycznej, z zastosowaniem podzespołów
                       elektronicznych. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
14 usługi w zakresie   Wyrób, naprawa, regeneracja i konserwacja
   mechaniki           narzędzi, aparatów precyzyjnych, medycznych,
   precyzyjnej         geodezyjnych, optycznych, nawigacyjnych, fotoi
kinotechnicznych, w tym również liczników
                       samochodowych i taksometrów oraz innych
                       aparatów specjalistycznych. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
15 usługi w zakresie   Naprawa i konserwacja wszelkich maszyn
   mechaniki maszyn    biurowych, księgujących i liczących oraz
   biurowych           urządzeń poligraficznych. Dorabianie części
                       do naprawianych maszyn i urządzeń. Inne prace
                       i czynności towarzyszące, niezbędne do
                       całkowitego wykonania świadczonej usługi.
16 usługi w zakresie   Wyrób, naprawa i konserwacja wag. Dorabianie
   wyrobu i naprawy    części do naprawianych wag. Wyrób odważników.
   wag                 Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia wyrobu lub świadczonej usługi.
17 usługi              Wyrób, naprawa, konserwacja i regulacja
   zegarmistrzowskie   zegarów, w tym elektronicznych i kurantów.
                       Dorabianie części zamiennych. Konstruowanie,
                       wyrób, naprawa i regulacja drobnych
                       mechanizmów precyzyjnych. Zakładanie i
                       naprawa pasków i bransolet z metali
                       nieszlachetnych do zegarków, wymiana baterii.
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia wyrobu lub świadczonej usługi.
18 usługi w zakresie   Antykorozyjne zabezpieczanie środków
   antykorozyjnego     transportu.
   zabezpieczania
   pojazdów
19 usługi w zakresie   Mycie i czyszczenie środków transportu.
   mycia oraz smaro-   Smarowanie i wymiana oleju w środkach
   wania samochodów    transportu, konserwacja podwozi oraz inne
   i motocykli oraz    usługi nie związane z naprawą środków
   inne usługi         transportu.
   nie związane z ich
   naprawą
20 usługi w zakresie   Wyrób i naprawa silników i prądnic
   elektromechaniki,   elektrycznych. Wyrób i naprawa
   z wyjątkiem         zmechanizowanego sprzętu gospodarstwa
   dźwigowej,          domowego, spawarek i zgrzewarek
   chłodniczej i       elektrycznych, prostowników, stabilizatorów i
   pojazdowej          transformatorów, aparatury łączeniowej
                       (łączników), urządzeń elektromechanicznych.
                       Wyrób i naprawa urządzeń grzejnych i
                       oświetleniowych. Wyrób i montaż zapalarek
                       piezoelektrycznych. Wykonywanie instalacji
                       elektrycznych do maszyn i urządzeń
                       elektrycznych. Naprawa i konserwacja
                       instalacji elektrycznych. Wyrób podzespołów i
                       urządzeń elektromechanicznych i
                       elektronicznych oraz rozdzielni
                       elektroenergetycznych do 1 kV. Pomiary
                       elektryczne i badania odbiorcze odbiorników
                       elektrycznych. Wyrób, regeneracja, naprawa i
                       ładowanie akumulatorów. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
21 naprawa i konser-   Naprawa zespołów i kompletnych urządzeń
   wacja urządzeń      w elektronicznym sprzęcie
   elektronicznych     powszechnego użytku, w energetyce,
                       elektronice, elektrotechnice i automatyce
                       przemysłowej, jak np. zasilacze, urządzenia
                       sygnalizacyjne, alarmowe, sterownicze,
                       wzmacniacze elektroakustyczne, magnetowidy,
                       kamery, aparatura diagnostyczna, naukowa,
                       elektroniczny sprzęt pomiarowo-kontrolny,
                       urządzenia cyfrowe służące do zbierania i
                       przetwarzania informacji itp. Naprawa i
                       konserwacja komputerów i urządzeń
                       pomocniczych. Naprawa zegarków i zegarów
                       elektronicznych (z wyłączeniem analogowych).
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia świadczonej usługi.
                               
22 usługi w zakresie   Naprawa i konserwacja oraz instalowanie
   tele- i             urządzeń i aparatów radiowych, telewizyjnych,
   radiomechaniki      teletechnicznych, elektroakustycznych, w tym
                       również domofonów, sygnalizacyjnych oraz
                       elektronicznych i elektromechanicznych
                       urządzeń współpracujących, w tym również
                       anten telewizyjnych, z wyjątkiem instalacji i
                       montażu anten satelitarnych i sieci telewizji
                       kablowej. Konserwacja i naprawa telefonicznej
                       sieci wewnątrzzakładowej, central
                       telefonicznych, aparatów telefonicznych i
                       urządzeń dyspozytorskich wewnątrzzakładowych.
                       Wykonywanie nietypowych urządzeń
                       stabilizacyjnych, antenowych oraz części
                       zamiennych, elementów i podzespołów. Inne
                       prace i czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
23 usługi w zakresie   Bieżnikowanie i kolcowanie opon.
   bieżnikowania opon
   
24 usługi              Naprawa wszelkiego rodzaju opon i dętek,
   wulkanizatorskie,   montaż i demontaż opon oraz wyważanie kół,
   z wyjątkiem         naprawa wszelkich wyrobów z gumy i
   bieżnikowania opon  gumopodobnych, w tym zabawek i artykułów
                       sanitarnych, naprawa obuwia gumowego oraz
                       artykułów gumowo-metalowych; powlekanie
                       tkanin gumą, wykonywanie klejów do gumy.
25 szklarstwo "*"      Szklenie ram okiennych, drzwiowych,
                       świetlików, wystaw, inspektów i szklarni.
                       Szklenie akwariów i gablot oraz mebli i
                       pojazdów. Wykładanie ścian i kolumn szkłem.
                       Łączenie szkła przez klejenie. Oprawa
                       obrazów, dyplomów, fotografii itp.
                       Sporządzanie kitów szklarskich. Wyrób,
                       renowacja i osadzanie luster. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
26 witrażownictwo "*"  Wyrób, rekonstrukcja, naprawa i konserwacja
                       witraży, wykonywanie kloszy do lamp lub
                       innych kompozycji ze szkła kolorowego
                       wiązanego metalami. Osadzanie witraży.
                       Barwienie i wypalanie, matowanie,
                       formatowanie i trawienie szkła,
                       przygotowywanie złączy. Wykonywanie
                       towarzyszących robót szklarskich. Inne prace
                       i czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
                               
27 usługi stolarskie,  Wyrób, naprawa, renowacja i konserwacja mebli
   z wyjątkiem wyrobu  z drewna i z zastosowaniem innych materiałów
   trumien             oraz wykonywanie części drewnianych do mebli
                       o mieszanym wykonawstwie. Obudowa wnętrz
                       drewnem i materiałami drewnopochodnymi. Wyrób
                       i naprawa stolarki budowlanej z drewna lub
                       materiałów drewnopochodnych. Wykonywanie
                       drewnianych schodów i balustrad. Wyrób i
                       naprawa przedmiotów galanteryjnych, artykułów
                       gospodarstwa domowego i sprzętu sportowego z
                       drewna. Usługowe wykonywanie elementów
                       podłogowych. Wyrób i naprawa futerałów z
                       drewna itp. Wykonywanie i naprawa budek,
                       altanek i kiosków. Usługowa stolarska obróbka
                       drewna. Wyrób ram. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
28 kołodziejstwo "*"   Wyrób i naprawa drewnianych pojazdów
                       zaprzęgowych, np. wozów gospodarczych,
                       powozów, karawanów, karet, sań i sanek.
                       Wykonywanie obudowy drewnianej maszyn
                       rolniczych i przyczep. Wyrób drewnianych
                       narzędzi rolniczych oraz drewnianego sprzętu
                       gospodarstwa wiejskiego, np. żłobków, koryt,
                       skrzyń, drabin itp. Naprawa wyrobów
                       bednarskich i z zakresu tokarstwa w drewnie.
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania i
                       wykończenia wyrobu lub świadczonej usługi.
29 usługi w zakresie   Pozłacanie folią i farbą przedmiotów
   pozłotnictwa        zabytkowych i użytkowych, np. ram, rzeźb,
                       listew, elementów sztukatorskich, tablic
                       nagrobkowych, elementów obiektów budowlanych
                       itp. Wyrób i naprawa pozłacanych ram
                       stylowych i listew ozdobnych. Renowacja i
                       rekonstrukcja wyżej wymienionych przedmiotów.
                       Oprawa obrazów, dyplomów, fotografii itp.
30 usługi tapicerskie  Wyrób i renowacja kompletów, zestawów i
                       pojedynczych mebli tapicerskich konstrukcji
                       skrzyniowej i szkieletowej o różnym
                       przeznaczeniu. Wyrób i naprawa tapicerskiego
                       sprzętu sportowego. Wykładanie wnętrz
                       mieszkalnych i w środkach transportu
                       materiałami przeciwakustycznymi i
                       ciepłochłonnymi oraz wykonywanie dekoracji
                       okolicznościowych. Szycie i zakładanie
                       zasłon. Szycie pokrowców. Zszywanie i
                       układanie dywanów i chodników. Wykonywanie i
                       naprawa tapicerki w środkach transportu. Inne
                       prace i czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
31 bednarstwo "*"      Wyrób, naprawa i impregnowanie drewnianych
                       naczyń klepkowych (np. wiader, cebrzyków,
                       beczek itp.). Wyrób klepki bednarskiej.
                               
32 koszykarstwo "*"    Wyrób przedmiotów koszykarskich i
                       plecionkarskich (np. koszy, mebli,
                       galanterii) z wikliny, trzciny, rogożyny i
                       bambusa oraz z zastosowaniem innych
                       materiałów. Barwienie wykonywanych wyrobów.
                       Naprawa i konserwacja mebli wiklinowych,
                       trzcinowych itp.
33 wytwarzanie mioteł, Wyrób mioteł i kropideł. Wyrób mat,
   wyrobów z gałęzi    słomianek, wycieraczek, koszyków oraz
   i słomy "*"         galanterii użytkowej i ozdobnej z gałęzi i
                       słomy.
34 gręplarstwo "*"     Usługowe zgrzeblenie (gręplowanie) różnych
                       surowców włókienniczych, w szczególności
                       wełny. Gręplowanie różnych materiałów i
                       odpadów produkcyjnych pochodzenia
                       włókienniczego.
35 kilimiarstwo "*"    Wyrób i naprawa na warsztatach ręcznych:
                       gobelinów i kilimów, makat, chodników,
                       dywanów itp. oraz tkanin artystycznych przy
                       użyciu różnych materiałów, np. wełny, lnu,
                       bawełny, skóry, szkła, drewna, ceramiki,
                       włosia końskiego, włókien syntetycznych, piór
                       itp. Barwienie i przędzenie surowca do
                       wytwarzania wyrobów.
36 koronkarstwo,       Wyrób koronek oraz przedmiotów użytkowych i
   z wyjątkiem         ozdobnych metodą koronkarską.
   maszynowego "*"
37 usługi hafciarskie  Ręczne i maszynowe haftowanie szat
                       liturgicznych, bielizny stołowej,
                       pościelowej, osobistej, odzieży itp. przy
                       zastosowaniu różnych stylów i metod
                       wykonawczych. Szycie i haftowanie sztandarów.
                       Naprawa i renowacja haftowanych przedmiotów
                       zabytkowych. Wyrób firan, galonów, taśm,
                       szychów, ozdób i dystynkcji do mundurów,
                       tarcz szkolnych itp. Szycie strojów ludowych
                       i regionalnych oraz elementów tych strojów,
                       zdobionych haftem i koronkami. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
38 dziewiarstwo na     Wyrób na drutach i szydełkach przedmiotów z
   drutach i           przędzy, włóczki, nici, sznurka itp.
   szydełkach "*"
39 repasacja pończoch  Naprawa pończoch, rajstop, skarpet oraz
                       innych wyrobów dziewiarskich metodą
                       podnoszenia oczek.
                               
40 usługi w zakresie   Wyrób, przerabianie i naprawa bielizny
   bieliźniarstwa      stołowej, pościelowej, niemowlęcej,
                       dziecięcej, damskiej i męskiej (dziennej i
                       nocnej), liturgicznej, podomek, szlafroków,
                       płaszczy kąpielowych, bonżurek męskich,
                       czepków pielęgniarskich, kucharskich itp.
                       Wyrób kołder, śpiworów, poduszek, pierzyn,
                       sienników. Wyrób z materiałów włókienniczych
                       odzieży, ubiorów i rękawiczek oraz nakryć
                       głowy dla dzieci do lat 6. Wyrób firanek i
                       zasłon okiennych, pledów, narzut, fartuchów,
                       bluzek, krawatów, chustek, apaszek, żabotów i
                       muszek, śliniaków, kostiumów plażowych itp.
                       Usługowe okrętkowanie. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
41 usługi w zakresie   Wyrób i naprawa gorsetów (z wyłączeniem
   gorseciarstwa       gorsetów ortopedycznych), biustonoszy, pasków
                       do pończoch, pasów ciążowych, kostiumów
                       plażowych, kąpielowych itp.
42 czapnictwo          Wyrób, naprawa, przerabianie i odświeżanie
   i kapelusznictwo    męskich, młodzieżowych, chłopięcych i
   męskie "*"          dziecięcych nakryć głowy, np. czapek
                       cywilnych, mundurowych, szkolnych, kapeluszy,
                       pilotek, beretów, biretów itp. z wszelkiego
                       rodzaju materiałów, w tym również ze skór,
                       futer i ich imitacji.
43 modniarstwo "*"     Wyrób, naprawa, przerabianie i odświeżanie
                       kapeluszy damskich, beretów, toczków, czapek
                       i czapeczek dziecięcych itp. nakryć głowy z
                       wszelkiego rodzaju materiałów, w tym również
                       ze skóry, futer i ich imitacji. Wyrób
                       sztucznych kwiatów oraz innych ozdób,
                       stroików do kapeluszy, welonów, wianków itp.
                       z tkanin i skóry. Szycie szalików jako
                       kompletu z nakryciem głowy. Szycie i upinanie
                       welonów ślubnych, żałobnych oraz kokard.
44 usługi krawieckie   Szycie miarowe ubiorów i okryć damskich,
                       męskich i dziecięcych, np.: kostiumy, kurtki,
                       wdzianka, płaszcze letnie, płaszcze zimowe,
                       spodnie, marynarki, prochowce z podpinką,
                       ubiory sportowe, fraki, smokingi, sutanny,
                       mundury itp., z wszelkich materiałów, w tym
                       również ze skór licowych i materiałów
                       skóropodobnych oraz pelis z użyciem skór
                       futerkowych naturalnych i sztucznych.
                       Pokrywanie błamów. Naprawa i renowacja ubrań
                       i okryć.
                       Wyrób, naprawa i renowacja odzieży lekkiej
                       damskiej i dziecięcej, jak: bluzki, sukienki,
                       suknie, spódnice, spodnie, spodniumy,
                       prochowce, wiatrówki, fartuchy, kombinezony,
                       ubiory sportowe, ubiory plażowe, odzież
                       młodzieżowa lekka itp. Szycie ocieplanej
                       odzieży dla dzieci do lat 6. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
                               
45 cerowanie i naprawa Cerowanie odzieży, w tym cerowanie
   odzieży, obciąganie artystyczne. Naprawa odzieży. Obciąganie
   guzików, okrętkowa- guzików, klamer i pasków. Ręczne i maszynowe
   nie, mereżkowanie,  okrętkowanie, mereżkowanie, plisowanie
   plisowanie          spódnic, bluzek i innej garderoby oraz zasłon
                       i firanek. Wszywanie zamków błyskawicznych,
                       obszywanie maszynowe i ręczne dziurek,
                       nabijanie zatrzasków i nap.
46 usługi w zakresie   Szycie, naprawa, odświeżanie i zdobienie
   kożuszkarstwa       kożuchów oraz ich imitacji. Szycie, naprawa i
                       odświeżanie galanterii z kożucha oraz
                       imitacji. Wykonywanie błamów i wyrobów ze
                       skór baranich z włosem.
47 cholewkarstwo "*"   Modelowanie i wykonywanie wierzchnich części
                       obuwia ze skór, surowców skóropodobnych,
                       materiałów włókienniczych i innych. Naprawa i
                       przeróbka wierzchnich części obuwia. Naprawa
                       wyrobów kaletniczych.
48 szewstwo miarowe    Usługowe wykonywanie, naprawa i odświeżanie
                       obuwia ze skóry, materiałów zastępczych,
                       skóropodobnych, tekstylnych, drewna, tworzyw
                       sztucznych itp.
49 szewstwo            Wykonywanie, naprawa oraz renowacja i
   ortopedyczne "*"    odświeżanie obuwia ortopedycznego
                       (korekcyjnego, protezowego, aparatowego itp.)
                       ze skór i innych materiałów. Formowanie kopyt
                       i cholewek do obuwia ortopedycznego, wyrób i
                       regeneracja wkładek ortopedycznych. Naprawy z
                       zakresu szewstwa.
50 szewstwo naprawkowe Naprawa i odświeżanie obuwia.
51 odświeżanie i       Odświeżanie i renowacja wyrobów skórzanych
   renowacja wyrobów   lub skóropodobnych. Naprawa odzieży skórzanej
   skórzanych i        i skóropodobnej, drobne przeróbki (np.
   futrzarskich, z     skracanie, zwężanie, przedłużanie).
   wyjątkiem mechani-  Barwienie. Odświeżanie wyrobów futrzarskich
   cznego czyszczenia  lub ich części. Renowacja wyrobów
   i farbowania wyro-  futrzarskich i naprawa odzieży futrzarskiej,
   bów futrzarskich    drobne przeróbki, np. skracanie,
                       przedłużanie. Czyszczenie włosa, czyszczenie
                       skóry (mizdry), barwienie, zszywanie i
                       wprawianie brakujących części wyrobów.
52 usługi w zakresie   Czyszczenie i farbowanie wyrobów futrzarskich
   mechanicznego czy-  w zmechanizowanych agregatach pralniczych
   szczenia i farbo-   przy użyciu środków czyszczących lub
   wania wyrobów       farbujących.
   futrzarskich
53 usługi w zakresie   Szycie, naprawa i renowacja futer, kołnierzy,
   kuśnierstwa         czapek, błamów oraz innej odzieży i
                       galanterii futrzanej ze skór szlachetnych,
                       półszlachetnych i zwykłych oraz z ich
                       imitacji. Szycie pokrowców, narzut, makat
                       dekoracyjnych, błamów, chodników itp. z futer
                       oraz z wszelkiego rodzaju ścinków i odpadów
                       futrzarskich.
54 rymarstwo,          Wyrób i naprawa przedmiotów użytkowych i
   z wyjątkiem         ozdobnych (np. uprzęży, siodeł, troków, pasów
   produkcji           pędnych i transmisyjnych, sprzętu sportowego
   plandek "*"         i myśliwskiego, kagańców, smyczy, biczów,
                       waliz itp.) ze skór rymarskich i materiałów
                       zastępczych. Drobne naprawy wyrobów
                       kaletniczych i pokryć tapicerskich.
55 rękawicznictwo -    Wyrób, naprawa i odświeżanie rękawiczek
   wyrób rękawiczek    skórzanych i skóropodobnych damskich, męskich
   skórzanych,         i dziecięcych oraz wkładów ocieplających do
   i skóropodobnych,   rękawiczek ze skóry i skóropodobnych.
   z wyjątkiem
   roboczych,
   ochronnych
   i sportowych "*"
56 młynarstwo "*"      Przemiał i przerób zbóż i innych nasion na
                       określone rodzaje i gatunki mąki, kaszy,
                       śruty i innych przetworów zbożowych.
                       Przygotowywanie ziarna zbóż i innych nasion
                       do przemiału i przerobu.
57 przemiał i przerób  Przemiał i przerób zbóż w wiatrakach,
   zbóż w wiatrakach   śrutownikach i mlewnikach na określone
   oraz śrutowanie     rodzaje i gatunki mąki, kaszy i śruty.
   zboża "*"           Przygotowywanie ziarna zbóż do przemiału i
                       przerobu.
58 wytwarzanie waty    Wytwarzanie waty z cukru. Prażenie kukurydzy
   z cukru "*"         (pop corn), z wyjątkiem prażenia kukurydzy
   prażenie kukurydzy  przeznaczonej do dalszej odprzedaży lub
   "*"                 wykorzystania jako surowiec do celów
                       wytwórczych lub usługowych.
59 usługi w zakresie   Oprawa książek, dokumentów, bloków, notesów
   introligatorstwa    itp., tłoczenie, złocenie i barwienie napisów
                       i ozdób introligatorskich. Konserwowanie
                       druków i naprawa zniszczonych kart, ksiąg i
                       opraw. Laminowanie. Naklejanie map, plansz,
                       planów, reprodukcji, fotografii itp. Wyrób
                       albumów, bloków, ksiąg handlowych, klaserów
                       itp. Wyrób ozdobnych pudeł i pudełek, etui do
                       biżuterii, futerałów, kaset do nakryć
                       stołowych itp. Wyrób teczek na akta, obwolut,
                       opakowań z kartonu i papieru oraz zdobionej
                       galanterii papierniczej. Usługowe krojenie
                       papieru, tektury i wszelkich materiałów
                       introligatorskich. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia wyrobu lub
                       świadczonej usługi.
                               
60 usługi w zakresie   Wyrób, naprawa, korekta, konserwacja i
   lutnictwa           strojenie instrumentów smyczkowych i
                       szarpanych, w tym z użyciem elementów
                       elektronicznych. Wyrób części i akcesoriów do
                       instrumentów smyczkowych i szarpanych.
                       Naprawa futerałów i pulpitów.
61 usługi w zakresie   Wyrób, naprawa, rekonstrukcja i konserwacja
   organmistrzostwa    organów i części zamiennych do organów (z
                       wyłączeniem organów elektronicznych). Montaż,
                       instalowanie i strojenie organów. Naprawa
                       fisharmonii.
62 usługowy wyrób i    Wyrób, przeróbka, naprawa, renowacja i
   naprawa instrumen-  konserwacja oraz strojenie, przestrajanie i
   tów dętych          korekta muzycznych instrumentów dętych
                       blaszanych i drewnianych. Wyrób, przeróbka i
                       montowanie części zespołów i podzespołów do
                       instrumentów muzycznych dętych blaszanych i
                       drewnianych. Polerowanie muzycznych
                       instrumentów dętych. Naprawa pulpitów oraz
                       futerałów do instrumentów dętych.
63 usługowy wyrób i   Wyrób, przeróbka, naprawa, konserwacja i
   naprawa instrumen- strojenie oraz korekta instrumentów
   tów perkusyjnych    perkusyjnych. Wyrób, przeróbka i montowanie
                       części detali, zespołów i podzespołów do
                       instrumentów perkusyjnych. Naprawa futerałów,
                       pulpitów i nośników do instrumentów
                       perkusyjnych.
64 usługowy wyrób i    Wyrób, przeróbka, naprawa, korekta,
   naprawa instrumen-  przestrajanie i strojenie instrumentów, np.:
   tów harmoniowych    fortepiany, pianina, klawikordy, klawesyny,
   i fortepianowych    pianole, fisharmonie oraz wszelkiego rodzaju
                       harmonie, akordeony i harmonijki ustne.
                       Wytwarzanie, przeróbka i montowanie części,
                       zespołów i podzespołów do instrumentów
                       harmoniowych i fortepianowych. Naprawa
                       pulpitów, futerałów i nośników do
                       instrumentów harmoniowych i fortepianowych.
65 usługowy wyrób i    Wyrób, przeróbka, naprawa, konserwacja oraz
   naprawa instrumen-  strojenie, przestrajanie i korekta
   tów muzycznych      instrumentów elektrycznych i elektronicznych,
   elektrycznych       np. klawisety, organy, gitary
   i elektronicznych   elektroakustyczne itp. Wykonywanie, przeróbka
                       i montaż części, zespołów i podzespołów
                       elektronicznych do naprawianych instrumentów.
                       Naprawa aparatury nagłaśniającej i sprzętu
                       elektroakustycznego (np.: wzmacniaczy,
                       głośników, mikrofonów, kamer pogłosowych,
                       mikserów wielokanałowych stereofonicznych
                       itp.) związanych z instrumentem. Sprawdzanie
                       i przystosowywanie poprawności działania
                       instrumentów elektroakustycznych oraz
                       aparatury nagłaśniającej.
                               
66 ortopedyka "*"      Wykonywanie wszelkich prac z zakresu
                       mechaniki ortopedycznej i bandażownictwa.
                       Wykonywanie i naprawa protez kończyn dolnych
                       i górnych, aparatów, gorsetów ortopedycznych,
                       pasów brzusznych i przepuklinowych, zawieszeń
                       protez, aparatów ortopedycznych itp.
                       Wykonywanie części do protez, aparatów,
                       gorsetów ortopedycznych itp.
67 usługi w zakresie   Lakierowanie powierzchni przedmiotów oraz
   lakiernictwa        zabezpieczanie materiałami antykorozyjnymi
                       konstrukcji metalowych, z wyłączeniem środków
                       transportu i obiektów budowlanych.
                       Lakierowanie mebli oraz stolarki i elementów
                       budowlanych, np. boazerii, balustrad,
                       lamperii itp. (nie związanych trwale z
                       budynkiem). Renowacja powłok lakierniczych.
                       Lakierowanie części, elementów zespołów,
                       przedmiotów przez zanurzanie.
68 usługowe wytwarza-  Wytwarzanie szyldów, tablic, wywieszek,
   nie szyldów         reklam, znaków firmowych, godeł i szablonów
                       na różnych podkładach (blacha, szkło, drewno
                       sztuczne, płótno i inne), w tym również
                       techniką złocenia i srebrzenia.
69 konserwacja i       Wykonywanie prac konserwatorskich i napraw
   naprawa przedmiotów wszelkiego rodzaju przedmiotów artystycznych
   zabytkowych         uznanych za zabytkowe, np.: obrazy, ramy,
   i artystycznych     rzeźby, meble, przedmioty muzealne, broń,
                       zbroje, zegary, tkaniny, stroje itp. Inne
                       prace i czynności towarzyszące, niezbędne do
                       całkowitego wykonania świadczonej usługi.
70 usługi w zakresie   Wykonywanie, naprawa i rozbiórka murowanych
   robót budowlanych:  obiektów budowlanych, ich części lub
   murarskich,         elementów, np. murów i stropów z różnych
   ciesielskich,       materiałów. Wykonywanie i naprawa ław
   dekarskich,         fundamentowych, tynków wewnętrznych i
   posadzkarskich,     zewnętrznych, elewacji. Wykonywanie, naprawa
   malarskich, związa- i rozbiórka ogrodzeń i kanałów murowanych i
   nych z wykładaniem  betonowych itp. Wykonywanie, konserwacja
   i tapetowaniem      i rozbiórka kominów. Układanie płytek
   ścian, izolator-    ceramicznych itp. Osadzanie w murach i
   skich, związanych   ściankach stolarki i ślusarki budowlanej.
   ze wznoszeniem i    Wykonywanie podłoży pod posadzki oraz
   montażem kon-       posadzek betonowych i cementowych.
   strukcji stalowych  Wykonywanie rusztowań. Wykonywanie wykopów
                       pod obiekty budowlane.
   - dla ludności      Wykonywanie, przebudowa, naprawa i rozbiórka
                       drewnianych obiektów budowlanych, konstrukcji
                       dachowych, mostków, ogrodzeń drewnianych.
                       Wykonywanie napraw i konserwacji drewnianych
                       ścian, pokryć dachowych i stropów, podłóg z
                       desek i schodów. Wykonywanie konstrukcji
                       drewnianych - trybun, pomostów estradowych,
                       przystani i drewnianych elementów terenów
                       osiedlowych i placów zabaw dziecięcych.
                       Deskowanie i stemplowanie konstrukcji
                       betonowych i żelbetowych. Montaż i demontaż
                       rusztowań i pomostów roboczych. Wykonywanie
                       deskowań (szalunków) i zabezpieczeń przy
                       robotach ziemnych. Zabezpieczanie elementów
                       drewnianych impregnatami, środkami malarskimi
                       i ognioochronnymi.
                       Krycie i naprawa wszelkiego rodzaju dachów
                       przy zastosowaniu tradycyjnych i
                       współczesnych pokryć dachowych. Wykonywanie
                       napraw i konserwacji drewnianych konstrukcji
                       dachowych. Zakładanie izolacji dachowych.
                       Impregnacja elementów drewnianych budynków.
                       Zakładanie, naprawa i konserwacja urządzeń
                       odwadniających. Naprawa i wykonywanie (na
                       własne potrzeby) rynien dachowych i elementów
                       blacharskich. Zakładanie i naprawa ław
                       kominowych. Zabezpieczanie antykorozyjne
                       pokryć dachowych i elementów blacharskich
                       oraz innych stalowych elementów dachów.
                       Montaż żaluzji, folii antywłamaniowych i
                       przeciwsłonecznych.
                       Układanie, remont i konserwacja podłóg i
                       posadzek z różnych materiałów, jak: drewno,
                       kamień, ceramika, tworzywo sztuczne, beton
                       itp. Przygotowanie podłoży pod posadzki.
                       Układanie płytek ceramicznych na ścianach.
                       Malowanie elewacji i wnętrz budynków przy
                       zastosowaniu różnych technik malarskich.
                       Przygotowanie podłoży. Malowanie urządzeń w
                       budynkach. Lakierowanie otworów i elementów
                       budowlanych, mebli ściennych itp. Nakładanie
                       różnego rodzaju tapet. Przygotowanie podłoży
                       pod tapety. Malowanie antykorozyjne obiektów
                       budowlanych o konstrukcji metalowej, np.
                       ogrodzeń, latarni, słupów wysokiego napięcia,
                       urządzeń energetycznych itp. Wykonywanie
                       rusztowań i zabezpieczeń bhp. Wykonywanie
                       przeciwwilgociowych, termicznych,
                       akustycznych, antykorozyjnych i biologicznych
                       izolacji budowlanych przy użyciu wszelkich
                       materiałów izolacyjnych. Przygotowanie
                       obiektu budowlanego do wykonania robót
                       izolacyjnych. Sprawdzanie i kontrola
                       działania urządzeń izolacyjnych. Usuwanie
                       uszkodzeń izolacji. Naprawa drenażu
                       odwadniającego budynki.
                       Krycie, naprawa i konserwacja dachów blachą.
                       Wykonywanie, naprawa i konserwacja wszelkich
                       elementów blacharskich. Naprawa i konserwacja
                       pokryć dachowych krytych blachą. Malowanie i
                       zabezpieczanie antykorozyjne elementów
                       dachowych.
                       Wykonywanie i montaż przewodów i elementów z
                       blachy dla instalacji wentylacyjnych i
                       klimatyzacyjnych. Naprawa i konserwacja
                       rynien dachowych, rur spustowych itp.
                       elementów. Wykonywanie i naprawa ław
                       kominowych. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania i wykończenia świadczonej usługi.
71 usługi w zakresie   Wyrób elementów budowlanych z gipsu, jak:
   sztukatorstwa       gzymsy, stiuki, sztablatury itp. Wytwarzanie
   - dla ludności      odlewów z gipsu, rzeźb, płaskorzeźb,
                       ornamentów i napisów oraz ich naprawa i
                       konserwacja. Wyrób gipsowych makiet budynków,
                       modeli i fragmentów elewacji. Wykonywanie
                       tynków zwykłych, specjalnych i szlachetnych,
                       dokonywanie ich naprawy i konserwacji.
                       Przygotowanie różnych form gipsowych. Wyrób
                       przedmiotów zdobniczych z gipsu. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania i wykończenia wyrobu
                       lub świadczonej usługi.
72 usługi w zakresie   Odgrzybianie i osuszanie wszelkiego rodzaju
   odgrzybiania        obiektów budowlanych różnymi technikami.
   budynków            Wzmacnianie odgrzybionych ścian np. poprzez
   - dla ludności      ściągi. Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania
                       świadczenia.
73 usługi w zakresie   Cyklinowanie podłóg, schodów drewnianych itp.
   cyklinowania        - ręczne lub maszynowe. Inne prace i
   - dla ludności      czynności towarzyszące, niezbędne do
                       całkowitego wykonania świadczenia.
74 usługi w zakresie   Wykonywanie, rozbiórka i naprawa ceramicznych
   zduństwa            pieców grzewczych i trzonów kuchennych.
   - dla ludności      Budowa, rozbiórka i naprawa pieców
                       piekarniczych, wędzarni, suszarni itp. Budowa
                       i naprawa pieców przenośnych (ceramicznych).
                       Budowa kominków. Wykonywanie czopuchów.
                       Remont i naprawa kanałów dymowych i kominów.
                       Inne prace i czynności towarzyszące,
                       niezbędne do całkowitego wykonania
                       świadczenia.
75 usługi w zakresie   Wykonywanie, naprawa i konserwacja wszelkiego
   robót budowlanych   rodzaju wewnętrznych i zewnętrznych
   instalacyjnych      instalacji oraz urządzeń: wodociągowych,
   związanych z zakła- kanalizacyjnych, centralnego ogrzewania,
   daniem instalacji:  ciepłej wody, klimatyzacyjnych, w tym również
   grzewczych, wenty-  przemysłowych i specjalnych (np. kuchni
   lacyjnych, klimaty- parowych, pralni mechanicznych, suszarni
   zacyjnych, wodnoka- itp.). Czyszczenie wykonanej instalacji i
   nalizacyjnych, ga-  zabezpieczenie jej przed korozją, a w razie
   zowych, elektrycz-  potrzeby położenie izolacji termicznej.
   nych i osprzętu     Wykonywanie nietypowych elementów
   elektrycznego       instalacyjnych.
   - dla ludności      Wykonywanie, naprawa i konserwacja wszelkiego
                       rodzaju wewnętrznych i zewnętrznych
                       instalacji i urządzeń gazowych, w tym również
                       przemysłowych i specjalnych (np. instalacji
                       gazów technicznych i sprężonego powietrza,
                       instalacji bezprzewodowej na gaz płynny
                       itp.). Czyszczenie instalacji i
                       zabezpieczanie antykorozyjne. Malowanie
                       instalacji gazów technicznych w kolorach
                       ustalonych przez dozór techniczny.
                       Wykonywanie nietypowych elementów
                       instalacyjnych. Wykonywanie, naprawa i
                       konserwacja: instalacji elektroenergetycznych
                       siły, światła i sygnalizacji sterowania i
                       słaboprądowych, instalacji piorunochronowej i
                       uziemiającej, linii elektroenergetycznych
                       i telefonicznych napowietrznych i kablowych,
                       rozdzielnic elektrycznych i pulpitów
                       sterujących, instalacji w stacjach "trafo" i
                       agregatów prądotwórczych. Instalowanie i
                       konserwacja odbiorników elektrycznych.
                       Pomiary i badania odbiorcze instalacji,
                       odbiorników i urządzeń elektrycznych.
                       Wykonywanie pomiarów stanu izolacji i
                       skuteczności zerowania, uziemienia itp.
                       Wykonywanie i naprawa instalacji aparatury
                       kontrolno-pomiarowej, sterowania i
                       automatyki. Inne prace i czynności
                       towarzyszące, niezbędne do całkowitego
                       wykonania świadczenia.
76 studniarstwo,       Kopanie szybów studziennych, wraz z
   z wyjątkiem         wydobywaniem urobku ręcznie lub za pomocą
   wiercenia studni    sprzętu zmechanizowanego. Wykonywanie obudowy
   o głębokości ponad  z kręgów lub murowanych z cegły i innych
   30 m                materiałów. Czyszczenie, konserwacja i
   - dla ludności      dezynfekcja studni. Wykonywanie w otworach
                       studni ekranów wodoszczelnych, np. z iłu.
                       Wykonywanie ze zbrojonego betonu pokrywy
                       studni z zamykanym włazem. Montowanie pomp
                       studziennych ręcznych lub hydroforowych.
                       Naprawa pomp studziennych. Montaż i demontaż
                       pomp głębinowych. Wykonywanie filarów
                       studziennych. Wiercenie studzien do
                       głębokości 30 m  i ich obudowa. Inne prace i
                       czynności towarzyszące, niezbędne do
                       całkowitego wykonania świadczenia.
77 obsługa maszynowa   Wykonywanie za pomocą maszyn rolniczych
   produkcji           zabiegów agrotechnicznych, np. orek,
   rolniczej i         podorywek, bronowań, wałowań, rozrzucania
   ogrodniczej         obornika, rozsiewania nawozów sztucznych,
                       siewu zbóż i traw, sadzenia ziemiopłodów,
                       opylania roślin środkami chemicznymi,
                       koszenia traw, sprzętu zbóż, wykopków
                       ziemniaków, buraków, omłotów, ustawiania
                       stert, zwożenia plonów itp.
78 usługi zwalczania   Zwalczanie szkodników roślin i zwierząt
   szkodników  roślin  wszelkiego rodzaju sposobami, np. środkami
   i zwierząt          chemicznymi, biologicznymi i mechanicznymi.
                               
79 usługi utrzymania   Zapewnienie prawidłowego utrzymywania
   i pielęgnacji       zwierząt przez przygotowanie i podawanie
   zwierząt            paszy i pojenie. Wypas zwierząt. Utrzymywanie
                       w czystości zwierząt i pomieszczeń dla
                       zwierząt. Ręczne i mechaniczne dojenie
                       zwierząt mlecznych. Wykonywanie podstawowych
                       zabiegów zootechnicznych. Pielęgnacja
                       zwierząt gospodarskich i domowych. Strzyżenie
                       psów i owiec.
80 usługi w zakresie   Pranie w wodzie i prasowanie bielizny,
   prania, prasowania  odzieży oraz firanek, zasłon itp. Prężenie
   i prężenia firanek  firanek i zasłon.
81 usługi w zakresie   Maglowanie bielizny oraz firanek, zasłon itp.
   maglowania          na gorąco i na zimno.
82 usługi w zakresie   Chemiczne czyszczenie, odplamianie oraz
   chemicznego         pranie odzieży i innych wyrobów
   czyszczenia         włókienniczych. Chemiczne czyszczenie wyrobów
   i farbowania, z     ze skóry. Barwienie, odbarwianie (wybielanie)
   wyjątkiem           tkanin, dzianin, przędzy, wyrobów odzieżowych
   czyszczenia i       itp. Apreturowanie wyrobów po praniu i
   farbowania          farbowaniu.
   wyrobów futrzarskich
83 pranie pierza i     Pranie pierza i puchu.
   puchu
84 kominiarstwo        Czyszczenie, sprawdzanie i wydawanie opinii o
                       stanie: przewodów kominowych, dymowych,
                       spalinowych i wentylacyjnych, kanałów
                       łączących paleniska z przewodami kominowymi,
                       kanałów przełazowych i nasad kominowych.
                       Badanie stanu przewodów kominowych, kominowoogrzewczych
i wentylacyjnych, kanałów oraz
                       połączeń paleniskowych w budynkach przed
                       oddaniem ich do użytku, wyszukiwanie przerw w
                       ścianach przewodów kominowych i
                       wentylacyjnych oraz sprawdzanie połączeń
                       paleniskowych w budynkach już użytkowanych.
                       Badanie przyczyn wadliwego działania
                       przewodów kominowych i kanałów. Wydawanie
                       opinii dotyczących połączenia palenisk i
                       wentylacji na podstawie przeprowadzonej
                       kontroli przewodów kominowych. Czyszczenie i
                       sprawdzanie kominów wolno stojących.
                       Czyszczenie i konserwacja wszelkiego rodzaju
                       palenisk i urządzeń grzewczych oraz pieców i
                       rur, a także kotłów niskoprężnych i
                       wysokoprężnych oraz kanałów spalinowych.
                       Czyszczenie wędzarni. Odgruzowywanie
                       przewodów kominowych i wentylacyjnych.
                       Wypalanie sadzy w przewodach kominowych.
                       Montaż i demontaż ław kominowych oraz naprawa
                       głowic kominowych.
                               
85 usługi odkażania,   Tępienie owadów w  pomieszczeniach
   tępienia owadów     mieszkalnych, biurowych, produkcyjnych,
   i odszczurzania     magazynowych itp. środkami chemicznymi.
                       Tępienie gryzoni w pomieszczeniach użytkowych
                       za pomocą środków chemicznych. Odkażanie
                       budynków, pomieszczeń inwentarskich, klatek
                       dla zwierząt hodowlanych itp. oraz obiektów
                       zagrożonych epidemiologicznie (szamb,
                       wysypisk śmieci i in.).
86 usługi fryzjerskie  Strzyżenie, upinanie i fryzowanie włosów.
   dla: kobiet i       Strzyżenie oraz pielęgnacja zarostu (brody,
i  dziewcząt oraz      wąsów). Golenie twarzy i głowy. Farbowanie i
87 mężczyzn i chłopców rozjaśnianie włosów. Farbowanie wąsów i
                       brody. Farbowanie brwi i rzęs, regulowanie
                       brwi. Komputerowy dobór fryzur. Wykonywanie
                       masażu twarzy i głowy, z wyjątkiem masaży
                       leczniczych. Usługowe ostrzenie brzytew.
                       Usługowy wyrób peruk.
88 usługi kosmetyczne  Wykonywanie zabiegów z zakresu kosmetyki
                       upiększającej: stosowanie maseczek
                       upiększających, farbowanie rzęs oraz
                       regulacja i farbowanie brwi, makijaż,
                       manicure i pedicure. Wykonywanie zabiegów z
                       zakresu kosmetyki zachowawczej, z
                       zastosowaniem odpowiednich aparatów i
                       preparatów: oczyszczanie skóry twarzy i
                       pleców, masaże pielęgnacyjne twarzy, szyi,
                       ramion, rąk, głowy itp., maski pielęgnacyjne
                       (np. ściągające, odżywcze, plastyczne i
                       ziołowe).
                       Wykonywanie zabiegów z zakresu kosmetyki
                       leczniczej, jak: lecznicze oczyszczanie skóry
                       z zaskórników, usuwanie prosaków oraz
                       wykwitów mięczaka zakaźnego, masaże
                       lecznicze, masaże refleksyjne, zabiegi
                       z dziedziny światło-, wodo- i elektroterapii,
                       wchodzące w zakres kosmetyki zachowawczej i
                       leczniczej.
89 manicure, pedicure  Nadawanie kształtu paznokciom przez skracanie
                       i opiłowanie. Wycinanie naskórka,
                       lakierowanie paznokci. Usuwanie zgrubień
                       skóry i wycinanie nagniotków.
90 usługi fotograficzne Wykonywanie w zakładzie i poza zakładem
                       fotografii, filmów                   i
                       przeźroczy: portretowych, reportażowych,
                       okolicznościowych, krajobrazowych,
                       technicznych itp. Wykonywanie fotokopii.
                       Wykonywanie prac powierzonych. Sporządzanie
                       odbitek fotograficznych na papierze,
                       porcelanie itp. Przedmiot działalności nie
                       obejmuje prac wykonywanych na kasetach
                       magnetowidowych.
                               
91 wyświetlanie        Wykonywanie odbitek rysunków, planów, tekstów
   rysunków, planów i  itp. techniką światłokopii (diazotypii) i
   tekstów techniką    kserografii.
   kserografii i
   diazotypii
92 sprzątanie wnętrz   Czyszczenie, mycie i odkurzanie pomieszczeń
                       oraz mebli. Czyszczenie i mycie urządzeń
                       sanitarnych. Czyszczenie dywanów itp.
                       Pastowanie - froterowanie podłóg.
93 ważenie osób        Ważenie osób i mierzenie wzrostu przy użyciu
                       wag osobowych.
94 usługi w zakresie   Wyznaczanie w terenie przebiegu żył wodnych,
   radiestezji         kreślenie ich głębokości, wydajności i
                       jakości wody. Wykrywanie stref zadrażnień.
                       Zabezpieczanie przed różnego rodzaju
                       szkodliwym promieniowaniem w mieszkaniach,
                       budynkach inwentarskich, zakładach pracy itp.
                       oraz zabezpieczanie szklarni i ogródków przed
                       szkodliwym dla roślin promieniowaniem.
                       
95 usługi świadczone   Obsługa gospodarstw domowych w zakresie
   w gospodarstwach    prowadzenia gospodarstwa domowego.
   domowych
   
                               
                                             Załącznik nr 5
          KWARTALNE STAWKI RYCZAŁTU OD PRZYCHODÓW
                        PROBOSZCZÓW
  Objaśnienie:
  Liczbę mieszkańców przyjmuje się według stanu na dzień
  31 grudnia roku poprzedzającego rok podatkowy, za który
  opłacany jest ryczałt, według danych właściwych organów
  administracji miast i gmin, prowadzących ewidencje
  ludności.
  
+----------------------------------------+-------------+
|   w parafiach o liczbie mieszkańców    |             |
+-------------------+--------------------+   złotych   |
|     powyżej       |          do        |             |
+-------------------+--------------------+-------------+
|                   |       1.000        |     257     |
|        1.000      |       2.000        |     292     |
|        2.000      |       3.000        |     316     |
|        3.000      |       4.000        |     344     |
|        4.000      |       5.000        |     374     |
|        5.000      |       6.000        |     408     |
|        6.000      |       7.000        |     443     |
|        7.000      |       8.000        |     479     |
|        8.000      |       9.000        |     516     |
|        9.000      |      10.000        |     560     |
|       10.000      |      12.000        |     608     |
|       12.000      |      14.000        |     661     |
|       14.000      |      16.000        |     717     |
|       16.000      |      18.000        |     777     |
|       18.000      |      20.000        |     840     |
|       20.000      |                    |     911     |
+-------------------+--------------------+-------------+
                                             Załącznik nr 6
                             
          KWARTALNE STAWKI RYCZAŁTU OD PRZYCHODÓW
                        WIKARIUSZY:
  
  Objaśnienie:
  Liczbę mieszkańców przyjmuje się według stanu na dzień
  31 grudnia roku poprzedzającego rok podatkowy, za który
  opłacany jest ryczałt, według danych właściwych organów
  administracji miast i gmin, prowadzących ewidencje
  ludności.
+-----------------------+--------------------------------------------+ 
| w parafiach o liczbie |             jeżeli siedziba                |
|     mieszkańców       |           parafii znajduje się:            |
+-----------+-----------+-------------+------------------------------+                                      
| powyżej   |   do      | na terenie  |   w mieście  o  liczbie      |
|           |           | gminy lub   |       mieszkańców            |
|           |           | miasta o    +---------------+--------------+
|           |           | liczbie     | powyżej 5.000 |   powyżej    |
|           |           | mieszkańców | do 50.000     |   50.000     |
|           |           | do 5.000    |               |              |
|           |           +-------------+---------------+--------------+
|           |           |                    w złotych               |
+-----------+-----------+-------------+---------------+--------------+
|           |   1.000   |     83      |     164       |     235      |
|   1.000   |   3.000   |    235      |     247       |     247      |
|   3.000   |   5.000   |    247      |     259       |     271      |
|   5.000   |   8.000   |    252      |     271       |     277      |
|   8.000   |  10.000   |    259      |     283       |     290      |
|  10.000   |           |    271      |     290       |     299      |       .
+-----------+-----------+-------------+---------------+--------------+"""